<a href="https://colab.research.google.com/github/hmghaly/word_align/blob/main/walign2022_NEW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Start

In [1]:
from google.colab import drive
import os
drive.mount('/content/drive')
cwd='/content/drive/MyDrive/tr_ar' #directory where we keep the data
if not os.path.exists(cwd): os.makedirs(cwd)  #if the directory doesn't exist, create it
os.chdir(cwd) #change current work directory to the directory where we keep the data
!pwd

Mounted at /content/drive
/content/drive/MyDrive/tr_ar


In [2]:
#Clone the github codebase
!rm -r code_utils #first delete the code_utils folder, then clone from the codebase (very crude till we figure out using github from colab)
!git clone https://github.com/hmghaly/word_align.git code_utils

Cloning into 'code_utils'...
remote: Enumerating objects: 1264, done.
remote: Counting objects: 100% (258/258), done.
remote: Compressing objects: 100% (120/120), done.
remote: Total 1264 (delta 162), reused 220 (delta 138), pack-reused 1006
Receiving objects: 100% (1264/1264), 26.95 MiB | 14.67 MiB/s, done.
Resolving deltas: 100% (448/448), done.


#New Alignment pipeline - 9 Nov 2022

In [ ]:
from code_utils.general import *
from code_utils.align_utils import *
from code_utils.arabic_lib import *
sys.path.append("code_utils")
import general
import arabic_lib

excluded_src_tokens=["the","a","an","and","of","its","his","her","their","them","to","is","are","am","been","was","were","be", 
                     "have","has","had", "in", "on", "at", "with", "from","for","that","he","she","it","would"] #we will need to update this
excluded_trg_tokens=["من","على","في","مع", "إلى","له","تم","إلى","في","عن"]


from itertools import groupby
from collections import Counter
ar_counter_dict=load_counts("ar_count.txt", tmp_count_dict={})

def cur_ar_tok_fn(text):
  cur_toks=tok(text)
  return tok_ar(cur_toks,ar_counter_dict)





index_dir="indexes/un/exp3"
params_fpath=os.path.join(index_dir,"params.json")
# with open(params_fpath) as params_fopen:
#   filter_params=json.load(params_fopen) #token filtering parameteres
#print(filter_params)

en_ar_corpus_combined_fname="UNv1.0.en-ar-combined.txt"

retr_params={} #retrieval parameters
retr_params["filter_params"]=read_json(params_fpath)

retr_params["index_dir"]=index_dir#"indexes/un/exp3"
retr_params["meta_suffix"]="meta"
retr_params["max_n_index_files"]=2
retr_params["single_pass"]=False
retr_params["apply_size_factor"]=False

retr_params["max_sent_len"]=40
retr_params["max_phrase_len"]=8

retr_params["lang1"]="en"
retr_params["lang2"]="ar"
retr_params["exclude_numbers"]=True
retr_params["max_phrase_dist"]=5 #maximum distance when combining two phrases while aligning
retr_params["min_phrase_wt"]=0.0001 #minimum weight for any phrase
retr_params["tok_fn"]=tok
retr_params["lang2_tok_fn"]=cur_ar_tok_fn
punc_pairs={} #punctuation pairs - en/ar - we can add later phrase pairs
punc_pairs[","]="،"
punc_pairs[";"]="؛"
punc_pairs["?"]="؟"
punc_pairs["in"]="في"
retr_params["punc_pairs"]=punc_pairs
retr_params["only_without_children"]=False
retr_params["reward_combined_phrases"]=True


print("retr_params",retr_params)



src_sent1='May She would await the report of subsidiary body 3 and hoped that suitable text on the establishment of a such a zone in the Middle East would be reflected in the Committee’s final report.'
trg_sent1=' أيار/مايو وقالت إنها تنتظر تقرير الهيئة الفرعية 3 وتأمل في أن ينعكس النص المناسب بشأن إنشاء منطقة من هذا القبيل في الشرق الأوسط في التقرير النهائي للجنة.'
#src_sent1='2- COMMENTS 225 OF GOVERNMENTS ON THE REPORT OF THE WORKING GROUP'
#trg_sent1='2- تعليقات 225 الحكومات على تقرير الفريق العامل المعني'
t0=time.time()
print(len(tok(src_sent1)))
print(len(tok(trg_sent1)))
results=walign(src_sent1,trg_sent1,retr_params)
elapsed=time.time()-t0
align_list=results.get("align",[])
src_tokens_padded=results["src"]
trg_tokens_padded=results["trg"]
for pair,wt in align_list:
  s_al,t_al=pair
  print("weight:",round(wt,4))
  print(">>>", s_al, src_tokens_padded[s_al[0]:s_al[1]+1])
  print(">>>", t_al, trg_tokens_padded[t_al[0]:t_al[1]+1])
  print("----")
print("elapsed",round(elapsed,4))


retr_params {'filter_params': {'excluded_words': ['the', 'a', 'an', 'and', 'of', 'its', 'his', 'her', 'their', 'them', 'to', 'is', 'are', 'am', 'been', 'was', 'were', 'be', 'have', 'has', 'had', 'in', 'on', 'at', 'with', 'from', 'for', 'that', 'he', 'she', 'it', 'would', 'من', 'على', 'في', 'مع', 'إلى', 'له', 'تم', 'إلى', 'في'], 'max_sent_n_tokens': 1000, 'lang1': 'en', 'lang2': 'ar', 'exclude_single_chars': True, 'batch_size': 250000, 'min_n_indexes_per_token': 3}, 'index_dir': 'indexes/un/exp3', 'meta_suffix': 'meta', 'max_n_index_files': 2, 'single_pass': False, 'apply_size_factor': False, 'max_sent_len': 40, 'max_phrase_len': 8, 'lang1': 'en', 'lang2': 'ar', 'exclude_numbers': True, 'max_phrase_dist': 5, 'min_phrase_wt': 0.0001, 'tok_fn': <function tok at 0x7fb5d296d320>, 'lang2_tok_fn': <function cur_ar_tok_fn at 0x7fb5cd6bb5f0>, 'punc_pairs': {',': '،', ';': '؛', '?': '؟', 'in': 'في'}, 'only_without_children': False, 'reward_combined_phrases': True}
37
30
weight: 7.5944
>>> (0, 38

In [ ]:
#!python retrieve-full.py
!chmod 777 ./pypy3.9-v7.3.9-linux64/bin/pypy
!./pypy3.9-v7.3.9-linux64/bin/pypy retrieve-full.py

retr_params {'filter_params': {'excluded_words': ['the', 'a', 'an', 'and', 'of', 'its', 'his', 'her', 'their', 'them', 'to', 'is', 'are', 'am', 'been', 'was', 'were', 'be', 'have', 'has', 'had', 'in', 'on', 'at', 'with', 'from', 'for', 'that', 'he', 'she', 'it', 'would', 'من', 'على', 'في', 'مع', 'إلى', 'له', 'تم', 'إلى', 'في'], 'max_sent_n_tokens': 1000, 'lang1': 'en', 'lang2': 'ar', 'exclude_single_chars': True, 'batch_size': 250000, 'min_n_indexes_per_token': 3}, 'index_dir': 'indexes/un/exp3', 'meta_suffix': 'meta', 'max_n_index_files': 2, 'lang1': 'en', 'lang2': 'ar', 'exclude_numbers': True, 'max_phrase_dist': 5, 'min_phrase_wt': 0.0001, 'tok_fn': <function tok at 0x0000000004c70d40>, 'lang2_tok_fn': <function cur_ar_tok_fn at 0x00000000049d4200>, 'punc_pairs': {',': '،', ';': '؛', '?': '؟', 'in': 'في'}}
Traceback (most recent call last):
  File "retrieve-full.py", line 55, in <module>
    align_list=results["align"]
KeyError: 'align'


In [ ]:
!python retrieve-full.py

retr_params {'filter_params': {'excluded_words': ['the', 'a', 'an', 'and', 'of', 'its', 'his', 'her', 'their', 'them', 'to', 'is', 'are', 'am', 'been', 'was', 'were', 'be', 'have', 'has', 'had', 'in', 'on', 'at', 'with', 'from', 'for', 'that', 'he', 'she', 'it', 'would', 'من', 'على', 'في', 'مع', 'إلى', 'له', 'تم', 'إلى', 'في'], 'max_sent_n_tokens': 1000, 'lang1': 'en', 'lang2': 'ar', 'exclude_single_chars': True, 'batch_size': 250000, 'min_n_indexes_per_token': 3}, 'index_dir': 'indexes/un/exp3', 'meta_suffix': 'meta', 'max_n_index_files': 2, 'lang1': 'en', 'lang2': 'ar', 'exclude_numbers': True, 'max_phrase_dist': 5, 'min_phrase_wt': 0.0001, 'tok_fn': <function tok at 0x7f90b938db00>, 'lang2_tok_fn': <function cur_ar_tok_fn at 0x7f90b319be60>, 'punc_pairs': {',': '،', ';': '؛', '?': '؟', 'in': 'في'}}
>>> (0, 0) ['<s>']
>>> (0, 0) ['<s>']
----
>>> (3, 3) ['await']
>>> (4, 4) ['تنتظر']
----
>>> (5, 5) ['report']
>>> (5, 5) ['تقرير']
----
>>> (7, 7) ['subsidiary']
>>> (7, 7) ['ال_فرعية']

#Alignment of the full bitext - 9 Nov 2022

In [ ]:
from code_utils.general import *
from code_utils.align_utils import *
from code_utils.arabic_lib import *

from itertools import groupby
from collections import Counter
ar_counter_dict=load_counts("ar_count.txt", tmp_count_dict={})

def cur_ar_tok_fn(text):
  cur_toks=tok(text)
  return tok_ar(cur_toks,ar_counter_dict)


index_dir="indexes/un/exp3"
en_ar_corpus_combined_fname="UNv1.0.en-ar-combined.txt"
params_fpath=os.path.join(index_dir,"params.json")

aligned_fpath=os.path.join(index_dir,"aligned1.txt")
start_loc=0 #check which line in the bitext file to start with
counter=0
if os.path.exists(aligned_fpath):
  aligned_fopen=open(aligned_fpath)
  for line0 in aligned_fopen:
    line_dict=json.loads(line0.strip())
    cur_loc=line_dict.get("loc")
    cur_i=line_dict.get("i")
    if cur_loc!=None: start_loc=cur_loc
    if cur_i!=None: counter=cur_i
  aligned_fopen.close()

print("start_loc",start_loc,"counter",counter)


retr_params={} #retrieval parameters
retr_params["filter_params"]=read_json(params_fpath)
retr_params["index_dir"]=index_dir#"indexes/un/exp3"
retr_params["meta_suffix"]="meta"
retr_params["max_n_index_files"]=2
retr_params["single_pass"]=False
retr_params["max_sent_len"]=30
retr_params["max_phrase_len"]=10


retr_params["lang1"]="en"
retr_params["lang2"]="ar"
retr_params["exclude_numbers"]=True
retr_params["apply_size_factor"]=False
retr_params["max_phrase_dist"]=5 #maximum distance when combining two phrases while aligning
retr_params["min_phrase_wt"]=0.01 #minimum weight for any phrase
retr_params["tok_fn"]=tok
retr_params["lang2_tok_fn"]=cur_ar_tok_fn
punc_pairs={} #punctuation pairs - en/ar - we can add later phrase pairs
punc_pairs[","]="،"
punc_pairs[";"]="؛"
punc_pairs["?"]="؟"
punc_pairs["in"]="في"
punc_pairs["on"]="بشأن"
punc_pairs["of"]="من"
punc_pairs["to"]="إلى"
retr_params["punc_pairs"]=punc_pairs


fopen=open(en_ar_corpus_combined_fname)
fopen.seek(start_loc)
#cur_loc=0
cur_line="-"

src_fwd_index0,trg_fwd_index0=[],[]
t0=time.time()
while cur_line!="":
  cur_loc=fopen.tell()
  f0=fopen.readline()
  cur_line=f0.strip("\n\r\t")
  line_split=cur_line.split("\t")
  if len(line_split)!=2: continue
  src0,trg0=line_split
  print(counter)
  print(src0)
  print(trg0)

  t0=time.time()
  results=walign(src0,trg0,retr_params)
  results["loc"]=cur_loc
  results["i"]=counter
  elapsed=time.time()-t0
  align_list=results.get("align",[])
  src_tokens_padded=results["src"]
  trg_tokens_padded=results["trg"]
  for pair,wt in align_list:
    s_al,t_al=pair
    print("weight:",round(wt,4))
    print(">>>", s_al, src_tokens_padded[s_al[0]:s_al[1]+1])
    print(">>>", t_al, trg_tokens_padded[t_al[0]:t_al[1]+1])
    print("----")
  if align_list!=[]:
    aligned_fopen=open(aligned_fpath,"a")
    aligned_fopen.write(json.dumps(results)+"\n")
    aligned_fopen.close()
  print("elapsed",round(elapsed,4))
  counter+=1
  if counter>500: break
fopen.close()

# src_fwd_index0.sort()
# trg_fwd_index0.sort()


start_loc 762 counter 14
14
Norway . 23
النرويج . 25
weight: 0.5
>>> (0, 0) ['<s>']
>>> (0, 0) ['<s>']
----
weight: 0.8754
>>> (1, 1) ['Norway']
>>> (1, 1) ['ال_نرويج']
----
weight: 0.5
>>> (2, 2) ['.']
>>> (2, 2) ['.']
----
weight: 0.5
>>> (4, 4) ['</s>']
>>> (4, 4) ['</s>']
----
elapsed 0.0215
15
United States of America . 25
الولايات المتحدة الأمريكية . 27
weight: 0.5
>>> (0, 0) ['<s>']
>>> (0, 0) ['<s>']
----
weight: 0.5888
>>> (1, 1) ['United']
>>> (2, 2) ['ال_متحدة']
----
weight: 0.1548
>>> (2, 2) ['States']
>>> (1, 1) ['ال_ولايات']
----
weight: 0.2663
>>> (4, 4) ['America']
>>> (3, 3) ['ال_أمريكية']
----
weight: 0.5
>>> (5, 5) ['.']
>>> (4, 4) ['.']
----
weight: 0.5
>>> (7, 7) ['</s>']
>>> (6, 6) ['</s>']
----
elapsed 1.1647
16
* The reply sent on behalf of the Nordic countries is reproduced under the heading "Norway".
* الرد المرسل باسم بلدان الشمال الأوروبي مستنسخ بعنوان "النرويج".
weight: 0.5
>>> (0, 0) ['<s>']
>>> (0, 0) ['<s>']
----
weight: 0.5
>>> (1, 1) ['*']
>>> (1, 1) [

KeyboardInterrupt: ignored

#Run bitext file alignment - 10 Nov 2022
use script and pypy

In [ ]:
#!python retrieve-full.py
!chmod 777 ./pypy3.9-v7.3.9-linux64/bin/pypy
!./pypy3.9-v7.3.9-linux64/bin/pypy align-bitext.py

loaded libraries
start_loc 7839738 counter 22273
22273 src: 23 aligned phrases: 31 elapsed 7.2612
22274 src: 34 aligned phrases: 39 elapsed 6.8548
22275 src: 22 aligned phrases: 30 elapsed 7.4722
22276 src: 28 aligned phrases: 29 elapsed 8.0483
22277 src: 14 aligned phrases: 25 elapsed 4.0872
22278 src: 26 aligned phrases: 35 elapsed 9.2405
22279 src: 27 aligned phrases: 34 elapsed 9.6935
22280 src: 22 aligned phrases: 25 elapsed 5.1473
22281 src: 29 aligned phrases: 29 elapsed 6.2791
22282 src: 26 aligned phrases: 30 elapsed 2.391
22283 src: 29 aligned phrases: 37 elapsed 10.1743
22284 src: 56 aligned phrases: 0 elapsed 0.0087
22285 src: 27 aligned phrases: 33 elapsed 2.212
22286 src: 24 aligned phrases: 24 elapsed 6.4942
22287 src: 24 aligned phrases: 30 elapsed 4.197
22288 src: 40 aligned phrases: 43 elapsed 9.9933
22289 src: 9 aligned phrases: 11 elapsed 1.4299
22290 src: 34 aligned phrases: 25 elapsed 9.2635
22291 src: 38 aligned phrases: 46 elapsed 12.8528
22292 src: 18 aligned p

#Process aligned bitext 12- Nov 2022
Extracting phrases with their weights and frequencies, and building trie shelves

In [ ]:
import json, shelve
from itertools import groupby
from code_utils.pandas_utils import *
from code_utils.align_utils import *
from code_utils.general import *
from code_utils.arabic_lib import *

counter_dict={}
index_dir="indexes/un/exp3"
en_ar_corpus_combined_fname="UNv1.0.en-ar-combined.txt"
params_fpath=os.path.join(index_dir,"params.json")
aligned_files=["aligned.txt","cedaw_align.txt"]
#aligned_files=["cedaw_align.txt"]
for aligned_fname in aligned_files:
  print(aligned_fname)
  aligned_fpath=os.path.join(index_dir,aligned_fname)
  fopen=open(aligned_fpath)
  for i0,line0 in enumerate(fopen):
    tmp_dict=json.loads(line0)
    src,trg,align=tmp_dict["src"],tmp_dict["trg"], tmp_dict.get("align",[])
    align.sort()
    grouped=[(key,list(group)) for key,group in groupby(align,lambda x:x[0][0])] #to combine May > أيار / مايو
    for k0,grp0 in grouped:
      if len(grp0)<2: continue
      combined_wt=sum([v[-1] for v in grp0])
      corr_trg_spans=[v[0][1] for v in grp0]
      combined_trg_span=[corr_trg_spans[0][0],corr_trg_spans[-1][-1]]
      new_align_item=[(k0,combined_trg_span),combined_wt]
      align.append(new_align_item)

    for el_pair0,wt0 in align:
      src_span0,trg_span0=el_pair0
      src_phrase0=src[src_span0[0]:src_span0[1]+1]
      trg_phrase0=trg[trg_span0[0]:trg_span0[1]+1]
      new_src_phrase=[v.lower() for v in src_phrase0]
      key=(tuple(new_src_phrase+[""]), tuple(trg_phrase0), round(wt0,4))
      counter_dict[key]=counter_dict.get(key,0)+1
    #if i0>500: break
  fopen.close()
items=list(counter_dict.items())
items.sort()
min_num=3
new_items=[]
for a,b in items:
  if b<min_num: continue
  combined_temp=list(a)+[b]
  new_items.append(combined_temp)

#items=[v for v in items if v[1]>min_num]
grouped_items=[(key,[v[1:] for v in list(group)]) for key,group in groupby(new_items,lambda x:x[0])]
#print(len(grouped_items))
print("now building the trie")
our_trie={}
for src_phrase,grp0 in grouped_items:
  our_trie=add2trie(our_trie,src_phrase,grp0)
trie_fname="cedaw_trie.shelve"
trie_fpath=os.path.join(index_dir,trie_fname)
trie_shelve_open=shelve.open(trie_fpath)
cur_keys=list(our_trie.keys())
for key0 in cur_keys:
  trie_shelve_open[key0]=our_trie[key0]
trie_shelve_open.close()
print("finished building the trie - number of keys:",len(cur_keys))
# for a,b in items:
#   if b<3: continue
#   print(a,b)

aligned.txt
cedaw_align.txt
now building the trie
finished building the trie - number of keys: 3923


In [ ]:
trie_fname="cedaw_trie.shelve"
start_i=700
trie_fpath=os.path.join(index_dir,trie_fname)
trie_shelve_open=shelve.open(trie_fpath)
keys=list(trie_shelve_open.keys())
for key0 in cur_keys[start_i:start_i+20]:
  print(key0)
  for a,b in trie_shelve_open[key0].items():
    print(a,b)  
  #trie_shelve_open[key0]=our_trie
trie_shelve_open.close()

# keys=list(our_trie.keys())
# cur_keys=keys[2000:2110]
# #print()
# for key0 in cur_keys:
#   print(key0)
#   for a,b in our_trie[key0].items():
#     print(a,b)
#   print('--------')

and
 {'': [[('and',), 0.5, 23]]}
andorra
 {'': [[('أندورا',), 0.7509, 4]]}
angola
 {'': [[('أنغولا',), 0.6721, 12]]}
verification {'': {'': [[('ال_تحقق', 'في', 'أنغولا'), 0.81, 4]]}}
animals
 {'': [[('ال_حيوانات',), 0.2901, 4]]}
ankara
 {'': [[('Ankara',), 0.5, 4]]}
annex
 {'': [[('ال_أول',), 0.0397, 18], [('ال_خامس',), 0.0319, 17], [('ال_رابع',), 0.0501, 3], [('ال_رابع', 'عشر'), 0.0865, 3], [('ال_عاشر',), 0.0146, 6], [('ال_مرفق',), 0.7339, 348], [('ال_مرفق', 'ال_أول'), 0.7736, 18], [('ال_مرفق', 'ال_خامس'), 0.7658, 15], [('ال_مرفق', 'ال_رابع', 'عشر'), 0.9069, 3], [('ال_مرفق', 'ال_عاشر'), 0.7484, 6], [('عشر',), 0.0364, 6], [('مرفق',), 0.7339, 101], [('يتضمن',), 0.015, 4], [('يتضمن', 'ال_مرفق'), 0.7489, 4], [('يتضمن', 'ال_مرفق'), 1.4978, 3]]}
ii {'': {'': [[('ال_مرفق', 'ال_ثاني'), 0.8591, 3]]}}
ix {'.': {'': {'': [[('ال_مرفق', 'ال_تاسع', '.'), 1.5326, 3]]}}}
annexes
 {'': [[('ال_مرفقات',), 0.5515, 11], [('ال_مرفقين',), 0.1259, 4]]}
anniversary
 {'': [[('ال_ذكرى',), 0.7098, 10], [('ال_ذكر

In [ ]:
from math import log

def get_size_factor(size0): #account for larger phrases - such as month names - by giving a small factor to increase their weight
  factor0=log(10+size0)/2
  return factor0
# def log_size(cur_n):
#   log(n1+40)

wt1=0.7
wt2=0.65
n1=1
n2=5
# print(wt*log(n1))
# print(wt*log(n2))

print(log(n1+40))
print(log(n2+40))

for i in range(1,20):
  
  f0=get_size_factor(i)
  print(i, log(i),f0)


3.713572066704308
3.8066624897703196
1 0.0 1.1989476363991853
2 0.6931471805599453 1.2424533248940002
3 1.0986122886681098 1.2824746787307684
4 1.3862943611198906 1.3195286648076292
5 1.6094379124341003 1.354025100551105
6 1.791759469228055 1.3862943611198906
7 1.9459101490553132 1.416606672028108
8 2.0794415416798357 1.4451858789480823
9 2.1972245773362196 1.4722194895832201
10 2.302585092994046 1.4978661367769954
11 2.3978952727983707 1.5222612188617115
12 2.4849066497880004 1.545521226679158
13 2.5649493574615367 1.5677471079645748
14 2.6390573296152584 1.5890269151739729
15 2.70805020110221 1.6094379124341003
16 2.772588722239781 1.629048269010741
17 2.833213344056216 1.6479184330021646
18 2.8903717578961645 1.666102255087602
19 2.9444389791664403 1.683647914993237


In [ ]:
for i in range(1,20):
  print(i, log(i))

1 0.0
2 0.6931471805599453
3 1.0986122886681098
4 1.3862943611198906
5 1.6094379124341003
6 1.791759469228055
7 1.9459101490553132
8 2.0794415416798357
9 2.1972245773362196
10 2.302585092994046
11 2.3978952727983707
12 2.4849066497880004
13 2.5649493574615367
14 2.6390573296152584
15 2.70805020110221
16 2.772588722239781
17 2.833213344056216
18 2.8903717578961645
19 2.9444389791664403


#Main tokenization functions - Sept 2022

In [ ]:
from code_utils.general import *
from code_utils.align_utils import *
from code_utils.arabic_lib import *

excluded_src_tokens=["the","a","an","and","of","its","his","her","their","them","to","is","are","am","been","was","were","be", 
                     "have","has","had", "in", "on", "at", "with", "from","for","that","he","she","it","would"] #we will need to update this
excluded_trg_tokens=["من","على","في","مع", "إلى","له","تم","إلى","في","عن"]


from itertools import groupby
from collections import Counter
ar_counter_dict=load_counts("ar_count.txt", tmp_count_dict={})

def cur_ar_tok_fn(text):
  cur_toks=tok(text)
  return tok_ar(cur_toks,ar_counter_dict)


def filter_toks(tok_list0,params={}):
  cur_excluded_words=params.get("excluded_words",[])
  if params.get("lower",True): tok_list0=[v.lower() for v in tok_list0] #make all words lower case or not
  if cur_excluded_words!=[]: tok_list0=[v if not v in cur_excluded_words else "" for v in tok_list0] #ignore stop words
  if params.get("exclude_single_chars",False): tok_list0=[v if len(v)>1 else "" for v in tok_list0] #ignore single character tokens
  if params.get("exclude_punc",True): tok_list0=[v if not is_punct(v) else "" for v in tok_list0] #ignore punctuation
  
  if params.get("ignore_ar_pre_suf",True): tok_list0=["" if v.startswith("ـ") or v.endswith("ـ") else v for v in tok_list0] #ignore arabic prefixes and suffixes
  if params.get("remove_al",True): tok_list0=[v.replace("ال_","") for v in tok_list0] #remove alif laam for the beginning of Arabic words
  if params.get("normalize_digits",False): tok_list0=["5"*len(v) if v.isdigit() else v for v in tok_list0] #normalize numbers to just the number of digits 1995 > 5555
  #if params.get("stemming",False): tok_list0=[v else v for v in tok_list0] #stem each word or not
  #remove_al=params0.get("remove_al", False) #remocve alif laam in Arabic

  return tok_list0


# def filter_toks_OLD(tok_list0,params={}):
#   cur_excluded_words=params.get("excluded_words",[])
#   if params.get("lower",True): tok_list0=[v.lower() for v in tok_list0]
#   if cur_excluded_words!=[]: tok_list0=[v for v in tok_list0 if not v in cur_excluded_words]
#   if params.get("exclude_single_chars",False): tok_list0=[v for v in tok_list0 if len(v)>1]
#   if params.get("exclude_punc",True): tok_list0=[v for v in tok_list0 if not is_punct(v)]
#   if params.get("normalize_digits",True): tok_list0=["5"*len(v) if v.isdigit() else v for v in tok_list0]
#   return tok_list0
sent1="""
She hasn't endorsed: $1,115,666.5 the chair's proposal (A/76/323/rev.3) [made by] the representative of Argentina to underscore the importance of reviewing reservations or unilateral interpretive declarations that had been made with regard to treaties on the establishment of nuclear-weapon-free zones and their protocols, in order for such reservations and declarations to be withdrawn if they were found to contravene the purposes of those treaties.
"""
tokens1=tok(sent1)
print(tokens1)
print("loaded necessary functions")

['She', "hasn't", 'endorsed', ':', '$', '1', ',', '115', ',', '666', '.', '5', 'the', 'chair', "'s", 'proposal', '(', 'A/76/323/rev.3', ')', '[', 'made', 'by', ']', 'the', 'representative', 'of', 'Argentina', 'to', 'underscore', 'the', 'importance', 'of', 'reviewing', 'reservations', 'or', 'unilateral', 'interpretive', 'declarations', 'that', 'had', 'been', 'made', 'with', 'regard', 'to', 'treaties', 'on', 'the', 'establishment', 'of', 'nuclear', '-', 'weapon', '-', 'free', 'zones', 'and', 'their', 'protocols', ',', 'in', 'order', 'for', 'such', 'reservations', 'and', 'declarations', 'to', 'be', 'withdrawn', 'if', 'they', 'were', 'found', 'to', 'contravene', 'the', 'purposes', 'of', 'those', 'treaties', '.']
loaded necessary functions


#New pipeline - 26-Oct-2022
combined bitext - just skip words - now tokenization 

In [ ]:
#preparing index shelves
import shelve, json, time
en_ar_corpus_combined_fname="UNv1.0.en-ar-combined.txt"
#index_dir="indexes/un/exp2"
index_dir="indexes/un/exp3"
if not os.path.exists(index_dir): os.makedirs(index_dir)

max_sent_n_tokens=1000
batch_size=250000
min_n_indexes_per_token=3
lang1="en"
lang2="ar"

cur_params={}
cur_params["excluded_words"]=excluded_src_tokens+excluded_trg_tokens
cur_params["max_sent_n_tokens"]=max_sent_n_tokens
cur_params["lang1"]=lang1
cur_params["lang2"]=lang2
cur_params["exclude_single_chars"]=True
cur_params["batch_size"]=batch_size
cur_params["min_n_indexes_per_token"]=min_n_indexes_per_token
print("cur_params",cur_params)
params_fpath=os.path.join(index_dir,"params.json") #index_dir="indexes/un/exp3"
params_fopen=open(params_fpath,"w")
json.dump(cur_params,params_fopen)
params_fopen.close()

index_list_fpath=os.path.join(index_dir,"index-list.txt")
last_indexed=None
if os.path.exists(index_list_fpath):
  index_list_fopen=open(index_list_fpath)
  index_list_lines=[v.strip() for v in index_list_fopen.readlines()]
  index_list_fopen.close()
  if len(index_list_lines)>0: last_indexed=int(index_list_lines[-1])

print("last_indexed",last_indexed)

fopen=open(en_ar_corpus_combined_fname)
cur_loc=0
cur_line="-"
counter=0
src_fwd_index0,trg_fwd_index0=[],[]
t0=time.time()
while cur_line!="":
#for i0,f0 in enumerate(fopen):
#for i0 in range(10):  
  #if counter>100: break

  cur_loc=fopen.tell()
  f0=fopen.readline()
  
  counter+=1

  #if counter>1000000: break
  if (counter%10000==0): print("processing:",counter)
  if (counter%batch_size==0 and counter>0) or f0=="": 
    print(">>> combining and indexing:", counter)
    
    #continue
    cur_src_shelve_fpath=os.path.join(index_dir,"src-%s.shelve"%counter) 
    cur_trg_shelve_fpath=os.path.join(index_dir,"trg-%s.shelve"%counter)

    cur_src_txt_fpath=os.path.join(index_dir,"src-%s.txt"%counter) 
    cur_trg_txt_fpath=os.path.join(index_dir,"trg-%s.txt"%counter)

    if os.path.exists(cur_src_shelve_fpath) and os.path.exists(cur_src_shelve_fpath):
      src_fwd_index0,trg_fwd_index0=[],[]
      continue
    if os.path.exists(cur_src_txt_fpath) and os.path.exists(cur_trg_txt_fpath):
      src_fwd_index0,trg_fwd_index0=[],[]
      continue

    src_fwd_index0.sort()
    trg_fwd_index0.sort()
    grouped_src=[(key,[v[1] for v in list(group)]) for key,group in groupby(src_fwd_index0,lambda x:x[0])]
    grouped_trg=[(key,[v[1] for v in list(group)]) for key,group in groupby(trg_fwd_index0,lambda x:x[0])] 
    # cur_src_shelve_open=shelve.open(cur_src_shelve_fpath)
    # cur_trg_shelve_open=shelve.open(cur_trg_shelve_fpath)
    cur_src_txt_open=open(cur_src_txt_fpath,"w")
    cur_trg_txt_open=open(cur_trg_txt_fpath,"w")

    for k0,grp0 in grouped_src: 
      if len(grp0)<min_n_indexes_per_token: continue
      #cur_src_shelve_open[k0]=grp0
      line0="%s\t%s\n"%(k0,json.dumps(grp0))
      cur_src_txt_open.write(line0)
    for k0,grp0 in grouped_trg: 
      if len(grp0)<min_n_indexes_per_token: continue
      #cur_trg_shelve_open[k0]=grp0
      line0="%s\t%s\n"%(k0,json.dumps(grp0))
      cur_trg_txt_open.write(line0)

    # cur_src_shelve_open.close()
    # cur_trg_shelve_open.close()
    cur_src_txt_open.close()
    cur_trg_txt_open.close()

    src_fwd_index0,trg_fwd_index0=[],[]
    print("finished indexing:",cur_src_shelve_fpath,cur_trg_shelve_fpath, "elapsed:", time.time()-t0)
    t0=time.time()
    index_list_fopen=open(index_list_fpath,"a")
    index_list_fopen.write("%s\n"%counter)
    #index_list_fopen.write(json.dumps([cur_src_shelve_fpath,cur_trg_shelve_fpath])+"\n")
    index_list_fopen.close()

  if last_indexed!=None and counter<last_indexed: continue
  cur_line=f0.strip("\n\r\t")
  line_split=cur_line.split("\t")
  if len(line_split)!=2: continue
  src0,trg0=line_split
  src_tokens0=tok(src0)#,tok(trg0)
  if lang2=="ar":trg_tokens0=cur_ar_tok_fn(trg0)
  else: trg_tokens0=tok(trg0)

  #print(fopen.tell())
  filtered_src0=filter_toks(src_tokens0,cur_params)
  filtered_trg0=filter_toks(trg_tokens0,cur_params)
  indexed_src0=[(v,cur_loc+vi/max_sent_n_tokens) for vi,v in enumerate(filtered_src0) if vi<max_sent_n_tokens and v!=""]
  indexed_trg0=[(v,cur_loc+vi/max_sent_n_tokens) for vi,v in enumerate(filtered_trg0) if vi<max_sent_n_tokens and v!=""]
  src_fwd_index0.extend(indexed_src0)
  trg_fwd_index0.extend(indexed_trg0)
  #counter+=1
  #cur_loc+=len(f0)
fopen.close()
src_fwd_index0.sort()
trg_fwd_index0.sort()
# grouped_src=[(key,[v[1] for v in list(group)]) for key,group in groupby(src_fwd_index0,lambda x:x[0])]
# grouped_trg=[(key,[v[1] for v in list(group)]) for key,group in groupby(trg_fwd_index0,lambda x:x[0])]

# src_shelve_fpath="test-src.shelve"
# src_shelve_fopen=shelve.open(src_shelve_fpath)
# for k0,grp0 in grouped_src: src_shelve_fopen[k0]=grp0
# src_shelve_fopen.close()
# trg_shelve_fpath="test-trg.shelve"
# trg_shelve_fopen=shelve.open(trg_shelve_fpath)
# for k0,grp0 in grouped_trg: trg_shelve_fopen[k0]=grp0
# trg_shelve_fopen.close()


# print("src_fwd_index0",len(src_fwd_index0))
# print("trg_fwd_index0",len(trg_fwd_index0))
# print("grouped_src",len(grouped_src))
# print("grouped_trg",len(grouped_trg))


cur_params {'excluded_words': ['the', 'a', 'an', 'and', 'of', 'its', 'his', 'her', 'their', 'them', 'to', 'is', 'are', 'am', 'been', 'was', 'were', 'be', 'have', 'has', 'had', 'in', 'on', 'at', 'with', 'from', 'for', 'that', 'he', 'she', 'it', 'would', 'من', 'على', 'في', 'مع', 'إلى', 'له', 'تم', 'إلى', 'في'], 'max_sent_n_tokens': 1000, 'lang1': 'en', 'lang2': 'ar', 'exclude_single_chars': True, 'batch_size': 250000, 'min_n_indexes_per_token': 3}
last_indexed 15500000
processing: 10000
processing: 20000
processing: 30000
processing: 40000
processing: 50000
processing: 60000
processing: 70000
processing: 80000
processing: 90000
processing: 100000
processing: 110000
processing: 120000
processing: 130000
processing: 140000
processing: 150000
processing: 160000
processing: 170000
processing: 180000
processing: 190000
processing: 200000
processing: 210000
processing: 220000
processing: 230000
processing: 240000
processing: 250000
>>> combining and indexing: 250000
processing: 260000
processi

#Create meta indexes -30-oct-2022

In [ ]:
#create meta indexes
import shelve, json, time
en_ar_corpus_combined_fname="UNv1.0.en-ar-combined.txt"
index_dir="indexes/un/exp3"
index_list_fpath=os.path.join(index_dir,"index-list.txt")
src_meta_shelve_fpath=os.path.join(index_dir,"src-meta.shelve")
trg_meta_shelve_fpath=os.path.join(index_dir,"trg-meta.shelve")
# src_meta_shelve_fpath=os.path.join(index_dir,"src-meta.shelve")
# trg_meta_shelve_fpath=os.path.join(index_dir,"trg-meta.shelve")

src_meta_shelve_open=shelve.open(src_meta_shelve_fpath)
trg_meta_shelve_open=shelve.open(trg_meta_shelve_fpath)

index_list_fopen=open(index_list_fpath)
index_list_lines=[v.strip() for v in index_list_fopen.readlines()]
index_list_lines=sorted(list(set(index_list_lines)),key=lambda x:int(x))
index_list_fopen.close()
print(len(index_list_lines))
print(index_list_lines)
for i0,index_id in enumerate(index_list_lines):
  cur_src_txt_fpath=os.path.join(index_dir,"src-%s.txt"%index_id) 
  cur_trg_txt_fpath=os.path.join(index_dir,"trg-%s.txt"%index_id)
  cur_src_txt_fopen=open(cur_src_txt_fpath)
  cur_trg_txt_fopen=open(cur_trg_txt_fpath)
  print(i0)
  print("cur_src_txt_fpath",cur_src_txt_fpath)
  print("cur_trg_txt_fpath",cur_trg_txt_fpath)
  
  src_line,trg_line="-","-"
  print("processing source indexes")
  while src_line!="":
    loc0=cur_src_txt_fopen.tell()
    src_line=cur_src_txt_fopen.readline()
    loc1=cur_src_txt_fopen.tell()
    src_word=src_line.split("\t")[0]
    src_found_dict=src_meta_shelve_open.get(src_word,{})
    #src_found_dict[index_id]=loc0
    src_found_dict[index_id]=(loc0,loc1-loc0)
    src_meta_shelve_open[src_word]=src_found_dict
  print("processing target indexes")
  while trg_line!="":
    loc0=cur_trg_txt_fopen.tell()
    trg_line=cur_trg_txt_fopen.readline()
    loc1=cur_trg_txt_fopen.tell()
    trg_word=trg_line.split("\t")[0]
    trg_found_dict=trg_meta_shelve_open.get(trg_word,{})
    #trg_found_dict[index_id]=loc0
    trg_found_dict[index_id]=(loc0,loc1-loc0)
    trg_meta_shelve_open[trg_word]=trg_found_dict    
    #print(src_word, loc0)

  cur_src_txt_fopen.close()
  cur_trg_txt_fopen.close()
  print("---")

src_meta_shelve_open.close()
trg_meta_shelve_open.close()

75
['250000', '500000', '750000', '1000000', '1250000', '1500000', '1750000', '2000000', '2250000', '2500000', '2750000', '3000000', '3250000', '3500000', '3750000', '4000000', '4250000', '4500000', '4750000', '5000000', '5250000', '5500000', '5750000', '6000000', '6250000', '6500000', '6750000', '7000000', '7250000', '7500000', '7750000', '8000000', '8250000', '8500000', '8750000', '9000000', '9250000', '9500000', '9750000', '10000000', '10250000', '10500000', '10750000', '11000000', '11250000', '11500000', '11750000', '12000000', '12250000', '12500000', '12750000', '13000000', '13250000', '13500000', '13750000', '14000000', '14250000', '14500000', '14750000', '15000000', '15250000', '15500000', '15750000', '16000000', '16250000', '16500000', '16750000', '17000000', '17250000', '17500000', '17750000', '18000000', '18250000', '18500000', '18539208']
0
cur_src_txt_fpath indexes/un/exp3/src-250000.txt
cur_trg_txt_fpath indexes/un/exp3/trg-250000.txt
processing source indexes
processing t

#Indexing an arbitrary bitext - 14 Nov 2022

In [ ]:
import math
from code_utils.pandas_utils import *
from code_utils.align_utils import *
from code_utils.general import *
from code_utils.arabic_lib import *

from itertools import groupby
from collections import Counter
ar_counter_dict=load_counts("ar_count.txt", tmp_count_dict={})

index_dir="indexes/un/exp3"
params_fpath=os.path.join(index_dir,"params.json") #
params_dict=read_json(params_fpath)
params_dict["ar_counter_dict"]=ar_counter_dict


wb_path='https://docs.google.com/spreadsheets/d/e/2PACX-1vRFH-hHy3lm7pIHpYrYwfWP4GA6GbX6l7uHwnczP6NKOcvXvdw0avrCqMJjxXA9R9LlbB85003FgYAF/pub?output=xlsx'
wb_obj=get_workbook_obj(wb_path)
sheets=list(wb_obj.keys())
ref_sheets=[v for v in sheets if v!="document"]
cols=["en","ar"]
data=get_sheets_cols(wb_obj,ref_sheets,cols)
#print(len(data))
src_index1,trg_index1=index_bitext_list(data,params_dict)

print("finished indexing extra sentences/references - number of sentences:",len(data))

# for d in data[:5]:
#   print(d)


finished indexing extra sentences/references - number of sentences: 1087


#creating an arbitrary aligned list - 14 Nov 2022

In [ ]:
import os, json
cur_align_fname="cedaw_align.txt"

index_dir="indexes/un/exp3"
params_fpath=os.path.join(index_dir,"params.json")
en_ar_corpus_combined_fname="UNv1.0.en-ar-combined.txt"

retr_params={} #retrieval parameters
retr_params["filter_params"]=read_json(params_fpath)

retr_params["index_dir"]=index_dir#"indexes/un/exp3"
retr_params["meta_suffix"]="meta"
retr_params["max_n_index_files"]=2
retr_params["single_pass"]=False
retr_params["apply_size_factor"]=False
retr_params["max_sent_len"]=40
retr_params["max_phrase_len"]=8
retr_params["lang1"]="en"
retr_params["lang2"]="ar"
retr_params["exclude_numbers"]=True
retr_params["max_phrase_dist"]=5 #maximum distance when combining two phrases while aligning
retr_params["min_phrase_wt"]=0.0001 #minimum weight for any phrase
retr_params["tok_fn"]=tok
retr_params["lang2_tok_fn"]=cur_ar_tok_fn
punc_pairs={} #punctuation pairs - en/ar - we can add later phrase pairs
punc_pairs[","]="،"
punc_pairs[";"]="؛"
punc_pairs["?"]="؟"
punc_pairs["in"]="في"
retr_params["punc_pairs"]=punc_pairs
retr_params["only_without_children"]=False
retr_params["reward_combined_phrases"]=True
retr_params["extra_src_index"]=src_index1
retr_params["extra_trg_index"]=trg_index1
#print("retr_params",retr_params)

aligned_fpath=os.path.join(index_dir,cur_align_fname)
start_loc=0 #check which line in the bitext file to start with
counter=0
if os.path.exists(aligned_fpath):
  aligned_fopen=open(aligned_fpath)
  for line0 in aligned_fopen:
    line_dict=json.loads(line0.strip())
    cur_loc=line_dict.get("loc")
    cur_i=line_dict.get("i")
    if cur_loc!=None: start_loc=cur_loc
    if cur_i!=None: counter=cur_i
  aligned_fopen.close()

print("start_loc",start_loc,"counter",counter)




sent_i=519

src_sent1='May She would await the report of subsidiary body 3 and hoped that suitable text on the establishment of a such a zone in the Middle East would be reflected in the Committee’s final report.'
trg_sent1=' أيار/مايو وقالت إنها تنتظر تقرير الهيئة الفرعية 3 وتأمل في أن ينعكس النص المناسب بشأن إنشاء منطقة من هذا القبيل في الشرق الأوسط في التقرير النهائي للجنة.'
#src_sent1='2- COMMENTS 225 OF GOVERNMENTS ON THE REPORT OF THE WORKING GROUP'
#trg_sent1='2- تعليقات 225 الحكومات على تقرير الفريق العامل المعني'
for sent_i,cur_item in enumerate(data):
  if sent_i<counter: continue
  print(sent_i)
  #cur_item=data[sent_i]
  src_sent1,trg_sent1=cur_item
  t0=time.time()
  print(len(tok(src_sent1)))
  print(len(tok(trg_sent1)))
  results=walign(src_sent1,trg_sent1,retr_params)
  elapsed=time.time()-t0
  align_list=results.get("align",[])
  src_tokens_padded=results["src"]
  trg_tokens_padded=results["trg"]
  results["i"]=sent_i
  for pair,wt in align_list:
    s_al,t_al=pair
    print("weight:",round(wt,4))
    print(">>>", s_al, src_tokens_padded[s_al[0]:s_al[1]+1])
    print(">>>", t_al, trg_tokens_padded[t_al[0]:t_al[1]+1])
    print("----")
  if align_list!=[]:
    aligned_fopen=open(aligned_fpath,"a")
    aligned_fopen.write(json.dumps(results)+"\n")
    aligned_fopen.close()

  print("elapsed",round(elapsed,4))

Streaming output truncated to the last 5000 lines.
>>> (8, 9) ['real', 'risk']
>>> (7, 8) ['خطرا', 'حقيقيا']
----
weight: 0.7275
>>> (11, 11) ['torture']
>>> (12, 12) ['ال_تعذيب']
----
weight: 0.1275
>>> (13, 13) ['other']
>>> (14, 14) ['غير']
----
weight: 0.122
>>> (14, 14) ['ill']
>>> (18, 18) ['ال_معاملة']
----
weight: 0.423
>>> (16, 16) ['treatment']
>>> (18, 18) ['ال_معاملة']
----
weight: 0.1195
>>> (17, 17) ['should']
>>> (3, 4) ['ال_رسالة', 'أن']
----
weight: 0.0213
>>> (17, 17) ['should']
>>> (3, 3) ['ال_رسالة']
----
weight: 0.0982
>>> (17, 17) ['should']
>>> (4, 4) ['أن']
----
weight: 0.0029
>>> (19, 19) ['return']
>>> (21, 21) ['عادت']
----
weight: 0.5464
>>> (21, 21) ['Somalia']
>>> (23, 23) ['ال_صومال']
----
weight: 0.8774
>>> (24, 28) ['breach', 'of', 'articles', '2', ',']
>>> (24, 30) ['،', 'ما', 'يشكل', 'انتهاكا', 'لـ', 'ال_مواد', '2']
----
weight: 0.6774
>>> (24, 27) ['breach', 'of', 'articles', '2']
>>> (26, 30) ['يشكل', 'انتهاكا', 'لـ', 'ال_مواد', '2']
----
weight: 0.

In [ ]:
import time, json, shelve, os, re, sys
from itertools import groupby
from math import log


#from general import * 
sys.path.append("code_utils")
import general
import arabic_lib

from itertools import groupby
from collections import Counter
ar_counter_dict=load_counts("ar_count.txt", tmp_count_dict={})

index_dir="indexes/un/exp3"
params_fpath=os.path.join(index_dir,"params.json") #
params_dict=read_json(params_fpath)
params_dict["ar_counter_dict"]=ar_counter_dict

def tok_bitext(list0,params0={}):
  cur_ar_counter_dict=params0.get("ar_counter_dict",{})
  tokenized_bitext_list0=[]
  for cur_loc,src_trg in enumerate(list0):
    src0,trg0=src_trg
    try:
      src_toks0=tok(src0)
      trg_toks0=tok(trg0)
      if params0.get("lang2")=="ar": trg_toks0=arabic_lib.tok_ar(trg_toks0,cur_ar_counter_dict)
    except: continue
    tokenized_bitext_list0.append((cur_loc,src_toks0,trg_toks0))
  return tokenized_bitext_list0

t_list=tok_bitext(data,params_dict)
for tl in t_list[:20]:
  print(tl)

(0, ['CEDAW/C/GAB/CO/7', '2202978E', '.', 'docx', '(', 'ENGLISH', ')'], ['CEDAW/C/GAB/CO/7', '2202978A', '.', 'docx', '(', 'ARABIC', ')'])
(1, ['CEDAW/C/GAB/CO/7'], ['CEDAW/C/GAB/CO/7'])
(2, ['United', 'Nations'], ['ال_أمــم', 'ال_متحـدة'])
(3, ['CEDAW/C/GAB/CO/7'], ['CEDAW/C/GAB/CO/7'])
(4, ['Convention', 'on', 'the', 'Elimination'], ['اتفاقية', 'ال_قضاء', 'على', 'جميـع'])
(5, ['of', 'All', 'Forms', 'of', 'Discrimination', 'against', 'Women'], ['أشكال', 'ال_تمييز', 'ضد', 'ال_مرأة'])
(6, ['Distr', '.', ':', 'General'], ['Distr', '.', ':', 'General'])
(8, ['Original', ':', 'English'], ['Arabic', 'Original', ':', 'English'])
(9, ['CEDAW/C/GAB/CO/7'], ['CEDAW/C/GAB/CO/7'])
(10, ['/', '3'], ['22', '-', '02978'])
(11, ['22', '-', '02978'], ['/', '3'])
(12, ['22', '-', '02978', '(', 'E', ')', '140322'], ['180322', '180322', '22', '-', '02978', 'X', '(', 'A', ')'])
(13, ['*', '2202978', '*'], ['*', '2202978', '*'])
(14, ['22', '-', '02978'], ['/', '4'])
(15, ['/', '4'], ['22', '-', '02978'])


#OLD - retrieving items from meta/indexes - 3-Nov-2022

In [ ]:
# import time, json, shelve, os
# def go2line(fpath0,loc0,size0=None):
#   fopen0=open(fpath0)
#   fopen0.seek(loc0)
#   #line0=fopen0.readline()
#   if size0!=None: line0=fopen0.read(size0)
#   else: line0=fopen0.readline()
#   line0=line0.split("\n")[0]
#   fopen0.close()
#   return line0

# def get_unigram_locs(tok_list0):
#   enum_list=[(v,i) for i,v in enumerate(tok_list0)]
#   enum_list.sort()
#   grouped=[(key,[v[1] for v in list(group)]) for key,group in groupby(enum_list,lambda x:x[0])]
#   loc_dict0=dict(iter(grouped))
#   return loc_dict0

# def count_results(result_dict0):
#   count0=0
#   for a,b in result_dict0.items():count0+=len(b)
#   return count0

# def list_results(result_dict0):
#   res_list0=[]
#   for a,b in result_dict0.items():res_list0.extend(b)#count0+=len(b)
#   return res_list0


# def offset_results(result_dict0,offset0,max_sent_size0=1000):
#   new_dict0={}
#   for a,b in result_dict0.items():
#     new_b=[v-(offset0/max_sent_size0) for v in b]
#     new_dict0[a]=new_b
#   return new_dict0

# #we start here, getting from the meta shelve the locations of the indexes of this word in each of the inverted index text files
# def get_word_indexes(word1,index_prefix1,retr_params1): #index_prefix is either src/trg
#   index_dir1=retr_params1.get("index_dir") #main directory where the inverted index text files are stored
#   meta_suffix1=retr_params1.get("meta_suffix","meta") #suffix of the shelve meta file (usually sr-meta.shelve or trg-meta.shelve)
#   max_n_index_files=retr_params1.get("max_n_index_files",50) #maximum number of inverted index files to process
#   cur_meta_shelve_path=os.path.join(index_dir1,"%s-%s.shelve"%(index_prefix1,meta_suffix1))
#   cur_meta_shelve_open=shelve.open(cur_meta_shelve_path)
#   cur_meta_found=cur_meta_shelve_open.get(word1,{})
#   result_dict={}
  
#   n_results=0
#   elapsed_list=[]
#   t0=time.time()
#   meta_index_items=list(cur_meta_found.items())
  
#   if max_n_index_files!=None: meta_index_items=meta_index_items[:max_n_index_files]
#   #print(meta_index_items)
#   #return result_dict
#   for index_id,index_loc in meta_index_items:
#     #print(index_id,index_loc)
#     cur_loc0,cur_size0=index_loc
#     cur_txt_fpath=os.path.join(index_dir1,"%s-%s.txt"%(index_prefix1,index_id)) 
#     #print(cur_txt_fpath,cur_loc0,cur_size0)
#     cur_meta_line=go2line(cur_txt_fpath,cur_loc0,cur_size0)
#     #cur_meta_line=go2line2(cur_txt_fpath,cur_loc0)
#     cur_raw_indexes_str=cur_meta_line.strip().split("\t")[1]
#     cur_raw_indexes=json.loads(cur_raw_indexes_str)
#     result_dict[index_id]=cur_raw_indexes
#     #n_results+=len(cur_raw_indexes)
#   #result_dict["n"]=n_results
#   #print(len(result_dict.keys()))
#   return result_dict

# def get_result_lines(result_dict1,bitext_fpath1,n_results1=20,n_items_per_meta1=50):
#   final_results=[]
#   for index_id,indexes0 in result_dict1.items():
#     if n_items_per_meta1!=None: indexes0=indexes0[:n_items_per_meta1]
#     for ind0 in indexes0:
#       final_results.append((index_id,ind0))

#   final_results.sort(key=lambda x:x[-1]-int(x[-1]))
#   final_src_trg_items=[]
#   for res0 in final_results[:n_results1]:
#     index_id,ind0=res0
#     #cur_line0=go2line2(bitext_fpath1,int(ind0))
#     cur_line0=go2line(bitext_fpath1,int(ind0))
#     cur_split0=cur_line0.strip().split("\t")
#     if len(cur_split0)!=2: continue
#     src0,trg0=cur_split0
#     final_src_trg_items.append((src0,trg0,ind0,index_id))  
#   return final_src_trg_items

# def get_index_intersection(index_dict1,offset_index_dict2): #when we are getting phrase indexes of multiple tokens
#   combined_dict0={}
#   for index_id,cur_indexes in index_dict1.items():
#     offset_indexes=offset_index_dict2.get(index_id,[])
#     intersection0=list(set(cur_indexes).intersection(set(offset_indexes)))
#     combined_dict0[index_id]=intersection0
#   return combined_dict0

# def get_index_match_ratio_count(index_dict1,index_dict2): #match src/trg indexes
#   index1_count,index2_count,intersection_count=0,0,0
#   for index_id,cur_indexes1 in index_dict1.items():
#     cur_indexes2=index_dict2.get(index_id,[])
#     cur_indexes1=[int(v) for v in cur_indexes1]
#     cur_indexes2=[int(v) for v in cur_indexes2]

#     intersection0=list(set(cur_indexes1).intersection(set(cur_indexes2)))
#     index1_count+=len(cur_indexes1)
#     index2_count+=len(cur_indexes2)
#     intersection_count+=len(intersection0)
#   ratio0=0
#   if index1_count+index2_count>0: ratio0=intersection_count/(index1_count+index2_count-intersection_count)
#   return ratio0, intersection_count
#     #combined_dict0[index_id]=intersection0


# def get_phrase_locs(tokens0,max_phrase_len=8): #identify locations of phrases of content words - excluding empty string/filtered tokens
#   content_words_locs=[w_i for w_i,wd0 in enumerate(tokens0) if wd0!=""]
#   all_phrases0=[]
#   for ii0,cur_loc0 in enumerate(content_words_locs):
#     cur_phrase=[cur_loc0]
#     all_phrases0.append(list(cur_phrase))
#     next_locs=content_words_locs[ii0+1:]
#     for nl in next_locs:
#       cur_phrase+=[nl]
#       all_phrases0.append(list(cur_phrase))
#   all_phrase_locs_str=[]
#   for ap0 in all_phrases0:
#     phrase_tokens=tokens0[ap0[0]:ap0[-1]+1]
#     if len(phrase_tokens)>max_phrase_len: continue
#     all_phrase_locs_str.append((tuple(phrase_tokens),ap0))
#   return all_phrase_locs_str

# def combine_phrase_locs(all_phrase_locs_str0):
#   phrase_locs_str_sorted=sorted(all_phrase_locs_str0)
#   combined_phrase_locs=[(key,[v[1] for v in list(group)]) for key,group in groupby(phrase_locs_str_sorted,lambda x:x[0])]
#   phrase_locs_dict0=dict(iter(combined_phrase_locs)) # key: phrase token strings > vals: corresponding locations
#   return phrase_locs_dict0


# def get_phrase_locs_indexes(tokens0,res_dict0,max_phrase_len=8): 
#   all_phrases_locs_str=get_phrase_locs(tokens0,max_phrase_len) #get the geometric listing of all possible phrases made of content words
#   phrase_locs_dict=combine_phrase_locs(all_phrases_locs_str)
#   # for a,b in phrase_locs_dict.items():
#   #   print(a,b, len(b))

#   all_phrases_locs_str.sort(key=lambda x:len(x[1])) #sort them by size
#   phrase_result_dict={} #we put the results in a dictionary, where the keys are the phrase string, and the values are the index/results with proper offset
#   for phrase_tokens,phrase0 in all_phrases_locs_str:
#     found_phrase_indexes=phrase_result_dict.get(phrase_tokens) #check if this phrase was found in previous iterations
#     if found_phrase_indexes!=None: #if it was found, we skip to the next
#       #print("found already", phrase_tokens)
#       continue
#     #print(phrase_tokens,phrase0)
#     prev_tokens=[] #to check if the phrase is just a single token or not
#     last_item,prev_items=phrase0[-1],phrase0[:-1] #we need to get the location of the last token
#     if len(prev_items)>0: prev_tokens=tokens0[prev_items[0]:prev_items[-1]+1] #if the phrase has multiple tokens, identify previous tokens // the ones before the last token
#     first_item=phrase0[0] #the location of the first token
#     last_word=tokens0[last_item] #the string of the last token
#     last_offset=last_item-first_item #the offset of the last token (location of last token minus location of first token)
#     cur_word_indexes=res_dict0.get(last_word,{})  #now retrieve the indexes for each word
#     cur_offset_indexes=offset_results(cur_word_indexes,last_offset) #offset the indexes according to the current offset
#     found_prev_tokens_index_dict=phrase_result_dict.get(tuple(prev_tokens),{}) #check if previous tokens in the phrase have indexes that were found already
#     if len(prev_items)>0 and found_prev_tokens_index_dict=={}: continue #if there are previous tokens, but no indexes found for them - NEED TO CHECK
#     if prev_items!=[]: 
#       cur_offset_indexes=get_index_intersection(found_prev_tokens_index_dict,cur_offset_indexes) #If there are previous tokens and indexes, find the intersection -ALSO CHECK THIS
#     phrase_result_dict[phrase_tokens]=cur_offset_indexes
#   new_dict={}
#   for a,b in phrase_locs_dict.items():
#     new_dict[a]=[b,phrase_result_dict.get(a,{})]
    
#   return new_dict

def get_matching(src_tok0,trg_toks0,src_index_dict0,trg_index_dict0,max_phrase_len=8):
  src_output=get_phrase_locs_indexes(src_tok0,src_index_dict0,max_phrase_len)
  trg_output=get_phrase_locs_indexes(trg_toks0,trg_index_dict0,max_phrase_len)
  matching_list=[]
  phrase_count_src_dict,phrase_count_trg_dict={},{} #how many instances of both src/trg phrases
  for src_phrase_str,src_indexes_locs in src_output.items(): phrase_count_src_dict[src_phrase_str]=len(src_indexes_locs[0])
  for trg_phrase_str,trg_indexes_locs in trg_output.items(): phrase_count_trg_dict[trg_phrase_str]=len(trg_indexes_locs[0])


  for src_phrase_str,src_indexes_locs in src_output.items():
    src_locs0,src_indexes0=src_indexes_locs
    for trg_phrase_str,trg_indexes_locs in trg_output.items():
      trg_locs0,trg_indexes0=trg_indexes_locs
      min_locs_count=min(len(src_locs0),len(trg_locs0)) #we subtract from the number of locs of each phrase the minimum
      ratio1,intersection1=get_index_match_ratio_count(src_indexes0,trg_indexes0)
      if ratio1==0: continue
      matching_list.append((src_phrase_str,trg_phrase_str,min_locs_count,intersection1,ratio1))

  matching_list.sort(key=lambda x:-x[-1])
  for ml in matching_list:
    src_phrase_str,trg_phrase_str,min_locs_count,intersection1,ratio1=ml
    if phrase_count_src_dict[src_phrase_str]>0 or phrase_count_trg_dict[trg_phrase_str]>0:
      print(src_phrase_str,trg_phrase_str, ratio1)
      print(phrase_count_src_dict[src_phrase_str],phrase_count_trg_dict[trg_phrase_str])
      print("----")

    phrase_count_src_dict[src_phrase_str]=phrase_count_src_dict[src_phrase_str]-min_locs_count
    phrase_count_trg_dict[trg_phrase_str]=phrase_count_trg_dict[trg_phrase_str]-min_locs_count

  return matching_list


index_dir="indexes/un/exp3"
params_fpath=os.path.join(index_dir,"params.json")
with open(params_fpath) as params_fopen:
  filter_params=json.load(params_fopen) #token filtering parameteres
print(filter_params)

en_ar_corpus_combined_fname="UNv1.0.en-ar-combined.txt"

retr_params={} #retrieval parameters
retr_params["index_dir"]="indexes/un/exp3"
retr_params["meta_suffix"]="meta"
retr_params["max_n_index_files"]=2

sent='The Regular Process is in its third cycle (2021–2025), with the first and second World Ocean Assessments published in 2016 and 2021, respectively.'
sent='More diverse perspectives, such as those of youth, should be integrated into that domain.'
sent="""	
Her delegation welcomed the inclusion of paragraphs on the central role of IAEA and the importance of safeguards, as well as the inclusion of paragraphs on the importance of gender in non-proliferation and safeguards-related activities.
"""
src_sent='Ireland supported the recommendation made by the representative of Australia on including the text on gender in the operational part of the report.'
trg_sent='وتؤيد أيرلندا التوصية التي قدمها ممثل أستراليا بشأن إدراج النص المتعلق بالجوانب الجنسانية في الجزء التنفيذي من التقرير.'

src_sent='The language on the verification standard, specifically with regard to the additional protocol, should be strengthened, as should the text on the small quantities protocol.'
trg_sent='وينبغي تعزيز الصياغة المتعلقة بمعيار التحقق، ولا سيما فيما يتعلق بالبروتوكول الإضافي، وكذلك النص المتعلق ببروتوكول الكميات الصغيرة.'

src_sent1='She would await the report of subsidiary body 3 and hoped that suitable text on the establishment of a such a zone in the Middle East would be reflected in the Committee’s final report.'
trg_sent1='وقالت إنها تنتظر تقرير الهيئة الفرعية 3 وتأمل في أن ينعكس النص المناسب بشأن إنشاء منطقة من هذا القبيل في الشرق الأوسط في التقرير النهائي للجنة.'

src_sent1="""
Failing to strike such a balance in any final move, without such a balance would be counterproductive and would critically affect the credibility of any final document.
"""
trg_sent1="""
ومن شأن الفشل في تحقيق هذا التوازن أن يؤدي إلى نتائج عكسية وأن يؤثر تأثيرا حاسما على مصداقية أي وثيقة ختامية.
"""

#sent='The Regular Process is in its third cycle'
#sent='Contribution made by Sustainable Development Goal 14 to achieving other Goals'
#sent='Statements were made by the representatives of 25 States and statements were also made by others'
src_tokens=tok(src_sent)
src_tokens1=filter_toks(src_tokens,filter_params) #need to make filter_tokens > OLD
trg_tokens=cur_ar_tok_fn(trg_sent)
trg_tokens1=filter_toks(trg_tokens,filter_params) #need to make filter_tokens > OLD

#print(src_tokens1)
unique_src_tokens=list(set([v for v in src_tokens1 if v!=""]))
unique_trg_tokens=list(set([v for v in trg_tokens1 if v!=""]))
t0=time.time()
src_retr_dict={}
for ut in unique_src_tokens:
  #print("src token indexes for:", ut)
  src_retr_dict[ut]=get_word_indexes(ut,"src",retr_params)
trg_retr_dict={}
for ut in unique_trg_tokens:
  #print("trg token indexes for:",ut)
  trg_retr_dict[ut]=get_word_indexes(ut,"trg",retr_params)
elapsed=time.time()-t0
print("src:",len(unique_src_tokens),"trg:",len(unique_trg_tokens),"elapsed:",round(elapsed,4))


src_tokens1=["<s>"]+src_tokens1+["</s>"]
trg_tokens1=["<s>"]+trg_tokens1+["</s>"]

print("now mtaching src -trg tokens/phrases")
t0=time.time()
matching_list=get_matching(src_tokens1,trg_tokens1,src_retr_dict,trg_retr_dict,4)
elapsed=time.time()-t0
for a in matching_list:
  print(a)
print("elapsed:",round(elapsed,4))


{'excluded_words': ['the', 'a', 'an', 'and', 'of', 'its', 'his', 'her', 'their', 'them', 'to', 'is', 'are', 'am', 'been', 'was', 'were', 'be', 'have', 'has', 'had', 'in', 'on', 'at', 'with', 'from', 'for', 'that', 'he', 'she', 'it', 'would', 'من', 'على', 'في', 'مع', 'إلى', 'له', 'تم', 'إلى', 'في'], 'max_sent_n_tokens': 1000, 'lang1': 'en', 'lang2': 'ar', 'exclude_single_chars': True, 'batch_size': 250000, 'min_n_indexes_per_token': 3}
src: 13 trg: 17 elapsed: 24.9597
now mtaching src -trg tokens/phrases
('protocol',) ('بروتوكول',) 0.7305428384565075
2 2
----
('small',) ('صغيرة',) 0.5574832775919732
1 1
----
('text',) ('نص',) 0.5405181347150259
1 1
----
('verification',) ('التحقق',) 0.4486652977412731
1 1
----
('should',) ('ينبغي',) 0.4287181242512408
2 1
----
('quantities',) ('كميات',) 0.4032697547683924
1 1
----
('small', 'quantities') ('كميات', 'صغيرة') 0.3333333333333333
1 1
----
('additional', 'protocol') ('بروتوكول', 'إضافي') 0.1724137931034483
1 1
----
('should', '', 'strengthene

#OLD - Building the align class - 7 Nov 2022

In [ ]:
# def get_unigram_locs(tok_list0):
#   enum_list=[(v,i) for i,v in enumerate(tok_list0)]
#   enum_list.sort()
#   grouped=[(key,[v[1] for v in list(group)]) for key,group in groupby(enum_list,lambda x:x[0])]
#   loc_dict0=dict(iter(grouped))
#   return loc_dict0
import time, json, shelve, os
def go2line(fpath0,loc0,size0=None):
  fopen0=open(fpath0)
  fopen0.seek(loc0)
  #line0=fopen0.readline()
  if size0!=None: line0=fopen0.read(size0)
  else: line0=fopen0.readline()
  line0=line0.split("\n")[0]
  fopen0.close()
  return line0

def get_unigram_locs(tok_list0):
  enum_list=[(v,i) for i,v in enumerate(tok_list0)]
  enum_list.sort()
  grouped=[(key,[v[1] for v in list(group)]) for key,group in groupby(enum_list,lambda x:x[0])]
  loc_dict0=dict(iter(grouped))
  return loc_dict0

def count_results(result_dict0):
  count0=0
  for a,b in result_dict0.items():count0+=len(b)
  return count0

def list_results(result_dict0):
  res_list0=[]
  for a,b in result_dict0.items():res_list0.extend(b)#count0+=len(b)
  return res_list0


def offset_results(result_dict0,offset0,max_sent_size0=1000):
  new_dict0={}
  for a,b in result_dict0.items():
    new_b=[v-(offset0/max_sent_size0) for v in b]
    new_dict0[a]=new_b
  return new_dict0

#we start here, getting from the meta shelve the locations of the indexes of this word in each of the inverted index text files
def get_word_indexes(word1,index_prefix1,retr_params1): #index_prefix is either src/trg
  index_dir1=retr_params1.get("index_dir") #main directory where the inverted index text files are stored
  meta_suffix1=retr_params1.get("meta_suffix","meta") #suffix of the shelve meta file (usually sr-meta.shelve or trg-meta.shelve)
  max_n_index_files=retr_params1.get("max_n_index_files",50) #maximum number of inverted index files to process
  cur_meta_shelve_path=os.path.join(index_dir1,"%s-%s.shelve"%(index_prefix1,meta_suffix1))
  cur_meta_shelve_open=shelve.open(cur_meta_shelve_path)
  cur_meta_found=cur_meta_shelve_open.get(word1,{})
  result_dict={}
  
  n_results=0
  elapsed_list=[]
  t0=time.time()
  meta_index_items=list(cur_meta_found.items())
  
  if max_n_index_files!=None: meta_index_items=meta_index_items[:max_n_index_files]
  #print(meta_index_items)
  #return result_dict
  for index_id,index_loc in meta_index_items:
    #print(index_id,index_loc)
    cur_loc0,cur_size0=index_loc
    cur_txt_fpath=os.path.join(index_dir1,"%s-%s.txt"%(index_prefix1,index_id)) 
    #print(cur_txt_fpath,cur_loc0,cur_size0)
    cur_meta_line=go2line(cur_txt_fpath,cur_loc0,cur_size0)
    #cur_meta_line=go2line2(cur_txt_fpath,cur_loc0)
    cur_raw_indexes_str=cur_meta_line.strip().split("\t")[1]
    cur_raw_indexes=json.loads(cur_raw_indexes_str)
    result_dict[index_id]=cur_raw_indexes
    #n_results+=len(cur_raw_indexes)
  #result_dict["n"]=n_results
  #print(len(result_dict.keys()))
  return result_dict

def get_result_lines(result_dict1,bitext_fpath1,n_results1=20,n_items_per_meta1=50):
  final_results=[]
  for index_id,indexes0 in result_dict1.items():
    if n_items_per_meta1!=None: indexes0=indexes0[:n_items_per_meta1]
    for ind0 in indexes0:
      final_results.append((index_id,ind0))

  final_results.sort(key=lambda x:x[-1]-int(x[-1]))
  final_src_trg_items=[]
  for res0 in final_results[:n_results1]:
    index_id,ind0=res0
    #cur_line0=go2line2(bitext_fpath1,int(ind0))
    cur_line0=go2line(bitext_fpath1,int(ind0))
    cur_split0=cur_line0.strip().split("\t")
    if len(cur_split0)!=2: continue
    src0,trg0=cur_split0
    final_src_trg_items.append((src0,trg0,ind0,index_id))  
  return final_src_trg_items

def get_index_intersection(index_dict1,offset_index_dict2): #when we are getting phrase indexes of multiple tokens
  combined_dict0={}
  for index_id,cur_indexes in index_dict1.items():
    offset_indexes=offset_index_dict2.get(index_id,[])
    intersection0=list(set(cur_indexes).intersection(set(offset_indexes)))
    combined_dict0[index_id]=intersection0
  return combined_dict0

def get_index_match_ratio_count(index_dict1,index_dict2): #match src/trg indexes
  index1_count,index2_count,intersection_count=0,0,0
  for index_id,cur_indexes1 in index_dict1.items():
    cur_indexes2=index_dict2.get(index_id,[])
    cur_indexes1=[int(v) for v in cur_indexes1]
    cur_indexes2=[int(v) for v in cur_indexes2]

    intersection0=list(set(cur_indexes1).intersection(set(cur_indexes2)))
    index1_count+=len(cur_indexes1)
    index2_count+=len(cur_indexes2)
    intersection_count+=len(intersection0)
  ratio0=0
  if index1_count+index2_count>0: ratio0=intersection_count/(index1_count+index2_count-intersection_count)
  return ratio0, intersection_count
    #combined_dict0[index_id]=intersection0


def get_phrase_locs(tokens0,max_phrase_len=8): #identify locations of phrases of content words - excluding empty string/filtered tokens
  content_words_locs=[w_i for w_i,wd0 in enumerate(tokens0) if wd0!=""]
  all_phrases0=[]
  for ii0,cur_loc0 in enumerate(content_words_locs):
    cur_phrase=[cur_loc0]
    all_phrases0.append(list(cur_phrase))
    next_locs=content_words_locs[ii0+1:]
    for nl in next_locs:
      cur_phrase+=[nl]
      all_phrases0.append(list(cur_phrase))
  all_phrase_locs_str=[]
  for ap0 in all_phrases0:
    phrase_tokens=tokens0[ap0[0]:ap0[-1]+1]
    if len(phrase_tokens)>max_phrase_len: continue
    all_phrase_locs_str.append((tuple(phrase_tokens),ap0))
  return all_phrase_locs_str

def combine_phrase_locs(all_phrase_locs_str0):
  phrase_locs_str_sorted=sorted(all_phrase_locs_str0)
  combined_phrase_locs=[(key,[v[1] for v in list(group)]) for key,group in groupby(phrase_locs_str_sorted,lambda x:x[0])]
  phrase_locs_dict0=dict(iter(combined_phrase_locs)) # key: phrase token strings > vals: corresponding locations
  return phrase_locs_dict0


def get_phrase_locs_indexes(tokens0,res_dict0,max_phrase_len=8): 
  all_phrases_locs_str=get_phrase_locs(tokens0,max_phrase_len) #get the geometric listing of all possible phrases made of content words
  phrase_locs_dict=combine_phrase_locs(all_phrases_locs_str)
  all_phrases_locs_str.sort(key=lambda x:len(x[1])) #sort them by size
  phrase_result_dict={} #we put the results in a dictionary, where the keys are the phrase string, and the values are the index/results with proper offset
  for phrase_tokens,phrase0 in all_phrases_locs_str:
    found_phrase_indexes=phrase_result_dict.get(phrase_tokens) #check if this phrase was found in previous iterations
    if found_phrase_indexes!=None: #if it was found, we skip to the next
      #print("found already", phrase_tokens)
      continue
    #print(phrase_tokens,phrase0)
    prev_tokens=[] #to check if the phrase is just a single token or not
    last_item,prev_items=phrase0[-1],phrase0[:-1] #we need to get the location of the last token
    if len(prev_items)>0: prev_tokens=tokens0[prev_items[0]:prev_items[-1]+1] #if the phrase has multiple tokens, identify previous tokens // the ones before the last token
    first_item=phrase0[0] #the location of the first token
    last_word=tokens0[last_item] #the string of the last token
    last_offset=last_item-first_item #the offset of the last token (location of last token minus location of first token)
    cur_word_indexes=res_dict0.get(last_word,{})  #now retrieve the indexes for each word
    cur_offset_indexes=offset_results(cur_word_indexes,last_offset) #offset the indexes according to the current offset
    found_prev_tokens_index_dict=phrase_result_dict.get(tuple(prev_tokens),{}) #check if previous tokens in the phrase have indexes that were found already
    if len(prev_items)>0 and found_prev_tokens_index_dict=={}: continue #if there are previous tokens, but no indexes found for them - NEED TO CHECK
    if prev_items!=[]: 
      cur_offset_indexes=get_index_intersection(found_prev_tokens_index_dict,cur_offset_indexes) #If there are previous tokens and indexes, find the intersection -ALSO CHECK THIS
    phrase_result_dict[phrase_tokens]=cur_offset_indexes
  new_dict={}
  for a,b in phrase_locs_dict.items():
    new_dict[a]=[b,phrase_result_dict.get(a,{})]
    
  return new_dict

def combine_els(el1,el2):
  src_span1,trg_span1=el1
  src_span2,trg_span2=el2
  x1_0,x1_1=src_span1
  x2_0,x2_1=src_span2
  y1_0,y1_1=trg_span1
  y2_0,y2_1=trg_span2
  min_x=min(x1_0,x1_1,x2_0,x2_1)
  max_x=max(x1_0,x1_1,x2_0,x2_1)
  min_y=min(y1_0,y1_1,y2_0,y2_1)
  max_y=max(y1_0,y1_1,y2_0,y2_1)
  new_src_span=(min_x,max_x)
  new_trg_span=(min_y,max_y)
  return (new_src_span,new_trg_span)

def get_span_dist(span1,span2):
  span1_x1,span1_x2=span1
  span2_x1,span2_x2=span2
  d1=abs(span1_x2-span1_x1)
  d2=abs(span2_x2-span2_x1)
  max_x=max(span1_x1,span1_x2,span2_x1,span2_x2)
  min_x=min(span1_x1,span1_x2,span2_x1,span2_x2)
  d_total=max_x-min_x
  return d_total-d1-d2

def get_rec_el_children(el0,el_child_dict0,el_list0=[]):
  cur_children0=el_child_dict0.get(el0,[])
  if cur_children0==[]: el_list0.append(el0)
  for ch0 in cur_children0:
    el_list0=get_rec_el_children(ch0,el_child_dict0,el_list0)
  return el_list0

# def djk(pt0,pt1,transition_dict0,pt_list0=None):
#   path_wt_dict0={}
#   prev_dict0={}
#   if pt_list0==None: pt_list0=list(transition_dict0.keys())
#   for cur_pt in pt_list0:
#     following_dict0=transition_dict0.get(cur_pt,{})
#     found_cur_path_wt=path_wt_dict0.get(cur_pt,0)
#     for next_pt,next_wt in following_dict0.items():
#       combined_wt=found_cur_path_wt+next_wt
#       found_next_wt=path_wt_dict0.get(next_pt,0)
#       if combined_wt>found_next_wt:
#         path_wt_dict0[next_pt]=combined_wt
#         prev_dict0[next_pt]=cur_pt
#   cur_pt=pt1
#   out_path=[]
#   while cur_pt!=None:
#     out_path.append(cur_pt)
#     if cur_pt==pt0: break
#     cur_pt=prev_dict0.get(cur_pt)
#   out_path=list(reversed(out_path))
#   return out_path, path_wt_dict0.get(pt1)


def get_ne_se_dict(all_elements0): #elements are pairs of src/trg spans
  se_transition_dict0,ne_transition_dict0={},{} #identify the possible transitions from one element to another - southeast or northeast
  for el0,el_wt0 in all_elements0:
    #print(el0,el_wt0)
    src_span0,trg_span0=el0
    se_transition_dict0[el0]={}
    ne_transition_dict0[el0]={}
    ne_list,se_list=[],[]
    for el1,el_wt1 in all_elements0:
      src_span1,trg_span1=el1
      if src_span0==src_span1 or trg_span0==trg_span1: continue
      if src_span1[0]<=src_span0[1]: continue
      src_span_dist=get_span_dist(src_span0,src_span1)
      trg_span_dist=get_span_dist(trg_span0,trg_span1)
      direction=None
      if trg_span1[0]>trg_span0[1]: 
        direction="se"
        se_list.append((el1,el_wt1))
        se_transition_dict0[el0][el1]=el_wt0
      if trg_span1[1]<trg_span0[0]: 
        direction="ne"
        ne_list.append((el1,el_wt1))
        ne_transition_dict0[el0][el1]=el_wt0
      if direction==None: continue
  return se_transition_dict0, ne_transition_dict0

def split_path_chunks(path_list0,skip_last=False): #split a path e.g. cur_list=["a","b","c","d","0"] to chunks e.g. ["a","b"], ["b","c","d"]
  path_chunks0=[]
  if skip_last: last_i0,last_i1=len(path_list0)-2,len(path_list0)-1
  else: last_i0,last_i1=len(path_list0)-1,len(path_list0)
  for path_i0 in range(last_i0):
    for path_i1 in range(path_i0+1,last_i1):
      cur_chunk=path_list0[path_i0:path_i1+1]
      path_chunks0.append(cur_chunk)
  return path_chunks0


def walign(src_sent0,trg_sent0,retr_align_params0={}):
  filter_params0=retr_align_params0.get("filter_params",{})
  lang1=retr_align_params0.get("lang1","en")
  lang2=retr_align_params0.get("lang2","ar")
  exclude_numbers=retr_align_params0.get("exclude_numbers",False)
  max_phrase_len=retr_align_params0.get("max_phrase_len",8) #when we get the phrases from a sentence
  max_phrase_dist=retr_align_params0.get("max_phrase_dist",10) #maximum distance when combining two phrases
  min_phrase_wt=retr_align_params0.get("min_phrase_wt",0.01)
  lang2_tok_fn=retr_align_params0.get("lang2_tok_fn")
  punc_pairs0=retr_align_params0.get("punc_pairs",{}) 
  src_tokens=tok(src_sent0)
  if lang2=="ar" and lang2_tok_fn!=None: trg_tokens=lang2_tok_fn(trg_sent0)
  else: trg_tokens=tok(trg_sent0)
  src_tokens_filtered=filter_toks(src_tokens,filter_params0)
  trg_tokens_filtered=filter_toks(trg_tokens,filter_params0) #need to make filter_tokens > OLD
  unique_src_tokens=list(set([v for v in src_tokens_filtered if v!=""]))
  unique_trg_tokens=list(set([v for v in trg_tokens_filtered if v!=""]))
  src_tokens_padded=["<s>"]+src_tokens+["</s>"] #filter_toks(self.src_tokens,filter_params0)
  trg_tokens_padded=["<s>"]+trg_tokens+["</s>"]
  src_tok_loc_dict=get_unigram_locs(src_tokens_padded)
  trg_tok_loc_dict=get_unigram_locs(trg_tokens_padded)
  src_tokens_filtered_padded=["<s>"]+src_tokens_filtered+["</s>"] #main list to be aligned
  trg_tokens_filtered_padded=["<s>"]+trg_tokens_filtered+["</s>"]

  #Now we load the indexes for our tokens
  t0=time.time()
  src_index_dict,trg_index_dict={},{}
  src_retr_dict={}
  for ut in unique_src_tokens: 
    if ut.isdigit() and exclude_numbers: continue
    src_index_dict[ut]=get_word_indexes(ut,"src",retr_align_params0)
  for ut in unique_trg_tokens: 
    if ut.isdigit() and exclude_numbers: continue
    trg_index_dict[ut]=get_word_indexes(ut,"trg",retr_align_params0)
  elapsed=time.time()-t0
  print("src:",len(unique_src_tokens),"trg:",len(unique_trg_tokens),"elapsed:",round(elapsed,4))
  #print("now mtaching src -trg tokens/phrases")
  #Now we start the matching between src/trg tokens
  t0=time.time()

  src_output=get_phrase_locs_indexes(src_tokens_filtered_padded,src_index_dict,max_phrase_len)
  trg_output=get_phrase_locs_indexes(trg_tokens_filtered_padded,trg_index_dict,max_phrase_len)
  matching_list=[]
  phrase_count_src_dict,phrase_count_trg_dict={},{} #how many instances of each src/trg phrases
  for src_phrase_str,src_indexes_locs in src_output.items(): phrase_count_src_dict[src_phrase_str]=len(src_indexes_locs[0])
  for trg_phrase_str,trg_indexes_locs in trg_output.items(): phrase_count_trg_dict[trg_phrase_str]=len(trg_indexes_locs[0])

  for src_phrase_str,src_indexes_locs in src_output.items():
    src_locs0,src_indexes0=src_indexes_locs
    for trg_phrase_str,trg_indexes_locs in trg_output.items():
      trg_locs0,trg_indexes0=trg_indexes_locs
      if src_phrase_str==trg_phrase_str: ratio1,intersection1=0.5, 1000
      else: ratio1,intersection1=get_index_match_ratio_count(src_indexes0,trg_indexes0)
      min_locs_count=min(len(src_locs0),len(trg_locs0)) #we subtract from the number of locs of each phrase the minimum
      if ratio1==0: continue
      if ratio1<min_phrase_wt: continue
      if intersection1<2: continue
      matching_list.append((src_phrase_str,trg_phrase_str,src_locs0,trg_locs0,min_locs_count,intersection1,ratio1))

  matching_list.sort(key=lambda x:-x[-1])
  new_matching_list=[]
  for ml in matching_list:
    src_phrase_str,trg_phrase_str,src_locs0,trg_locs0,min_locs_count,intersection1,ratio1=ml
    if phrase_count_src_dict[src_phrase_str]>0 or phrase_count_trg_dict[trg_phrase_str]>0:
      new_matching_list.append((src_phrase_str,trg_phrase_str,src_locs0,trg_locs0,intersection1,ratio1))
    phrase_count_src_dict[src_phrase_str]=phrase_count_src_dict[src_phrase_str]-min_locs_count #subtract the minimum count of instances from both src and trg
    phrase_count_trg_dict[trg_phrase_str]=phrase_count_trg_dict[trg_phrase_str]-min_locs_count
  
  for src_punc0,trg_punc0 in punc_pairs0.items(): #now let's match punctuation from the original tokens
    punc_src_locs=src_tok_loc_dict.get(src_punc0,[])
    if punc_src_locs==[]: continue
    punc_trg_locs=trg_tok_loc_dict.get(trg_punc0,[])
    if punc_trg_locs==[]: continue
    ratio1,intersection1=0.2, 100
    src_punc0,trg_punc0=tuple([src_punc0]),tuple([trg_punc0])
    punc_src_locs=[[v] for v in punc_src_locs]
    punc_trg_locs=[[v] for v in punc_trg_locs]
    new_matching_list.append((src_punc0,trg_punc0,punc_src_locs,punc_trg_locs,intersection1,ratio1))

  new_matching_list.sort(key=lambda x:-x[-1])
  elapsed=time.time()-t0
  print("finished matching words and phrases:", elapsed)
  #Now we have the matching list - let's align
  el_dict={} #weight of each element
  el_child_dict={} #children of each element
  for a in new_matching_list:
    #print(a)
    src_phrase_str,trg_phrase_str,src_locs0,trg_locs0,intersection1,ratio1=a
    for sl0 in src_locs0:
      s_min,s_max=sl0[0],sl0[-1]
      for tl0 in trg_locs0:
        t_min,t_max=tl0[0],tl0[-1]
        el0=((s_min,s_max) ,(t_min,t_max))
        el_dict[el0]=ratio1
  first_src_span,first_trg_span=(0,0),(0,0)
  last_src_span=(len(src_tokens_padded)-1,len(src_tokens_padded)-1)
  last_trg_span=(len(trg_tokens_padded)-1,len(trg_tokens_padded)-1)
  ne_el=(last_src_span,first_trg_span) #north eastern element - uppermost rightmost, so we go diagonally up to the right
  se_el=(last_src_span,last_trg_span)
  el_dict[ne_el]=0
  top_wt=0
  full_src_span0=(0,len(src_tokens_padded)-1)
  full_trg_span0=(0,len(trg_tokens_padded)-1)
  full_el=(full_src_span0,full_trg_span0)

  #start iteration here
  n_epochs=10
  for epoch0 in range(n_epochs):
    print("epoch0",epoch0)
    all_elements=list(el_dict.items())
    all_elements.sort()
    se_transition_dict,ne_transition_dict=get_ne_se_dict(all_elements)
    new_el_counter=0
    for cur_el,b in se_transition_dict.items():
      next_els=list(b.keys())
      if len(next_els)<2: continue
      cur_pts=[cur_el]+next_els
      cur_path,cur_path_wt=djk(cur_el,se_el,se_transition_dict,cur_pts)
      path_el_wts=[(v, el_dict.get(v,0)) for v in cur_path]
      path_el_wts_chunks=split_path_chunks(path_el_wts)
      #print("se",cur_el,next_els)
      for chunk in path_el_wts_chunks:
        chunk_wt=sum([v[1] for v in chunk])
        chunk_els=[v[0] for v in chunk]
        combined_el=combine_els(chunk_els[0],chunk_els[-1])
        found_wt=el_dict.get(combined_el,0)
        found_children=el_child_dict.get(combined_el,[])
        if chunk_wt>found_wt:
          #print("combined_el",combined_el,"chunk_wt",chunk_wt,"found_wt",found_wt,"found_children",found_children)
          el_dict[combined_el]=chunk_wt
          el_child_dict[combined_el]=chunk_els
          new_el_counter+=1
    print("finished processing SE, new_el_counter:",new_el_counter)
    for cur_el,b in ne_transition_dict.items():
      next_els=list(b.keys())
      if len(next_els)<2: continue
      cur_pts=[cur_el]+next_els
      cur_path,cur_path_wt=djk(cur_el,ne_el,ne_transition_dict,cur_pts)
      path_el_wts=[(v, el_dict.get(v,0)) for v in cur_path]
      path_el_wts_chunks=split_path_chunks(path_el_wts)
      #print("ne",cur_el,next_els)
      for chunk in path_el_wts_chunks:
        chunk_wt=sum([v[1] for v in chunk])
        chunk_els=[v[0] for v in chunk]
        combined_el=combine_els(chunk_els[0],chunk_els[-1])
        found_wt=el_dict.get(combined_el,0)
        found_children=el_child_dict.get(combined_el,[])
        if chunk_wt>found_wt:
          #print("combined_el",combined_el,"chunk_wt",chunk_wt,"found_wt",found_wt,"found_children",found_children)
          el_dict[combined_el]=chunk_wt
          el_child_dict[combined_el]=chunk_els
          new_el_counter+=1
    print("finished processing NE, new_el_counter:",new_el_counter)
    cur_full_wt=el_dict.get(full_el,0)
    print("full_el",full_el,"cur_full_wt",cur_full_wt)
    if cur_full_wt>0 and cur_full_wt==top_wt: break
    top_wt=cur_full_wt
  align_list=get_rec_el_children(full_el,el_child_dict,el_list0=[])
  # for s_al,t_al in align_list:
  #   print(">>>", s_al, src_tokens_padded[s_al[0]:s_al[1]+1])
  #   print(">>>", t_al, trg_tokens_padded[t_al[0]:t_al[1]+1])
  #   print("----")
  result_dict0={}
  result_dict0["src"]=src_tokens_padded
  result_dict0["trg"]=trg_tokens_padded
  result_dict0["align"]=align_list
  return result_dict0

  # return
  
  # used_el_pairs={}
  # child_dict={}
  # top_wt=0
  # full_src_span0=(0,len(src_tokens_filtered_padded)-1)
  # full_trg_span0=(0,len(trg_tokens_filtered_padded)-1)
  # full_el=(full_src_span0,full_trg_span0)
  # for epoch0 in range(20):
  #   print("epoch0",epoch0)
  #   all_elements=list(el_dict.items())
  #   all_elements.sort()
  #   for el_i,el_item0 in enumerate(all_elements):
  #     el0,el_wt0=el_item0
  #     src_span0,trg_span0=el0
  #     for el_item1 in all_elements[el_i+1:]:
  #       if used_el_pairs.get((el_item0,el_item1),False): continue
  #       el1,el_wt1=el_item1
  #       el_pair_key=(el0,el1)
  #       src_span1,trg_span1=el1
  #       if src_span0==src_span1 or trg_span0==trg_span1: continue
  #       #if src_span1[0]<=src_span0[1]: continue
  #       src_span_dist=get_span_dist(src_span0,src_span1)
  #       trg_span_dist=get_span_dist(trg_span0,trg_span1)
  #       if trg_span_dist<1 or src_span_dist<1: continue
  #       if src_span_dist> max_phrase_dist or trg_span_dist> max_phrase_dist: continue
  #       used_el_pairs[(el_item0,el_item1)]=True
  #       combined_el=combine_els(el0,el1)
  #       combined_wt=el_wt0+el_wt1
  #       found_wt=el_dict.get(combined_el,0)
  #       if combined_wt>found_wt:
  #         el_dict[combined_el]=combined_wt
  #         child_dict[combined_el]=(el0,el1)
  #     #print("----")
  #   cur_full_wt=el_dict.get(full_el,0)
  #   print("full_el",full_el,"cur_full_wt",cur_full_wt)
  #   if cur_full_wt>0 and cur_full_wt==top_wt: break
  #   top_wt=cur_full_wt
  # if cur_full_wt>0:
  #   align_list=get_rec_el_children(full_el,child_dict,el_list0=[])
  # else:
  #   wt_list=list(el_dict.items())
  #   wt_list.sort(key=lambda x:-x[1])
  #   top_el=wt_list[0][0]
  #   print("couldn't find full el:",full_el, "used the top el instead", top_el)
  #   align_list=get_rec_el_children(top_el,child_dict,el_list0=[])
  # for s_al,t_al in align_list:
  #   print(">>>", s_al, src_tokens_padded[s_al[0]:s_al[1]+1])
  #   print(">>>", t_al, trg_tokens_padded[t_al[0]:t_al[1]+1])
  #   print("----")
  # result_dict0={}
  # result_dict0["src"]=src_tokens_padded
  # result_dict0["trg"]=trg_tokens_padded
  # result_dict0["align"]=align_list
  # return result_dict0


index_dir="indexes/un/exp3"
params_fpath=os.path.join(index_dir,"params.json")
with open(params_fpath) as params_fopen:
  filter_params=json.load(params_fopen) #token filtering parameteres
#print(filter_params)

en_ar_corpus_combined_fname="UNv1.0.en-ar-combined.txt"

retr_params={} #retrieval parameters
retr_params["filter_params"]=filter_params

retr_params["index_dir"]="indexes/un/exp3"
retr_params["meta_suffix"]="meta"
retr_params["max_n_index_files"]=2
retr_params["lang1"]="en"
retr_params["lang2"]="ar"
retr_params["exclude_numbers"]=True
retr_params["max_phrase_dist"]=5 #maximum distance when combining two phrases while aligning
retr_params["min_phrase_wt"]=0.0001 #minimum weight for any phrase
retr_params["lang2_tok_fn"]=cur_ar_tok_fn



punc_pairs={} #punctuation pairs - en/ar - we can add later phrase pairs
punc_pairs[","]="،"
punc_pairs[";"]="؛"
punc_pairs["?"]="؟"
#punc_pairs["in"]="في"
retr_params["punc_pairs"]=punc_pairs



src_sent1='She would await the report of subsidiary body 3 and hoped that suitable text on the establishment of a such a zone in the Middle East would be reflected in the Committee’s final report.'
trg_sent1='وقالت إنها تنتظر تقرير الهيئة الفرعية 3 وتأمل في أن ينعكس النص المناسب بشأن إنشاء منطقة من هذا القبيل في الشرق الأوسط في التقرير النهائي للجنة.'

# src_sent1="At any rate, this would work?"
# trg_sent1="؟على أي حال، سوف ينجح هذا الأمر"

src_sent1="In 2015, the General assembly decided that the secretary-general will continue his work."
trg_sent1="في عام 2015، قررت الجمعية العامة أن يواصل الأمين العام عمله."

src_sent1='Having listened carefully to discussions on the issue of export controls throughout the Review Conference, her delegation had not heard of specific instances to support the view that export controls constrained peaceful uses of nuclear energy.'
trg_sent1='وقالت إن وفد بلدها، بعد أن استمع بعناية إلى المناقشات المتعلقة بمسألة ضوابط التصدير طوال المؤتمر الاستعراضي، لم يسمع عن حالات محددة تؤيد الرأي القائل بأن ضوابط التصدير تقيد الاستخدامات السلمية للطاقة النووية.'

src_sent0='Her delegation echoed calls by other delegations on the importance of including suitable wording on a nuclear-weapon-free zone in the Middle East.'
trg_sent0='وقالت إن وفد بلدها يكرر نداءات الوفود الأخرى بشأن أهمية إدراج صيغة مناسبة بشأن إنشاء منطقة خالية من الأسلحة النووية في الشرق الأوسط.'
walign_obj=walign(src_sent1,trg_sent1,retr_params)
print(src_sent1)
print(trg_sent1)

src: 22 trg: 29 elapsed: 2.2115
finished matching words and phrases: 15.283461809158325
epoch0 0
finished processing SE, new_el_counter: 505
finished processing NE, new_el_counter: 672
full_el ((0, 39), (0, 41)) cur_full_wt 5.894223582822251
epoch0 1
finished processing SE, new_el_counter: 716
finished processing NE, new_el_counter: 1075
full_el ((0, 39), (0, 41)) cur_full_wt 6.97697024115995
epoch0 2
finished processing SE, new_el_counter: 546
finished processing NE, new_el_counter: 722
full_el ((0, 39), (0, 41)) cur_full_wt 7.368244665737796
epoch0 3
finished processing SE, new_el_counter: 167
finished processing NE, new_el_counter: 185
full_el ((0, 39), (0, 41)) cur_full_wt 7.527105428247841
epoch0 4
finished processing SE, new_el_counter: 26
finished processing NE, new_el_counter: 29
full_el ((0, 39), (0, 41)) cur_full_wt 7.527105428247841
>>> (0, 0) ['<s>']
>>> (0, 0) ['<s>']
----
>>> (1, 1) ['Having']
>>> (7, 8) ['بعد', 'أن']
----
>>> (2, 2) ['listened']
>>> (9, 9) ['استمع']
----

In [ ]:
def split_path_chunks(path_list0,skip_last=True):
  path_chunks0=[]
  if skip_last: last_i0,last_i1=len(path_list0)-2,len(path_list0)-1
  else: last_i0,last_i1=len(path_list0)-1,len(path_list0)
  for path_i0 in range(last_i0):
    for path_i1 in range(path_i0+1,last_i1):
      cur_chunk=path_list0[path_i0:path_i1+1]
      path_chunks0.append(cur_chunk)
  return path_chunks0

cur_list=["a","b","c","d","0"]
ch_list=split_path_chunks(cur_list,False)
for a in ch_list:
  print(a)

['a', 'b']
['a', 'b', 'c']
['a', 'b', 'c', 'd']
['a', 'b', 'c', 'd', '0']
['b', 'c']
['b', 'c', 'd']
['b', 'c', 'd', '0']
['c', 'd']
['c', 'd', '0']
['d', '0']


In [ ]:
def djk(pt0,pt1,transition_dict0,pt_list0=None):
  path_wt_dict0={}
  prev_dict0={}
  if pt_list0==None: pt_list0=list(transition_dict0.keys())
  for cur_pt in pt_list0:
    following_dict0=transition_dict0.get(cur_pt,{})
    found_cur_path_wt=path_wt_dict0.get(cur_pt,0)
    for next_pt,next_wt in following_dict0.items():
      combined_wt=found_cur_path_wt+next_wt
      found_next_wt=path_wt_dict0.get(next_pt,0)
      if combined_wt>found_next_wt:
        path_wt_dict0[next_pt]=combined_wt
        prev_dict0[next_pt]=cur_pt
  cur_pt=pt1
  out_path=[]
  while cur_pt!=None:
    out_path.append(cur_pt)
    if cur_pt==pt0: break
    cur_pt=prev_dict0.get(cur_pt)
  out_path=list(reversed(out_path))
  return out_path, path_wt_dict0.get(pt1)

transition_dict1={}
transition_dict1["0"]={"a":0.1,"b":0.1}
transition_dict1["a"]={"b":0.1,"c":0.1,"d":0.2,"e":0.9}
transition_dict1["b"]={"c":0.1,"d":0.1,"e":0.1}
transition_dict1["c"]={"d":0.5,"e":0.9}
transition_dict1["d"]={"e":0.4}

#path,wt=djk("a","e",transition_dict1,pt_list0=["a","c","e"])
path,wt=djk("0","d",transition_dict1)
print("path",path,wt)

path ['0', 'a', 'b', 'c', 'd'] 0.8


In [ ]:
#actual class - OLD


# class walign_cls:
#   def __init__(self,src_sent0,trg_sent0,retr_align_params0={},filter_params0={}) -> None:
#     #first - load parameters and adjust tokenization and location info
#     #self.retr_align_params=retr_align_params0
#     lang1=retr_align_params0.get("lang1","en")
#     lang2=retr_align_params0.get("lang2","ar")
#     exclude_numbers=retr_align_params0.get("exclude_numbers",False)
#     max_phrase_len=retr_align_params0.get("max_phrase_len",8) #when we get the phrases from a sentence
#     max_phrase_dist=retr_align_params0.get("max_phrase_dist",10) #maximum distance when combining two phrases
#     lang2_tok_fn=retr_align_params0.get("lang2_tok_fn")
#     punc_pairs0=retr_align_params0.get("punc_pairs",{}) 
#     self.src_tokens=tok(src_sent0)
#     if lang2=="ar" and lang2_tok_fn!=None: self.trg_tokens=lang2_tok_fn(trg_sent0)
#     else: self.trg_tokens=tok(trg_sent0)
#     self.src_tokens_filtered=filter_toks(self.src_tokens,filter_params0)
#     self.trg_tokens_filtered=filter_toks(self.trg_tokens,filter_params0) #need to make filter_tokens > OLD
#     self.unique_src_tokens=list(set([v for v in self.src_tokens_filtered if v!=""]))
#     self.unique_trg_tokens=list(set([v for v in self.trg_tokens_filtered if v!=""]))
#     self.src_tokens_padded=["<s>"]+self.src_tokens+["</s>"] #filter_toks(self.src_tokens,filter_params0)
#     self.trg_tokens_padded=["<s>"]+self.trg_tokens+["</s>"]
#     self.src_tok_loc_dict=get_unigram_locs(self.src_tokens_padded)
#     self.trg_tok_loc_dict=get_unigram_locs(self.trg_tokens_padded)
#     self.src_tokens_filtered_padded=["<s>"]+self.src_tokens_filtered+["</s>"] #main list to be aligned
#     self.trg_tokens_filtered_padded=["<s>"]+self.trg_tokens_filtered+["</s>"]

#     #Now we load the indexes for our tokens
#     t0=time.time()
#     self.src_index_dict,self.trg_index_dict={},{}
#     src_retr_dict={}
#     for ut in self.unique_src_tokens: 
#       if ut.isdigit() and exclude_numbers: continue
#       self.src_index_dict[ut]=get_word_indexes(ut,"src",retr_align_params0)
#     for ut in self.unique_trg_tokens: 
#       if ut.isdigit() and exclude_numbers: continue
#       self.trg_index_dict[ut]=get_word_indexes(ut,"trg",retr_align_params0)
#     elapsed=time.time()-t0
#     print("src:",len(self.unique_src_tokens),"trg:",len(self.unique_trg_tokens),"elapsed:",round(elapsed,4))
#     #print("now mtaching src -trg tokens/phrases")
#     #Now we start the matching between src/trg tokens
#     t0=time.time()

#     src_output=get_phrase_locs_indexes(self.src_tokens_filtered_padded,self.src_index_dict,max_phrase_len)
#     trg_output=get_phrase_locs_indexes(self.trg_tokens_filtered_padded,self.trg_index_dict,max_phrase_len)
#     matching_list=[]
#     phrase_count_src_dict,phrase_count_trg_dict={},{} #how many instances of each src/trg phrases
#     for src_phrase_str,src_indexes_locs in src_output.items(): phrase_count_src_dict[src_phrase_str]=len(src_indexes_locs[0])
#     for trg_phrase_str,trg_indexes_locs in trg_output.items(): phrase_count_trg_dict[trg_phrase_str]=len(trg_indexes_locs[0])

#     for src_phrase_str,src_indexes_locs in src_output.items():
#       src_locs0,src_indexes0=src_indexes_locs
#       for trg_phrase_str,trg_indexes_locs in trg_output.items():
#         trg_locs0,trg_indexes0=trg_indexes_locs
#         if src_phrase_str==trg_phrase_str: ratio1,intersection1=1., 1000
#         else: ratio1,intersection1=get_index_match_ratio_count(src_indexes0,trg_indexes0)
#         min_locs_count=min(len(src_locs0),len(trg_locs0)) #we subtract from the number of locs of each phrase the minimum
        
#         if ratio1==0: continue
#         matching_list.append((src_phrase_str,trg_phrase_str,src_locs0,trg_locs0,min_locs_count,intersection1,ratio1))

#     matching_list.sort(key=lambda x:-x[-1])
#     new_matching_list=[]
#     for ml in matching_list:
#       src_phrase_str,trg_phrase_str,src_locs0,trg_locs0,min_locs_count,intersection1,ratio1=ml
#       if phrase_count_src_dict[src_phrase_str]>0 or phrase_count_trg_dict[trg_phrase_str]>0:
#         new_matching_list.append((src_phrase_str,trg_phrase_str,src_locs0,trg_locs0,intersection1,ratio1))
#       phrase_count_src_dict[src_phrase_str]=phrase_count_src_dict[src_phrase_str]-min_locs_count #subtract the minimum count of instances from both src and trg
#       phrase_count_trg_dict[trg_phrase_str]=phrase_count_trg_dict[trg_phrase_str]-min_locs_count
    
#     for src_punc0,trg_punc0 in punc_pairs0.items(): #now let's match punctuation from the original tokens
#       punc_src_locs=self.src_tok_loc_dict.get(src_punc0,[])
#       if punc_src_locs==[]: continue
#       punc_trg_locs=self.trg_tok_loc_dict.get(trg_punc0,[])
#       if punc_trg_locs==[]: continue
#       ratio1,intersection1=0.2, 100
#       src_punc0,trg_punc0=tuple([src_punc0]),tuple([trg_punc0])
#       punc_src_locs=[[v] for v in punc_src_locs]
#       punc_trg_locs=[[v] for v in punc_trg_locs]
#       new_matching_list.append((src_punc0,trg_punc0,punc_src_locs,punc_trg_locs,intersection1,ratio1))

#     new_matching_list.sort(key=lambda x:-x[-1])
#     elapsed=time.time()-t0
#     print("finished matching words and phrases:", elapsed)
#     #Now we have the matching list - let's align
#     el_dict={}
#     for a in new_matching_list:
#       src_phrase_str,trg_phrase_str,src_locs0,trg_locs0,intersection1,ratio1=a
#       for sl0 in src_locs0:
#         s_min,s_max=sl0[0],sl0[-1]
#         for tl0 in trg_locs0:
#           t_min,t_max=tl0[0],tl0[-1]
#           el0=((s_min,s_max) ,(t_min,t_max))
#           el_dict[el0]=ratio1
    
#     used_el_pairs={}
#     child_dict={}
#     top_wt=0
#     full_src_span0=(0,len(self.src_tokens_filtered_padded)-1)
#     full_trg_span0=(0,len(self.trg_tokens_filtered_padded)-1)
#     full_el=(full_src_span0,full_trg_span0)
#     for epoch0 in range(20):
#       print("epoch0",epoch0)
#       all_elements=list(el_dict.items())
#       all_elements.sort()
#       for el_i,el_item0 in enumerate(all_elements):
#         el0,el_wt0=el_item0
#         src_span0,trg_span0=el0
#         for el_item1 in all_elements[el_i+1:]:
#           if used_el_pairs.get((el_item0,el_item1),False): continue
#           el1,el_wt1=el_item1
#           el_pair_key=(el0,el1)
#           src_span1,trg_span1=el1
#           if src_span0==src_span1 or trg_span0==trg_span1: continue
#           #if src_span1[0]<=src_span0[1]: continue
#           src_span_dist=get_span_dist(src_span0,src_span1)
#           trg_span_dist=get_span_dist(trg_span0,trg_span1)
#           if trg_span_dist<1 or src_span_dist<1: continue
#           if src_span_dist> max_phrase_dist: continue
#           used_el_pairs[(el_item0,el_item1)]=True
#           combined_el=combine_els(el0,el1)
#           combined_wt=el_wt0+el_wt1
#           found_wt=el_dict.get(combined_el,0)
#           if combined_wt>found_wt:
#             el_dict[combined_el]=combined_wt
#             child_dict[combined_el]=(el0,el1)
#         #print("----")
#       cur_full_wt=el_dict.get(full_el,0)
#       print("full_el",full_el,"cur_full_wt",cur_full_wt)
#       if cur_full_wt>0 and cur_full_wt==top_wt: break
#       top_wt=cur_full_wt
#     if cur_full_wt>0:
#       align_list=get_rec_el_children(full_el,child_dict,el_list0=[])
#     else:
#       wt_list=list(el_dict.items())
#       wt_list.sort(key=lambda x:-x[1])
#       top_el=wt_list[0][0]
#       print("couldn't find full el:",full_el, "used the top el instead", top_el)
#       align_list=get_rec_el_children(top_el,child_dict,el_list0=[])
#     for s_al,t_al in align_list:
#       print(">>>", s_al, self.src_tokens_padded[s_al[0]:s_al[1]+1])
#       print(">>>", t_al, self.trg_tokens_padded[t_al[0]:t_al[1]+1])
#       print("----")


NameError: ignored

In [ ]:
def get_unigram_locs(tok_list0):
  enum_list=[(v,i) for i,v in enumerate(tok_list0)]
  enum_list.sort()
  grouped=[(key,[v[1] for v in list(group)]) for key,group in groupby(enum_list,lambda x:x[0])]
  loc_dict0=dict(iter(grouped))
  return loc_dict0

sent="this is, indeed, what I thought. But this is not, at all, everything I wanted."
toks1=tok(sent)
out=get_unigram_locs(toks1)
print(toks1)
print(out)

['this', 'is', ',', 'indeed', ',', 'what', 'I', 'thought', '.', 'But', 'this', 'is', 'not', ',', 'at', 'all', ',', 'everything', 'I', 'wanted', '.']
{',': [2, 4, 13, 16], '.': [8, 20], 'But': [9], 'I': [6, 18], 'all': [15], 'at': [14], 'everything': [17], 'indeed': [3], 'is': [1, 11], 'not': [12], 'this': [0, 10], 'thought': [7], 'wanted': [19], 'what': [5]}


#OLD - Retrieve sentence - 1-Nov-2022

In [ ]:
import json
index_dir="indexes/un/exp2"
params_fpath=os.path.join(index_dir,"params.json")
with open(params_fpath) as params_fopen:
  filter_params=json.load(params_fopen) #token filtering parameteres
print(filter_params)

en_ar_corpus_combined_fname="UNv1.0.en-ar-combined.txt"




# for a in tokens1:
#   print(a)
#print(tokens1)
def get_phrase_locs(tokens0): #identify locations of phrases of content words - excluding empty string/filtered tokens
  content_words_locs=[w_i for w_i,wd0 in enumerate(tokens0) if wd0!=""]
  all_phrases0=[]
  for ii0,cur_loc0 in enumerate(content_words_locs):
    cur_phrase=[cur_loc0]
    all_phrases0.append(list(cur_phrase))
    next_locs=content_words_locs[ii0+1:]
    for nl in next_locs:
      cur_phrase+=[nl]
      all_phrases0.append(list(cur_phrase))
  all_phrase_locs_str=[]
  for ap0 in all_phrases0:
    phrase_tokens=tokens0[ap0[0]:ap0[-1]+1]
    all_phrase_locs_str.append((tuple(phrase_tokens),ap0))
  return all_phrase_locs_str

def combine_phrase_locs(all_phrase_locs_str0):
  phrase_locs_str_sorted=sorted(all_phrase_locs_str0)
  combined_phrase_locs=[(key,[v[1] for v in list(group)]) for key,group in groupby(phrase_locs_str_sorted,lambda x:x[0])]
  phrase_locs_dict0=dict(iter(combined_phrase_locs)) # key: phrase token strings > vals: corresponding locations
  return phrase_locs_dict0


def get_phrase_locs_indexes(tokens0,res_dict0): 
  all_phrases_locs_str=get_phrase_locs(tokens0) #get the geometric listing of all possible phrases made of content words
  phrase_locs_dict=combine_phrase_locs(all_phrases_locs_str)

  all_phrases_locs_str.sort(key=lambda x:len(x[1])) #sort them by size
  phrase_result_dict={} #we put the results in a dictionary, where the keys are the phrase string, and the values are the index/results with proper offset
  for phrase_tokens,phrase0 in all_phrases_locs_str:
    found_phrase_indexes=phrase_result_dict.get(phrase_tokens) #check if this phrase was found in previous iterations
    if found_phrase_indexes!=None: #if it was found, we skip to the next
      #print("found already", phrase_tokens)
      continue
    #print(phrase_tokens,phrase0)
    prev_tokens=[] #to check if the phrase is just a single token or not
    last_item,prev_items=phrase0[-1],phrase0[:-1] #we need to get the location of the last token
    if len(prev_items)>0: prev_tokens=tokens0[prev_items[0]:prev_items[-1]+1] #if the phrase has multiple tokens, identify previous tokens // the ones before the last token
    first_item=phrase0[0] #the location of the first token
    last_word=tokens0[last_item] #the string of the last token
    last_offset=last_item-first_item #the offset of the last token (location of last token minus location of first token)
    cur_word_indexes=res_dict0.get(last_word,{})  #now retrieve the indexes for each word
    cur_offset_indexes=offset_results(cur_word_indexes,last_offset) #offset the indexes according to the current offset
    found_prev_tokens_index_dict=phrase_result_dict.get(tuple(prev_tokens),{}) #check if previous tokens in the phrase have indexes that were found already
    if len(prev_items)>0 and found_prev_tokens_index_dict=={}: continue #if there are previous tokens, but no indexes found for them - NEED TO CHECK
    if prev_items!=[]: 
      cur_offset_indexes=get_index_intersection(found_prev_tokens_index_dict,cur_offset_indexes) #If there are previous tokens and indexes, find the intersection -ALSO CHECK THIS
    phrase_result_dict[phrase_tokens]=cur_offset_indexes
  new_dict={}
  for a,b in phrase_locs_dict.items():
    new_dict[a]=[b,phrase_result_dict.get(a,{})]
    
  return new_dict

  # for a,phrase_results0 in phrase_result_dict.items():
  #   #print(a,b.keys())
  #   combined_results=count_results(phrase_results0)
  #   phrase_str=" ".join(a)
  #   print(phrase_str, n_results)
  #   print("---")
  #   t1=time.time()
  #   lines=get_result_lines(phrase_results0,en_ar_corpus_combined_fname,n_results1=5)
  #   elapsed2=time.time()-t1
  #   print("elapsed2",round(elapsed2,4))
  #   for a in lines:
  #     print(">>>",phrase_str)
  #     src,trg=a[:2]
  #     print(src)
  #     print(trg)
  #     print("----")  
  #   print("==========")


retr_params={} #retrieval parameters
retr_params["index_dir"]="indexes/un/exp2"
retr_params["meta_suffix"]="meta2"
retr_params["max_n_index_files"]=10

sent='The Regular Process is in its third cycle (2021–2025), with the first and second World Ocean Assessments published in 2016 and 2021, respectively.'
#sent='The Regular Process is in its third cycle'
sent='Contribution made by Sustainable Development Goal 14 to achieving other Goals'
sent='Statements were made by the representatives of 25 States and statements were also made by others'
tokens=tok(sent)
print("tokens",tokens)
tokens1=filter_toks(tokens,filter_params) #need to make filter_tokens > OLD
print(tokens1)
unique_tokens=list(set([v for v in tokens1 if v!=""]))
src_retr_dict={}
for ut in unique_tokens:
  print(ut)
  src_retr_dict[ut]=get_word_indexes(ut,"src",retr_params)

tokens1=["<s>"]+tokens1+["</s>"]

# phrase_locs_str=get_phrase_locs(tokens1)
# phrase_locs_str_sorted=sorted(phrase_locs_str)
# combined_phrase_locs=[(key,[v[1] for v in list(group)]) for key,group in groupby(phrase_locs_str_sorted,lambda x:x[0])]
# for phrase_str,indexes_locs in combined_phrase_locs:
#   indexes0,locs0=b
#   listed_results=list_results(indexes0)
#   print(phrase_str)
#   print(locs0)
#   print(len(listed_results),listed_results[:5])
#   print("------")


output=get_phrase_locs_indexes(tokens1,src_retr_dict)
for phrase_str,indexes_locs in output.items():
  locs0,indexes0=indexes_locs
  #print("indexes0",indexes0)
  listed_results=list_results(indexes0)
  if len(listed_results)==0: continue
  print(phrase_str)
  print(locs0)
  print(len(listed_results),listed_results[:5])
  print("------")
  lines=get_result_lines(indexes0,en_ar_corpus_combined_fname,n_results1=3)
  # elapsed2=time.time()-t1
  # print("elapsed2",round(elapsed2,4))
  for a in lines:
    #print(">>>",phrase_str)
    src,trg=a[:2]
    print(src)
    print(trg)
    print("----")  
  print("==========")


# all_phrases=get_phrase_locs(tokens1)
# all_phrases.sort(key=lambda x:len(x))
# phrase_result_dict={}
# for phrase0 in all_phrases:
#   phrase_tokens=tokens1[phrase0[0]:phrase0[-1]+1]
#   prev_tokens=[]
#   last_item,prev_items=phrase0[-1],phrase0[:-1]
#   if len(prev_items)>0: prev_tokens=tokens1[prev_items[0]:prev_items[-1]+1]
#   last_word=tokens1[last_item]
#   cur_word_indexes=src_retr_dict.get(last_word,{}) 
#   cur_offset_indexes=offset_results(cur_word_indexes,last_item)
#   found_prev_tokens_index_dict=phrase_result_dict.get(tuple(prev_tokens),{})
#   if len(prev_items)>0 and found_prev_tokens_index_dict=={}: continue
#   if prev_items!=[]: 
#     cur_offset_indexes=get_index_intersection(found_prev_tokens_index_dict,cur_offset_indexes)
#   cur_count0=count_results(cur_offset_indexes)
#   if cur_count0>0: phrase_result_dict[tuple(phrase_tokens)]=cur_offset_indexes

# for a,phrase_results0 in phrase_result_dict.items():
#   #print(a,b.keys())
#   combined_results=count_results(phrase_results0)
#   phrase_str=" ".join(a)
#   print(phrase_str, n_results)
#   print("---")
#   t1=time.time()
#   lines=get_result_lines(phrase_results0,en_ar_corpus_combined_fname,n_results1=5)
#   elapsed2=time.time()-t1
#   print("elapsed2",round(elapsed2,4))
#   for a in lines:
#     print(">>>",phrase_str)
#     src,trg=a[:2]
#     print(src)
#     print(trg)
#     print("----")  
#   print("==========")



# for ii0,cur_loc0 in enumerate(content_words_locs):
#   next_locs=content_words_locs[ii0+1:]
#   print(cur_loc0, next_locs)
#   for ii1,next_loc0 in enumerate(next_locs):
#     print(content_words_locs[ii0:ii0+ii1+1], next_loc0)
  





{'excluded_words': ['the', 'a', 'an', 'and', 'of', 'its', 'his', 'her', 'their', 'them', 'to', 'is', 'are', 'am', 'been', 'have', 'has', 'had', 'in', 'on', 'at', 'with', 'from', 'for', 'that', 'he', 'she', 'it', 'would', 'من', 'على', 'في', 'مع', 'إلى', 'له', 'تم', 'إلى', 'في'], 'max_sent_n_tokens': 1000, 'lang1': 'en', 'lang2': 'ar', 'exclude_single_chars': True, 'batch_size': 250000, 'min_n_indexes_per_token': 3}
tokens ['Statements', 'were', 'made', 'by', 'the', 'representatives', 'of', '25', 'States', 'and', 'statements', 'were', 'also', 'made', 'by', 'others']
['statements', 'were', 'made', 'by', '', 'representatives', '', '25', 'states', '', 'statements', 'were', 'also', 'made', 'by', 'others']
statements
25
states
made
also
were
others
by
representatives
('25',)
[[8]]
14711 [794.006, 1115.016, 154074.011, 220431.009, 252545.006]
------
25/ KPMG, "Cross-border M & A recovers", op. cit.
)25( KPMG, "Cross-border M & A recovers", op.
----
25/ UNCTC, Foreign Direct Investment and Indu

In [ ]:
#!python retrieve-full.py
!chmod 777 ./pypy3.9-v7.3.9-linux64/bin/pypy
!./pypy3.9-v7.3.9-linux64/bin/pypy retrieve-full.py

{'excluded_words': ['the', 'a', 'an', 'and', 'of', 'its', 'his', 'her', 'their', 'them', 'to', 'is', 'are', 'am', 'been', 'was', 'were', 'be', 'have', 'has', 'had', 'in', 'on', 'at', 'with', 'from', 'for', 'that', 'he', 'she', 'it', 'would', 'من', 'على', 'في', 'مع', 'إلى', 'له', 'تم', 'إلى', 'في'], 'max_sent_n_tokens': 1000, 'lang1': 'en', 'lang2': 'ar', 'exclude_single_chars': True, 'batch_size': 250000, 'min_n_indexes_per_token': 3}
src: 15 trg: 20 elapsed: 8.5834
now mtaching src -trg tokens/phrases
(('await', '', 'report'), ('أن',), 6.0339833944776985e-06)
(('reflected', '', '', 'committee'), ('أن',), 1.2068112426535365e-05)
(('committee', '', 'final'), ('لجنة',), 1.7494751574527642e-05)
(('committee', '', 'final', 'report'), ('لجنة',), 1.7494751574527642e-05)
(('reflected', '', '', 'committee'), ('هذا',), 2.183263105036788e-05)
(('such', '', 'zone'), ('أن',), 2.4135351048681e-05)
(('such', '', 'zone'), ('إن',), 2.5555186425084973e-05)
(('reflected', '', '', 'committee'), ('إن',), 

In [ ]:
!python retrieve-full.py

{'excluded_words': ['the', 'a', 'an', 'and', 'of', 'its', 'his', 'her', 'their', 'them', 'to', 'is', 'are', 'am', 'been', 'was', 'were', 'be', 'have', 'has', 'had', 'in', 'on', 'at', 'with', 'from', 'for', 'that', 'he', 'she', 'it', 'would', 'من', 'على', 'في', 'مع', 'إلى', 'له', 'تم', 'إلى', 'في'], 'max_sent_n_tokens': 1000, 'lang1': 'en', 'lang2': 'ar', 'exclude_single_chars': True, 'batch_size': 250000, 'min_n_indexes_per_token': 3}
src: 15 trg: 20 elapsed: 1.1892
now mtaching src -trg tokens/phrases
(('await', '', 'report'), ('أن',), 6.0339833944776985e-06)
(('reflected', '', '', 'committee'), ('أن',), 1.2068112426535365e-05)
(('committee', '', 'final'), ('لجنة',), 1.7494751574527642e-05)
(('committee', '', 'final', 'report'), ('لجنة',), 1.7494751574527642e-05)
(('reflected', '', '', 'committee'), ('هذا',), 2.183263105036788e-05)
(('such', '', 'zone'), ('أن',), 2.4135351048681e-05)
(('such', '', 'zone'), ('إن',), 2.5555186425084973e-05)
(('reflected', '', '', 'committee'), ('إن',), 

In [ ]:
# import time, json
# index_dir="indexes/un/exp2"
# en_ar_corpus_combined_fname="UNv1.0.en-ar-combined.txt"
# def go2line(fpath0,loc0,size0=None):
#   fopen0=open(fpath0)
#   fopen0.seek(loc0)
#   #line0=fopen0.readline()
#   if size0!=None: line0=fopen0.read(size0)
#   else: line0=fopen0.readline()
#   line0=line0.split("\n")[0]
#   fopen0.close()
#   return line0

# def get_word_indexes(word1,index_prefix1,index_dir1,meta_suffix1="meta2",max_n_index_files=50):
#   cur_meta_shelve_path=os.path.join(index_dir1,"%s-%s.shelve"%(prefix,meta_suffix1))
#   cur_meta_shelve_open=shelve.open(cur_meta_shelve_path)
#   cur_meta_found=cur_meta_shelve_open.get(word1,{})
#   result_dict={}
#   n_results=0
#   elapsed_list=[]
#   t0=time.time()
#   meta_index_items=list(cur_meta_found.items())
#   for index_id,index_loc in meta_index_items[:max_n_index_files]:
#     cur_loc0,cur_size0=index_loc
#     cur_txt_fpath=os.path.join(index_dir1,"%s-%s.txt"%(index_prefix1,index_id)) 
#     cur_meta_line=go2line(cur_txt_fpath,cur_loc0,cur_size0)
#     cur_raw_indexes_str=cur_meta_line.strip().split("\t")[1]
#     cur_raw_indexes=json.loads(cur_raw_indexes_str)
#     result_dict[index_id]=cur_raw_indexes
#     n_results+=len(cur_raw_indexes)
#   result_dict["n"]=n_results
#   return result_dict
#     # elapsed2=time.time()-t1
#     # elapsed_list.append((cur_txt_fpath,len(cur_raw_indexes),round(elapsed,4),"elapsed2",round(elapsed2,4)))
#     # t0=time.time()
#     # t1=time.time()


# # word1="يحاول"
# # prefix="trg"

# word1="associated"
# prefix="src"
# # src_meta_shelve_fpath=os.path.join(index_dir,"src-meta1.shelve")
# # trg_meta_shelve_fpath=os.path.join(index_dir,"trg-meta1.shelve")
# # # src_meta_shelve_fpath=os.path.join(index_dir,"src-meta.shelve")
# # # trg_meta_shelve_fpath=os.path.join(index_dir,"trg-meta.shelve")

# # src_meta_shelve_open=shelve.open(src_meta_shelve_fpath)
# # trg_meta_shelve_open=shelve.open(trg_meta_shelve_fpath)

# cur_meta_shelve_path=os.path.join(index_dir,"%s-meta2.shelve"%prefix)
# cur_meta_shelve_open=shelve.open(cur_meta_shelve_path)


# #src_meta_found=src_meta_shelve_open.get(word1,{})
# cur_meta_found=cur_meta_shelve_open.get(word1,{})
# print(cur_meta_found)
# result_dict={}
# n_results=0
# elapsed_list=[]
# t0=time.time()
# #for index_id,index_loc in src_meta_found.items():
# print("retrieving meta indexes")
# meta_index_items=list(cur_meta_found.items())
# for index_id,index_loc in meta_index_items[:50]:
#   cur_loc0,cur_size1=index_loc
#   #print(index_id,index_loc)
#   cur_txt_fpath=os.path.join(index_dir,"%s-%s.txt"%(prefix,index_id)) 
#   cur_meta_line=go2line(cur_txt_fpath,cur_loc0,cur_size1)

#   elapsed=time.time()-t0
#   t0=time.time()
#   t1=time.time()

#   cur_raw_indexes_str=cur_meta_line.strip().split("\t")[1]
#   cur_raw_indexes=json.loads(cur_raw_indexes_str)
#   result_dict[index_id]=cur_raw_indexes
#   n_results+=len(cur_raw_indexes)
#   elapsed2=time.time()-t1
#   elapsed_list.append((cur_txt_fpath,len(cur_raw_indexes),round(elapsed,4),"elapsed2",round(elapsed2,4)))
#   t0=time.time()
#   t1=time.time()

#   #print(cur_line)

# for el in elapsed_list:
#   print(el)
# print("n_results",n_results)
# final_results=[]
# for index_id,indexes0 in result_dict.items():
#   for ind0 in indexes0[:50]:
#     final_results.append((index_id,ind0))

# final_results.sort(key=lambda x:x[-1]-int(x[-1]))
# for res0 in final_results[:20]:
#   #index_id,ind0,src0,trg0=res0
#   index_id,ind0=res0
#   cur_line0=go2line(en_ar_corpus_combined_fname,int(ind0))
#   src0,trg0=cur_line0.strip().split("\t")  
#   print(index_id)
#   print(ind0)
#   print(src0)
#   print(trg0)
#   print("=============")

# # src_meta_shelve_open.close()
# # trg_meta_shelve_open.close()

# cur_meta_shelve_open.close()


In [ ]:
cur_meta_line

'strive\t[193757760.016, 194033638.046, 195467981.035, 195947014.004, 196053535.011, 196124083.004, 196442913.013, 196444654.013, 197038348.011, 197275843.012, 197355076.053, 198460496.019, 198971034.002, 199012841.01, 199249043.002, 199528309.016, 200228593.002, 201012690.027, 201066332.01, 201128574.012, 201418719.023, 201663728.004, 201977086.008, 203032874.004, 203141216.003, 203527800.004, 203616437.036, 203675639.029, 203776199.044, 204181112.012, 204704173.027, 205203725.074, 205429542.002, 205670254.004, 206138414.013, 206166143.014, 207529492.005, 207785332.012, 207813930.002, 208640226.01, 208643630.014, 211082892.002, 211221058.009, 212237911.017, 212346694.006, 214195584.01, 214766192.008, 216413899.01, 216603699.01, 217324793.006, 220697158.004, 221063736.008, 221381790.008, 221605242.019, 222251009.019, 223521502.016, 223666528.011, 224246850.011, 224646277.01, 225120636.004, 225235082.013, 225929521.024, 226679392.021, 227508747.005, 227784334.004, 227907520.007, 2282249

In [ ]:
loc,size=(2000060, 143881)
#indexes/un/exp2/trg-1000000.txt 31374060 143881
t0=time.time()
fopen=open('indexes/un/exp2/trg-8000000.txt',"r")
fopen.seek(loc)
elapsed1=time.time()-t0
#line0=fopen.readline()
t1=time.time()
#line0=fopen.read(1500)
line0=fopen.readline()
#line0=fopen.readable()
elapsed2=time.time()-t1
fopen.close()
elapsed=time.time()-t0
print("elapsed2",round(elapsed2,4))
#print("elapsed1",round(elapsed1,4), "elapsed",round(elapsed,4),"elapsed2",round(elapsed2,4))
print(line0)

elapsed2 0.0058
.0, 3089484747.0, 3089497480.0, 3090260310.0, 3090543473.0, 3090570775.0, 3090602901.0, 3097994799.0, 3099767982.002, 3100424987.0, 3101687175.0, 3102794931.0, 3104508507.02, 3105288064.0, 3105302876.012, 3109732376.001, 3109833320.024, 3113908363.002, 3118960535.0, 3120778671.002, 3122309964.0, 3123153492.015, 3123998217.0, 3124133360.0, 3124270994.0, 3124295833.0, 3124341351.0, 3124363501.0, 3124370295.0, 3124376345.0, 3124470885.0, 3124480897.0, 3124492758.0, 3124521782.0, 3124575691.0, 3124620941.0, 3124699506.0, 3127049711.0, 3127057068.003, 3127590328.0, 3127598557.003, 3127791439.0, 3127804697.003, 3128142220.0, 3130580272.0]



In [ ]:
!tail -50 indexes/un/exp2/src-17000000.txt
fpath='UNv1.0.en-ar-combined.txt'
fopen=open(fpath)
fopen.seek(7335429745)
fopen.seek(6772369758)

line=fopen.readline()
fopen.close()
print(">>> line:", line)

zerrougui's	[6770623081.007, 6770625635.005, 6770626601.028, 6770678966.012, 6770679473.015, 6770694882.01, 6770713382.028, 6770713796.016, 6770715900.017, 6770716405.1, 6770718769.004, 6770721053.005, 6770723215.034]
zeyad	[6773280361.003, 6773283717.016, 6773284763.007, 6773290992.033, 6773312224.057, 6773326591.031]
zeydulla	[6772369758.003, 6772373237.016, 6772374254.007]
zfns	[6756340178.056, 6756341756.006, 6756428599.056, 6756430216.006]
zhang	[6672990318.001, 6675447682.002, 6676228619.002, 6676634522.002, 6676738713.002, 6678563929.002, 6678969186.002, 6684981481.0, 6685022241.012, 6685022769.001, 6685752053.014, 6713586327.001, 6767605704.002, 6767676965.001]
zhangshi	[6762624457.015, 6762645025.015, 6762667308.034]
zhanna	[6769020395.003, 6769023397.016, 6769024405.007]
zhao	[6670350562.002, 6680578568.001, 6680596608.001, 6681579483.002, 6696763730.001, 6696794336.001, 6696798679.001, 6696808951.002]
zheglov	[6675825834.002, 6676131023.002, 6676273071.002, 6676447241.002]
z

In [ ]:
# #preparing index shelves
# import shelve, json, time
# en_ar_corpus_combined_fname="UNv1.0.en-ar-combined.txt"
# index_dir="indexes/un/exp2"
# if not os.path.exists(index_dir): os.makedirs(index_dir)

# max_sent_n_tokens=1000
# lang1="en"
# lang2="ar"

# cur_params={}
# cur_params["excluded_words"]=excluded_src_tokens+excluded_trg_tokens
# cur_params["max_sent_n_tokens"]=max_sent_n_tokens
# cur_params["lang1"]=lang1
# cur_params["lang2"]=lang2
# cur_params["exclude_single_chars"]=True
# params_fpath=os.path.join(index_dir,"params.json")
# params_fopen=open(params_fpath,"w")
# json.dump(cur_params,params_fopen)
# params_fopen.close()

# index_list_fpath=os.path.join(index_dir,"index-list.txt")
# last_indexed=None
# if os.path.exists(index_list_fpath):
#   index_list_fopen=open(index_list_fpath)
#   index_list_lines=[v.strip() for v in index_list_fopen.readlines()]
#   index_list_fopen.close()
#   if len(index_list_lines)>0: last_indexed=int(index_list_lines[-1])

# print("last_indexed",last_indexed)

# fopen=open(en_ar_corpus_combined_fname)
# cur_loc=0
# cur_line="-"
# counter=0
# src_fwd_index0,trg_fwd_index0=[],[]
# t0=time.time()
# while cur_line!="":
# #for i0,f0 in enumerate(fopen):
# #for i0 in range(10):  
#   #if counter>100: break

#   cur_loc=fopen.tell()
#   f0=fopen.readline()
  
#   counter+=1

#   #if counter>1000000: break
#   if (counter%50000==0 and counter>0) or f0=="": 
#     print(counter)
    
#     #continue
#     cur_src_shelve_fpath=os.path.join(index_dir,"src-%s.shelve"%counter) 
#     cur_trg_shelve_fpath=os.path.join(index_dir,"trg-%s.shelve"%counter)
#     if os.path.exists(cur_src_shelve_fpath) and os.path.exists(cur_src_shelve_fpath):
#       src_fwd_index0,trg_fwd_index0=[],[]
#       #counter+=1
#       continue

#     src_fwd_index0.sort()
#     trg_fwd_index0.sort()
#     grouped_src=[(key,[v[1] for v in list(group)]) for key,group in groupby(src_fwd_index0,lambda x:x[0])]
#     grouped_trg=[(key,[v[1] for v in list(group)]) for key,group in groupby(trg_fwd_index0,lambda x:x[0])] 
#     cur_src_shelve_open=shelve.open(cur_src_shelve_fpath)
#     cur_trg_shelve_open=shelve.open(cur_trg_shelve_fpath)
#     for k0,grp0 in grouped_src: cur_src_shelve_open[k0]=grp0
#     for k0,grp0 in grouped_trg: cur_trg_shelve_open[k0]=grp0
#     cur_src_shelve_open.close()
#     cur_trg_shelve_open.close()
#     src_fwd_index0,trg_fwd_index0=[],[]
#     print("finished indexing:",cur_src_shelve_fpath,cur_trg_shelve_fpath, "elapsed:", time.time()-t0)
#     t0=time.time()
#     index_list_fopen=open(index_list_fpath,"a")
#     index_list_fopen.write("%s\n"%counter)
#     #index_list_fopen.write(json.dumps([cur_src_shelve_fpath,cur_trg_shelve_fpath])+"\n")
#     index_list_fopen.close()

#   if last_indexed!=None and counter<last_indexed: continue
#   cur_line=f0.strip("\n\r\t")
#   line_split=cur_line.split("\t")
#   if len(line_split)!=2: continue
#   src0,trg0=line_split
#   src_tokens0=tok(src0)#,tok(trg0)
#   if lang2=="ar":trg_tokens0=cur_ar_tok_fn(trg0)
#   else: trg_tokens0=tok(trg0)

#   #print(fopen.tell())
#   filtered_src0=filter_toks(src_tokens0,cur_params)
#   filtered_trg0=filter_toks(trg_tokens0,cur_params)
#   indexed_src0=[(v,cur_loc+vi/max_sent_n_tokens) for vi,v in enumerate(filtered_src0) if vi<max_sent_n_tokens and v!=""]
#   indexed_trg0=[(v,cur_loc+vi/max_sent_n_tokens) for vi,v in enumerate(filtered_trg0) if vi<max_sent_n_tokens and v!=""]
#   src_fwd_index0.extend(indexed_src0)
#   trg_fwd_index0.extend(indexed_trg0)
#   #counter+=1
#   #cur_loc+=len(f0)
# fopen.close()
# src_fwd_index0.sort()
# trg_fwd_index0.sort()
# # grouped_src=[(key,[v[1] for v in list(group)]) for key,group in groupby(src_fwd_index0,lambda x:x[0])]
# # grouped_trg=[(key,[v[1] for v in list(group)]) for key,group in groupby(trg_fwd_index0,lambda x:x[0])]

# # src_shelve_fpath="test-src.shelve"
# # src_shelve_fopen=shelve.open(src_shelve_fpath)
# # for k0,grp0 in grouped_src: src_shelve_fopen[k0]=grp0
# # src_shelve_fopen.close()
# # trg_shelve_fpath="test-trg.shelve"
# # trg_shelve_fopen=shelve.open(trg_shelve_fpath)
# # for k0,grp0 in grouped_trg: trg_shelve_fopen[k0]=grp0
# # trg_shelve_fopen.close()


# # print("src_fwd_index0",len(src_fwd_index0))
# # print("trg_fwd_index0",len(trg_fwd_index0))
# # print("grouped_src",len(grouped_src))
# # print("grouped_trg",len(grouped_trg))


last_indexed 17300000
50000
finished indexing: indexes/un/exp1/src-50000.shelve indexes/un/exp1/trg-50000.shelve elapsed: 6.691086530685425
100000
finished indexing: indexes/un/exp1/src-100000.shelve indexes/un/exp1/trg-100000.shelve elapsed: 5.057390928268433
150000
finished indexing: indexes/un/exp1/src-150000.shelve indexes/un/exp1/trg-150000.shelve elapsed: 4.746932506561279
200000
finished indexing: indexes/un/exp1/src-200000.shelve indexes/un/exp1/trg-200000.shelve elapsed: 4.8925981521606445
250000
finished indexing: indexes/un/exp1/src-250000.shelve indexes/un/exp1/trg-250000.shelve elapsed: 4.951043605804443
300000
finished indexing: indexes/un/exp1/src-300000.shelve indexes/un/exp1/trg-300000.shelve elapsed: 4.826070547103882
350000
finished indexing: indexes/un/exp1/src-350000.shelve indexes/un/exp1/trg-350000.shelve elapsed: 5.300532579421997
400000
finished indexing: indexes/un/exp1/src-400000.shelve indexes/un/exp1/trg-400000.shelve elapsed: 4.924514293670654
450000
finis

#Retrieving tokens and phrases - 27-Oct-2012
from multiple shelves sr/trg

In [ ]:
import shelve, json, time
en_ar_corpus_combined_fname="UNv1.0.en-ar-combined.txt"
index_dir="indexes/un/exp1"
index_list_fpath=os.path.join(index_dir,"index-list.txt")
index_list_lines=[]
if os.path.exists(index_list_fpath):
  index_list_fopen=open(index_list_fpath)
  index_list_lines=[v.strip() for v in index_list_fopen.readlines()]
  index_list_fopen.close()
index_list_lines=sorted(list(set(index_list_lines)),key=lambda x:int(x))
print("index_list_lines",index_list_lines)
combined_src_shelve_fpath=os.path.join(index_dir,"src-combined.shelve") 
combined_trg_shelve_fpath=os.path.join(index_dir,"trg-combined.shelve")
combined_src_shelve_fopen=shelve.open(combined_src_shelve_fpath)
combined_trg_shelve_fopen=shelve.open(combined_trg_shelve_fpath)

src_tok_counter,trg_tok_counter={},{}
shelve_src_trg_handles_dict={}
for item0 in index_list_lines[:5]:
  cur_src_shelve_fpath=os.path.join(index_dir,"src-%s.shelve"%item0) 
  cur_trg_shelve_fpath=os.path.join(index_dir,"trg-%s.shelve"%item0)
  print(item0)
  print(cur_src_shelve_fpath)
  print(cur_trg_shelve_fpath)
  cur_src_shelve_fopen=shelve.open(cur_src_shelve_fpath)
  cur_trg_shelve_fopen=shelve.open(cur_trg_shelve_fpath)
  for i0,key0 in enumerate(cur_src_shelve_fopen):
    vals=cur_src_shelve_fopen[key0]
    src_tok_counter[key0]=src_tok_counter.get(key0,0)+len(vals)
    #print(key0,len(vals),vals[:5])
    if i0>1000: break

  #shelve_src_trg_handles_dict[item0]=[cur_src_shelve_fopen,cur_trg_shelve_fopen]
  cur_src_shelve_fopen.close()
  cur_trg_shelve_fopen.close()

cur_items=list(src_tok_counter.items())
cur_items.sort(key=lambda x:-x[-1])
for a,b in cur_items[:200]:
  if b<5: continue
  if len(a)<2: continue
  print(a,b)
# print("opened shelve handles")
# for item0,shelve_handles0 in shelve_src_trg_handles_dict.items():
#   for sh_handle0 in shelve_handles0: sh_handle0.close()
# print("closed shelve handles")
combined_src_shelve_fopen.close()
combined_trg_shelve_fopen.close()

index_list_lines ['50000', '100000', '150000', '200000', '250000', '300000', '350000', '400000', '450000', '500000', '550000', '600000', '650000', '700000', '750000', '800000', '850000', '900000', '950000', '1000000', '1050000', '1100000', '1150000', '1200000', '1250000', '1300000', '1350000', '1400000', '1450000', '1500000', '1550000', '1600000', '1650000', '1700000', '1750000', '1800000', '1850000', '1900000', '1950000', '2000000', '2050000', '2100000', '2150000', '2200000', '2250000', '2300000', '2350000', '2400000', '2450000', '2500000', '2550000', '2600000', '2650000', '2700000', '2750000', '2800000', '2850000', '2900000', '2950000', '3000000', '3050000', '3100000', '3150000', '3200000', '3250000', '3300000', '3350000', '3400000', '3450000', '3500000', '3550000', '3600000', '3650000', '3700000', '3750000', '3800000', '3850000', '3900000', '3950000', '4000000', '4050000', '4100000', '4150000', '4200000', '4250000', '4300000', '4350000', '4400000', '4450000', '4500000', '4550000', '

In [ ]:
is_punct("$")

False

In [ ]:
#from itertools import chain
from code_utils.general import *
import re, time
# def tok_new(str0):
#   pass

removelist=""
#re.sub(r'[^\w'+removelist+']', '',mystring)
# multi_dot_words=["e.g.","i.e.","U.S.A.","U.K.","O.K."," v."," vs."," v.s.", " et al."," etc.", " al."]
# dot_words=["Mr","Ms","Dr","Art","art","Chap","chap","No","no","rev","Rev","Add","para","Para","Paras","paras"]
multi_dot_words=["e.g.","i.e.","u.s.a.","u.k.","o.k."," v."," vs."," v.s.", " et al."," etc.", " al."]
dot_words=["Mr","Ms","Dr","Art","art","Chap","chap","No","no","rev","Rev","Add","para","Para","Paras","paras"]

text="""
This is a new, maybe old, Mr. Ghaly, and Dr. Kala text: U.S.A please check? and e.g. also "WHAT IS THIS?" Here is our 1) question b) answer! The secretary-general 1.687% and $1000 to do this at #metoo @secgen re-imagined this A. B. "here is it." said the monk
what else? A/751/Add.12 and A.12/45/55.
I don't know, probably we should try "this other approach"!
"""
#text="$$what is this $1000 bill @me"
# def tok_new(text0):
#   iter0=re.finditer("^(\W+)(\S+)", text0) #items with non-alpha in the beginning of the string
#   iter1=re.finditer("\s(\W+)(\S+)", text0) #items with non-alpha in the beginning
#   iter2=re.finditer("(\S+?)(\W+)\s", text0) #items ending with non-alpha
#   full_list=[]
#   for a in chain(iter0,iter1,iter2):
#     start_i,end_i=a.span()
#     grp0=a.group(0)#.strip()
#     grp1=a.group(1)
#     grp2=a.group(2)
#     full_list.append((start_i,end_i,grp0,grp1,grp2))
#     #print("grp0",grp0,"grp1",grp1,"grp2",grp2)
#   full_list.sort()
#   final_str=""
#   prev_i=0
#   for a in full_list:
#     start_i,end_i,grp0,grp1,grp2=a
#     final_str+=text0[prev_i:start_i]
#     prev_i=end_i

#     add_space_after,add_space_before=True,True
#     if not grp1[0].isalnum():
#       if grp1.strip() in "@#": add_space_after=False
#       if add_space_after: final_str+=" "+grp1+" "+grp2
#       else: final_str+=" "+grp0
#     else:
#       full0,prev0,cur0=grp0,grp1,grp2
#       print(a)
#       if full0.strip().lower() in multi_dot_words: add_space_before=False
#       if prev0 in dot_words: add_space_before=False
#       if prev0.isdigit() and prev0 in ".-": add_space_before=False
#       if prev0.isupper() and prev0 in ".-": add_space_before=False
#       if len(prev0)==1: add_space_before=False
#       final_str+=prev0
#       if add_space_before: 
#         final_str+=" "
#       final_str+=" ".join(list(cur0))+" "
#   final_str+=text0[prev_i:]
#   out_tokens=[v for v in re.split("\s+",final_str) if v]
#   return out_tokens

  # final_str_tokens=final_str.split()

  # print(final_str_tokens)

ar_text=""" 22-
وقد استضافت الصين بنجاح دورة الألعاب الأولمبية الشتوية للأشخاص ذوي الإعاقة في بيجين 2022، مستفيدة من هذه الفرصة للتعجيل بتطوير خدمات إمكانية الوصول وتعزيز إدماج الأشخاص ذوي الإعاقة في التعليم والتوظيف.
"""
text="""
Ms. Kayess (Chair of the Committee on the Rights of Persons with Disabilities), panellist, said that she recognized all the candidates that demonstrated their commitment to the Convention by standing for election to the Committee on the Rights of Persons with Disabilities.
"""
text="""
ورحب مجلس حقوق الإنسان، في دورته الخامسة والأربعين، في قراره 45/5 بشأن حقوق الإنسان والتدابير القسرية الانفرادية، بالوثيقة الختامية والإعلان المعتمدين في مؤتمر القمة الثامن عشر لرؤساء دول وحكومات حركة بلدان عدم الانحياز، الذي عقد في باكو، أذربيجان، يومي 25 و 26 تشرين الأول/ أكتوبر 2019، وأكدت فيهما الحركة من جديد، ضمن أمور أخرى، موقفها المبدئي المتمثل في إدانة اتخاذ وتطبيق تدابير قسرية انفرادية ضد بلدان الحركة، وهي تدابير تنتهك ميثاق الأمم المتحدة والقانون الدولي، وتقوض، في جملة ما تقوضه، مبادئ السيادة والسلامة الإقليمية والاستقلال السياسي وتقرير المصير وعدم التدخل.

"""
#tok_new(text)
#tok_new(ar_text)


#non_space=re.findall("\S+",text)

def tok_new(text0):
  non_space=text0.split() #simple split by space
  all_tokens0=[]
  for a in non_space:
    leading0,trailing0="","" #get the leading and trailing non-alpha characters
    max_val=min(len(a),3) #iterating a number of characters from the beginning and end of token
    for i0 in range(1,max_val+1): 
      if not a[-i0].isalnum():trailing0=a[-i0]+trailing0
      else: break
    for i0 in range(0,max_val): 
      if not a[i0].isalnum():leading0+=a[i0]
      else: break
    main=a[len(leading0):len(a)-len(trailing0)]
    main_trailing=main+trailing0 #combine main part of the token with the 
    add_space_after,add_space_before=True,True #if we need to add space  between lead/main/trailing parts
    if leading0!="" and leading0 in "#@": add_space_after=False #keep hashtags and mentions
    if main in dot_words or main_trailing.lower() in multi_dot_words: add_space_before=False #things like e.g. U.S.A.
    if main.isdigit() and trailing0 in ".-)(": add_space_before=False #For things like 1. 2) 
    if len(main)==1: add_space_before=False #and A. B.
    cur_items_str="" #start building the string
    if leading0!="": cur_items_str+=" ".join(list(leading0)) #split the leading chars, add them to final str
    if add_space_after: cur_items_str+=" " #if there is a space between leading and main
    cur_items_str+=main #add the main part
    if add_space_before: cur_items_str+=" " #if there is space between main and trailing
    if trailing0!="": cur_items_str+=" ".join(list(trailing0)) #split trailing chars and add them
    all_tokens0.extend(cur_items_str.split()) #split the final string and add it to the list of tokens
  return all_tokens0
#print(all_tokens)
t0=time.time()
for _ in range(1000):
  all_tokens=tok_new(text)
t1=time.time()
print("new tokenizer:", t1-t0)
print(all_tokens)

print("---")
t0=time.time()
for _ in range(1000):
  all_tokens=tok(text)
t1=time.time()
print("old tokenizer:", t1-t0)
print(all_tokens)
# for a in all_tokens:
#   print(a)



new tokenizer: 0.3965146541595459
['ورحب', 'مجلس', 'حقوق', 'الإنسان', '،', 'في', 'دورته', 'الخامسة', 'والأربعين', '،', 'في', 'قراره', '45/5', 'بشأن', 'حقوق', 'الإنسان', 'والتدابير', 'القسرية', 'الانفرادية', '،', 'بالوثيقة', 'الختامية', 'والإعلان', 'المعتمدين', 'في', 'مؤتمر', 'القمة', 'الثامن', 'عشر', 'لرؤساء', 'دول', 'وحكومات', 'حركة', 'بلدان', 'عدم', 'الانحياز', '،', 'الذي', 'عقد', 'في', 'باكو', '،', 'أذربيجان', '،', 'يومي', '25', 'و', '26', 'تشرين', 'الأول', '/', 'أكتوبر', '2019', '،', 'وأكدت', 'فيهما', 'الحركة', 'من', 'جديد', '،', 'ضمن', 'أمور', 'أخرى', '،', 'موقفها', 'المبدئي', 'المتمثل', 'في', 'إدانة', 'اتخاذ', 'وتطبيق', 'تدابير', 'قسرية', 'انفرادية', 'ضد', 'بلدان', 'الحركة', '،', 'وهي', 'تدابير', 'تنتهك', 'ميثاق', 'الأمم', 'المتحدة', 'والقانون', 'الدولي', '،', 'وتقوض', '،', 'في', 'جملة', 'ما', 'تقوضه', '،', 'مبادئ', 'السيادة', 'والسلامة', 'الإقليمية', 'والاستقلال', 'السياسي', 'وتقرير', 'المصير', 'وعدم', 'التدخل', '.']
---
old tokenizer: 0.30153751373291016
['ورحب', 'مجلس', 'حقوق'

In [ ]:
"$".isalnum()

False

##RNN definition

In [ ]:
from collections import OrderedDict
#Let's build the network - here is a small cheat sheet for possible RNN classes based on input and output size
#https://github.com/hmghaly/rnn/blob/master/classes.py

#here the size of the output is the same as the size of the input
#the depth of the output depends on the number of possible outcome categories (e.g. different phonemes)
import torch
import torch.nn as nn
#import torch.nn.functional as F
import torch.optim as optim
import random
import dill as pickle
import numpy as np
from code_utils import np_lstm

torch.manual_seed(1)
random.seed(1)

torch.manual_seed(1)
random.seed(1)

device = torch.device('cpu')
#device = torch.device('cuda')

class RNN(nn.Module):
  def __init__(self, input_size, hidden_size, output_size,num_layers, matching_in_out=False, apply_sigmoid=False, apply_softmax=False, batch_size=1):
    super(RNN, self).__init__()
    self.input_size = input_size
    self.hidden_size = hidden_size
    self.output_size = output_size
    self.num_layers = num_layers
    self.batch_size = batch_size
    self.apply_softmax=apply_softmax
    self.apply_sigmoid=apply_sigmoid
    self.matching_in_out = matching_in_out #length of input vector matches the length of output vector 
    self.lstm = nn.LSTM(input_size, hidden_size,num_layers)
    self.hidden2out = nn.Linear(hidden_size, output_size)
    if self.apply_softmax: self.softmax =nn.Softmax(dim=2)
    if self.apply_sigmoid: self.sigmoid =nn.Sigmoid() 
    
    #self.sigmoid = torch.sigmoid(dim=1)
    self.hidden = self.init_hidden()
  def forward(self, feature_list):
    self.hidden = self.init_hidden() ### check
    feature_list=torch.tensor(feature_list)
    feature_list=feature_list.to(device) #### <<<<<<<<<<<<<<<<< 
    if self.matching_in_out:
      lstm_out, _ = self.lstm( feature_list.view(len( feature_list), 1, -1))
      output_scores = self.hidden2out(lstm_out.view(len( feature_list), -1))
      if self.apply_sigmoid: output_scores=self.sigmoid(output_scores).to(device)
      elif self.apply_softmax: output_scores=self.softmax(output_scores).to(device)
      #output_scores = torch.sigmoid(output_space) #we'll need to check if we need this sigmoid
      return output_scores #output_scores
    else:
      outs=[]
      for i in range(len(feature_list)):
        cur_ft_tensor=feature_list[i]#.view([1,1,self.input_size])
        cur_ft_tensor=cur_ft_tensor.view([1,1,self.input_size])
        lstm_out, self.hidden = self.lstm(cur_ft_tensor, self.hidden)
        outs=self.hidden2out(lstm_out)
        if self.apply_sigmoid: outs = self.sigmoid(outs).to(device) #self.sigmoid =nn.Sigmoid()
        elif self.apply_softmax: outs = self.softmax(outs).to(device)
        
      return outs
  def init_hidden(self):
    #return torch.rand(self.num_layers, self.batch_size, self.hidden_size)
    return (torch.rand(self.num_layers, self.batch_size, self.hidden_size).to(device),
            torch.rand(self.num_layers, self.batch_size, self.hidden_size).to(device))
    
def split_train_test(all_data0,train_ratio=0.8):
  train_size=int(len(all_data0)*train_ratio)
  train_set0,tes_set0=all_data0[:train_size],all_data0[train_size:]
  return train_set0,tes_set0    

#Loading and indexing sentences
new approach - just indexing and word alignment for all the data

## Loading functions

In [ ]:
#main indexing functions
import os, time, json
from collections import Counter


def get_words_vector(words,wv_model,excluded_words=[]):
  wd_counter=Counter(words)
  flag=False
  total_count0=0
  wd_vector_dict={}
  for wd0,wd0_n in wd_counter.items():
    if wd0 in excluded_words: continue
    try: cur_vec0=wv_model[wd0]
    except: continue
    wd_vector_dict[wd0]=cur_vec0
    weighted_vec0=cur_vec0*wd0_n
    if flag==False: total_vec0=weighted_vec0
    else: total_vec0+=weighted_vec0
    flag=True
    total_count0+=wd0_n
  if total_count0==0: return [],{}
  avg_vec=total_vec0/total_count0
  return avg_vec,wd_vector_dict

def file_len(fpath0): #get the number of lines in the cache file
  if not os.path.exists(fpath0): return 0
  with open(fpath0) as fopen:
    for i0,f0 in enumerate(fopen): pass
  return i0+1

def idx_match(idx1,idx2,min_common=5): #This function is to give a ratio of matching between the inverted indexes for two words
    if len(idx1)==0 or len(idx2)==0: return 0
    intersection=len(list(set(idx1).intersection(set(idx2))))
    union=len(idx1)+len(idx2)-intersection
    if union==0: return 0
    if intersection<min_common: return 0    
    return float(intersection)/union


def rtrv_idx(tokens,idx_dict, params0={}): #get the indexes of a phrase - we can extend it to as many tokens as we want
    lower=params0.get("lower",True)
    remove_al=params0.get("remove_al",True)
    cur_indexes=idx_dict.get(tokens[0],[])
    for ti,ta in enumerate(tokens[1:]):
        cur_offset=ti+1
        cur_raw_idx=idx_dict.get(ta,[])
        cur_offset_idx=[(v[0],v[1]-cur_offset) for v in cur_raw_idx]
        cur_indexes=list(set(cur_indexes).intersection(set(cur_offset_idx)))
    cur_indexes=[v[0] for v in cur_indexes]
    cur_indexes=sorted(list(set(cur_indexes)))
    return cur_indexes


class sent_cls:
  def __init__(self,raw_sent0="",raw_tokens0=[],tokens0=[],map0=[]) -> None:
      self.raw_sent=raw_sent0
      self.raw_tokens=raw_tokens0
      self.tokens=tokens0 #filtered tokens
      self.map=map0 #mapping of filtered tokens to raw tokens #raw_tokens,map

class indexing: #get a list of sentences, outputs indexes
  def __init__(self,raw_sentences0,tok_function, params0={}):
    lang=params0.get("lang","en")
    stop_words=params0.get("stop_words",[]) #excluded words
    ignore_punc=params0.get("ignore_punc",False) #skip punctuation or not
    ignore_ar_pre_suf=params0.get("ignore_ar_pre_suf",False) #ignore arabic prefixes and suffixes
    remove_al=params0.get("remove_al", False) #remocve alif laam in Arabic
    index_words=params0.get("index_words", True) #keep the index of each word in a sentence, not just the sentence number
    lower=params0.get("lower", False) #make all words lower case or not
    stemming=params0.get("stemming", False) #stem each word or not
    self.fwd_index=[]
    self.all_tok_sentences=[]
    self.all_tok_original_sentences=[]
    self.all_mappings=[]
    self.sent_obj_list=[]
    self.inv_index={}
    for sent_i,sent0 in enumerate(raw_sentences0):
      #if sent_i%5000==0: print(sent_i)
      tokens=tok_function(sent0)
      #tokens,cur_mapping,original_tokens=filter_tokens(tokens, lang,stop_words, ignore_punc,ignore_ar_pre_suf,remove_al,index_words,lower,stemming)
      tokens,cur_mapping,original_tokens=filter_tokens(tokens, params0)
      self.all_tok_original_sentences.append(original_tokens)
      self.all_mappings.append(cur_mapping)
      enumerated_toks=[(v,sent_i,s_i) for s_i,v in enumerate(tokens)]
      cur_sent_obj0=sent_cls(sent0,original_tokens,tokens,cur_mapping)
      self.sent_obj_list.append(cur_sent_obj0)
      #cur_sent_obj0
      self.all_tok_sentences.append(tokens)
      self.fwd_index.extend(enumerated_toks)
    self.fwd_index.sort()
    grouped=[(key,[v[1:] for v in list(group)]) for key,group in groupby(self.fwd_index,lambda x:x[0])]
    self.inv_index=dict(iter(grouped))

def filter_tokens(token_list0, params0={}):
#def filter_tokens(token_list0, lang="en",stop_words=[], ignore_punc=False,ignore_ar_pre_suf=False,remove_al=True,index_words=True,lower=True,stemming=False):
  tokens_original=list(token_list0)
  tokens_copy_enum=[(i,v) for i,v in enumerate(token_list0)]   
  ignore_punc=params0.get("ignore_punc",False)
  ignore_ar_pre_suf=params0.get("ignore_ar_pre_suf",False)
  remove_al=params0.get("remove_al",False)
  lower=params0.get("lower",False)
  stemming=params0.get("stemming",False)
  lang=params0.get("lang","en")
  stop_words=params0.get("stop_words",[])
  #if lang=="ar": tokens=tok_ar(tokens,count_dict) #tok_ar(tokens)
  if ignore_punc: tokens_copy_enum=[(i,v) for i,v in tokens_copy_enum if not is_punct(v)]
  if ignore_ar_pre_suf: tokens_copy_enum=[(i,v) for i,v in tokens_copy_enum if not v.startswith("ـ") and not v.endswith("ـ")]
  if stop_words: tokens_copy_enum=[(i,v) for i,v in tokens_copy_enum if not v.lower() in stop_words]
  if remove_al: tokens_copy_enum=[(i,v.replace("ال_","")) for i,v in tokens_copy_enum if not v.lower() in stop_words]
  tokens_out=[v[1] for v in tokens_copy_enum]
  if lower: tokens_out=[v.lower() for v in tokens_out]
  cur_mapping=[v[0] for v in tokens_copy_enum]
  return tokens_out,cur_mapping, tokens_original

def get_tok_loc(tok_list):
  tok_locs_list=[]
  for i_,cur_tok in enumerate(tok_list):
    tok_locs_list.append(((cur_tok,),(i_,i_))) #to get unigrams and their locations
    if i_<len(tok_list)-1: tok_locs_list.append(((cur_tok,tok_list[i_+1]), (i_,i_+1))) #to get bigrams and their locations
  tok_locs_list.sort(key=lambda x:x[0])
  grouped=[(key,[v[1] for v in list(group)]) for key,group in groupby(tok_locs_list,lambda x:x[0])]
  return grouped

def get_rec_el_children(el0,el_child_dict0,el_list0=[]):
  cur_children0=el_child_dict0.get(el0,[])
  if cur_children0==[]: el_list0.append(el0)
  for ch0 in cur_children0:
    el_list0=get_rec_el_children(ch0,el_child_dict0,el_list0)
  return el_list0

def combine_els(el1,el2):
  src_span1,trg_span1=el1
  src_span2,trg_span2=el2
  x1_0,x1_1=src_span1
  x2_0,x2_1=src_span2
  y1_0,y1_1=trg_span1
  y2_0,y2_1=trg_span2
  min_x=min(x1_0,x1_1,x2_0,x2_1)
  max_x=max(x1_0,x1_1,x2_0,x2_1)
  min_y=min(y1_0,y1_1,y2_0,y2_1)
  max_y=max(y1_0,y1_1,y2_0,y2_1)
  new_src_span=(min_x,max_x)
  new_trg_span=(min_y,max_y)
  return (new_src_span,new_trg_span)

# def check_span_overlap(span1,span2):
#   span1_x1,span1_x2=span1
#   span2_x1,span2_x2=span2
#   d1=abs(span1_x2-span1_x1)
#   d2=abs(span2_x2-span2_x1)
#   max_x=max(span1_x1,span1_x2,span2_x1,span2_x2)
#   min_x=min(span1_x1,span1_x2,span2_x1,span2_x2)
#   d_total=max_x-min_x
#   overlapping=False
#   if d_total<=d1+d2: overlapping=True
#   return overlapping

def get_span_dist(span1,span2):
  span1_x1,span1_x2=span1
  span2_x1,span2_x2=span2
  d1=abs(span1_x2-span1_x1)
  d2=abs(span2_x2-span2_x1)
  max_x=max(span1_x1,span1_x2,span2_x1,span2_x2)
  min_x=min(span1_x1,span1_x2,span2_x1,span2_x2)
  d_total=max_x-min_x
  return d_total-d1-d2



# def overlaps(el1,el2,max_dist=10):
#   src_span1,trg_span1=el1
#   src_span2,trg_span2=el2
#   x1_0,x1_1=src_span1
#   x2_0,x2_1=src_span2
#   y1_0,y1_1=trg_span1
#   y2_0,y2_1=trg_span2
#   #if x2_0<=x1_1: return True
#   output=False
#   if check_span_overlap(src_span1,src_span2): output=True
#   if check_span_overlap(trg_span1,trg_span2): output=True
#   if get_span_dist(trg_span1,trg_span2)>max_dist: output=True

#   # min_x=min(x1_0,x1_1,x2_0,x2_1)
#   # max_x=max(x1_0,x1_1,x2_0,x2_1)
#   # min_y=min(y1_0,y1_1,y2_0,y2_1)
#   # max_y=max(y1_0,y1_1,y2_0,y2_1)
#   # new_src_span=(min_x,max_x)
#   # new_trg_span=(min_y,max_y)
#   return output

# def overlaps_OLD(span1,span2):
#   span1_i0,span1_i1=span1
#   span2_i0,span2_i1=span2
#   if span1_i0>=span2_i0 and span1_i0<=span2_i1: return True
#   if span1_i1>=span2_i0 and span1_i1<=span2_i1: return True
#   if span2_i0>=span1_i0 and span2_i0<=span1_i1: return True
#   if span2_i1>=span1_i0 and span2_i1<=span1_i1: return True
#   return False


def walign(src_obj0,trg_obj0,src_inv_dict0,trg_inv_dict0,walign_params0={}):
  min_retrieved_index_count=walign_params0.get("min_retrieved_index_count",3) #3
  max_dist=walign_params0.get("max_dist",10)
  min_ratio=walign_params0.get("min_ratio",0.05)
  max_sent_size=walign_params0.get("max_sent_size")
  n_epochs=walign_params0.get("n_epochs",20)
  output={}

  span_el_dict={} #which elements corresponds to this span - sorted by weight
  start_span_dict={} #show all the spans corresponding to a src location
  el_child_dict={} #show which elements are children of which
  el_wt_dict={} #show the weight of each element

  
  span_wt_dict={} #wight of each span -> the weight of the highest element it includes
  span_child_dict={} #which subspans this span consists of
  span_top_elms_dict={} #show the top elements for each span
  
  
  
  all_phrase_pairs=[]
  src0,trg0=src_obj0.tokens, trg_obj0.tokens
  original_src0,original_trg0=src_obj0.raw_tokens, trg_obj0.raw_tokens
  map_src0,map_trg0=src_obj0.map, trg_obj0.map
  #trg0=trg_obj0.tokens #raw_tokens,map
  #if len(src0)>35: return


  src0=['<s>']+src0+['</s>'] #first we pad the sentence tokens with start and end sentence markers
  trg0=['<s>']+trg0+['</s>'] #for both src and trg
  original_src0=['<s>']+original_src0+['</s>'] #first we pad the sentence tokens with start and end sentence markers
  original_trg0=['<s>']+original_trg0+['</s>'] #for both src and trg

  output["src"]=original_src0
  output["trg"]=original_trg0
  

  
  if max_sent_size!=None and len(src0)>max_sent_size: return output


  # self.src0=src0
  # self.trg0=trg0
  #print(src0)
  src_grouped=get_tok_loc(src0) #group all the locations for a list of consecutive tokens (src then trg)
  trg_grouped=get_tok_loc(trg0)
  src_side,trg_side=[],[] #mapping unigrams and bigrams to both their locations within the sentence and retrieved indexes from inverted dict
  src_locs_count_dict={} #how many times did the current token/bigram/phrase appeared in the src sent tokens
  trg_locs_count_dict={} #and same for trg
  for s_toks0,s_locs_grp0 in src_grouped: #retrive the indexes for each token sequence, map them to the locations
    src_indexes0= rtrv_idx(s_toks0,src_inv_dict0)
    s_toks_str0=" ".join(s_toks0) 
    src_side.append((s_toks_str0,s_locs_grp0,src_indexes0))
    src_locs_count_dict[s_toks_str0]=len(s_locs_grp0)
  for t_toks0,t_locs_grp0 in trg_grouped:
    trg_indexes0= rtrv_idx(t_toks0,trg_inv_dict0)
    t_toks_str0=" ".join(t_toks0)
    trg_side.append((t_toks_str0,t_locs_grp0,trg_indexes0))
    trg_locs_count_dict[t_toks_str0]=len(t_locs_grp0)

  #now starting the match the indexes of both src and trg token locations and get the corresponding rations/weights
  matching_list=[]
  all_elements=[] #to create elements (pairs of src-trg spans)
  tmp_ratio_dict={}
  for src_item0 in src_side: #examining all src tokens/bigrams with their locs/spans and retrieved indexes
    s_toks_str0,s_locs_grp0,src_indexes0=src_item0
    for trg_item0 in trg_side: #and get also all trg tokens/bigrams with locs/spand and retrieved indexes
      t_toks_str0,t_locs_grp0,trg_indexes0=trg_item0
      cur_ratio=0
      if s_toks_str0==t_toks_str0: cur_ratio=1.0 #if the src equals the target (e.g. numbers), the matching ratio is 1
      else: 
        if len(src_indexes0)<min_retrieved_index_count or len(trg_indexes0)<min_retrieved_index_count: continue #if the number of retrieved indexes is less than certain threshold, we keep the ratio as zero
        cur_ratio=idx_match(src_indexes0,trg_indexes0,5)
      if cur_ratio==0: continue
      matching_list.append(([s_toks_str0,s_locs_grp0],[t_toks_str0,t_locs_grp0], cur_ratio))

  #####
  #Now filtering the matching list
  matching_list.sort(key=lambda x:-x[-1])
  valid_list=[] #the final list of acceptable phrase pairs, their locs, and matching ratio
  for ml in matching_list:
    a,b,r=ml
    src_phrase0,src_locs=a #for the current src phrase and how many locations it occurs in the current sentence
    trg_phrase0,trg_locs=b #and same for trg phrase
    min_count=min(len(src_locs), len(trg_locs)) #and we get the minimum occurance between the src and trg phrase
    found_src_count=src_locs_count_dict[src_phrase0] #and then we check whether the src phrase was used before
    found_trg_count=trg_locs_count_dict[trg_phrase0] #and same for trg phrase
    if found_src_count<1 or found_trg_count<1: continue #if either was used before, their count would be zero or less, so we exclude current pair
    src_locs_count_dict[src_phrase0]=found_src_count-min_count #if not, we proceed, abd subtract the minimum count from the "unused" count of the current src phrase
    trg_locs_count_dict[trg_phrase0]=found_trg_count-min_count #and same for trg
    valid_list.append(ml)
  #Now we star processing the valid pairs, with their actual geometric spans and weights
  #first populate the main dictionaries: span_el_dict,  start_span_dict, el_child_dict, el_wt_dict
  span_el_list=[] #list of spans and elements
  all_src_spans=[]
  for vl in valid_list:
    #print(">>>", vl)
    a,b,cur_ratio=vl
    src_phrase0,src_locs=a #for the current src phrase and how many locations it occurs in the current sentence
    trg_phrase0,trg_locs=b #and same for trg phrase
    for sl0 in src_locs:
      if not sl0 in all_src_spans: all_src_spans.append(sl0)
      for tl0 in trg_locs:
        cur_el=(sl0,tl0) #an element is a tuple containing the span of src indexes and span of trg indexes
        el_wt_dict[cur_el]=cur_ratio #initially populating the element weight dict
        span_el_list.append((sl0,cur_el,cur_ratio))
  span_el_list.sort(key=lambda x:x[0])
  span_el_list_grouped=[(key,[v[1] for v in list(group)]) for key,group in groupby(span_el_list,lambda x:x[0])]
  all_src_spans.sort()
  all_src_spans_grouped=[(key,list(group)) for key,group in groupby(all_src_spans,lambda x:x[0])]
  for sp0,elements0 in span_el_list_grouped: span_el_dict[sp0]=elements0 #initially populating the span-elements dict
  for start_i0,spans0 in all_src_spans_grouped: start_span_dict[start_i0]=spans0 #initially poplating the start/spans dict
  top_score=0
  full_src_span0=(0,len(src0)-1)
  full_trg_span0=(0,len(trg0)-1)
  full_el=(full_src_span0,full_trg_span0)
  #all_top_scores=[]

  for ep0 in range(n_epochs):
    #continue
    cur_start_locs=sorted(list(set(start_span_dict.keys())))
    #print(ep0, "cur_start_locs",cur_start_locs)
    for i_,loc_i in enumerate(cur_start_locs):
      next_locs=cur_start_locs[i_+1:]
      cur_spans=start_span_dict[loc_i]
      next_spans=[]
      for nl in next_locs: next_spans.extend(start_span_dict[nl])
      #print(ep0,"cur_spans",cur_spans,"next_spans",next_spans)

      for cs in cur_spans: 
        cur_els=span_el_dict.get(cs,[])
        #print("$$$$$ cur_els",cur_els)
        for ns in next_spans:
          #if ns[0]<=cs[1]: continue #print("exclude:",cs,ns)
          x_dist=get_span_dist(cs,ns)
          #print("????", cs,ns, "x_dist",x_dist)
          if x_dist<1 or x_dist>max_dist: continue
          #else: print("++++", cs,ns)
          next_els=span_el_dict.get(ns,[])
          #print("#### next_els",next_els)
          for cur_el0 in cur_els:
            cur_wt0=el_wt_dict[cur_el0]
            cur_x_span,cur_y_span=cur_el0
            for next_el0 in next_els:
              if cur_el0==next_el0: continue
              next_x_span,next_y_span=next_el0
              y_dist=get_span_dist(cur_y_span,next_y_span)
              if y_dist<1 or y_dist>max_dist: continue #print("Excluded XXX: cur_el0",cur_el0, "next_el0",next_el0,"y_dist",y_dist)
              next_wt0=el_wt_dict[next_el0] 
              combined_wt0=cur_wt0+next_wt0
              combined_el0=combine_els(cur_el0,next_el0) #get_min_max(cur_el0,next_el0)
              found_combined_wt=el_wt_dict.get(combined_el0,0)
              #print(">>>", cur_el0, next_el0,"combined_wt0",combined_wt0,"found_combined_wt",found_combined_wt)       
              if combined_wt0>found_combined_wt:
                el_wt_dict[combined_el0]=combined_wt0
                el_child_dict[combined_el0]=[cur_el0,next_el0]
                tmp_src_span0=combined_el0[0]
                #print("tmp_src_span0",tmp_src_span0)
                found_span_elements=span_el_dict.get(tmp_src_span0,[])
                if not combined_el0 in found_span_elements: found_span_elements.append(combined_el0)
                found_span_elements.sort(key=lambda x:-el_wt_dict[x])
                span_el_dict[tmp_src_span0]=found_span_elements
                #print("span_el_dict[tmp_src_span0]",span_el_dict[tmp_src_span0])
                found_start_spans=start_span_dict.get(tmp_src_span0[0])
                if not tmp_src_span0 in found_start_spans: found_start_spans.append(tmp_src_span0)
                start_span_dict[tmp_src_span0[0]]=found_start_spans
                #print("start_span_dict[tmp_src_span0[0]]",start_span_dict[tmp_src_span0[0]])
                #print(ep0, "cur_el0",cur_el0, "next_el0",next_el0,"combined_el0",combined_el0, "combined_wt0",round(combined_wt0,4), "y_dist",y_dist)
        

    cur_score=el_wt_dict.get(full_el,0)
    wt_list=list(el_wt_dict.items())
    wt_list.sort(key=lambda x:-x[1])
    #print(ep0, wt_list[0], "cur_score",cur_score, "top",top_score)
    # for wl in wt_list[:5]:
    #   print(ep0, wl)
    # print("----")
    # print(all_top_scores)
    # all_top_scores.append(cur_score)

    if cur_score>top_score: top_score=cur_score
    elif cur_score==top_score and cur_score>0: break

  if el_wt_dict.get(full_el,0)>0:
    align_list=get_rec_el_children(full_el,el_child_dict,el_list0=[])
  else:
    top_el=wt_list[0][0]
    print("couldn't find full el:",full_el, "used the top el instead", top_el)
    align_list=get_rec_el_children(top_el,el_child_dict,el_list0=[])

  
  if align_list!=[]:
    # src0=src_obj0.tokens
    # trg0=trg_obj0.tokens
    output_align_list=[]

    # print("aligned")
    # print(src0)
    # print(trg0)
    # print(map_src0)
    # print(map_trg0)
    adj_src_map=[0]+[v+1 for v in map_src0]+[len(original_src0)-1]
    adj_trg_map=[0]+[v+1 for v in map_trg0]+[len(original_trg0)-1]
    # print(original_src0)
    # print(original_trg0)
    # print("adj_src_map",adj_src_map)
    # print("adj_trg_map",adj_trg_map)
    for a in align_list:
      src_span0,trg_span0=a
      x0,x1=src_span0
      y0,y1=trg_span0
      new_x0,new_x1=adj_src_map[x0],adj_src_map[x1]
      new_y0,new_y1=adj_trg_map[y0],adj_trg_map[y1]
      new_src_pan=new_x0,new_x1
      new_trg_span=new_y0,new_y1
      
      src_phrase0=src0[src_span0[0]:src_span0[1]+1]
      trg_phrase0=trg0[trg_span0[0]:trg_span0[1]+1]

      output_align_list.append((new_src_pan,new_trg_span))

      src_phrase1=original_src0[new_x0:new_x1+1]
      trg_phrase1=original_trg0[new_y0:new_y1+1]
    #   print(a,src_phrase0, trg_phrase0)
    #   print("src_phrase1",src_phrase1)
    #   print("trg_phrase1",trg_phrase1)
    #   print("---")
    # print("==============")
  #out["align"]=output_align_list
  output["align"]=output_align_list
  return output


##Starting Indexing/Alignment

In [ ]:
#Starting the experiment
exp_dir="exp00"
batch_size=100000
#batch_size=5000


ar_corpus_fname="UNv1.0.ar-en-clean2.ar"
en_corpus_fname="UNv1.0.ar-en.en"
cur_walign_dir="un-walign"

full_align_dir_path=os.path.join(cur_walign_dir,exp_dir)
if not os.path.exists(full_align_dir_path): os.makedirs(full_align_dir_path)


params={}
params["lang"]="en"
params["ignore_punc"]=True
params["stop_words"]=[]
params["ignore_ar_pre_suf"]=True
params["remove_al"]=True
params["index_words"]=True
params["lower"]=True
params["stemming"]=False

walign_params={}
walign_params["min_retrieved_index_count"]=3
walign_params["max_dist"]=5
walign_params["max_sent_size"]=60
walign_params["min_ratio"]=0.05
walign_params["n_epochs"]=5

walign_params_fpath=os.path.join(full_align_dir_path,"walign-params.json")
indexing_params_fpath=os.path.join(full_align_dir_path,"indexing-params.json")
with open(walign_params_fpath,"w") as walign_fopen:
  json.dump(walign_params,walign_fopen)


#print("file size:", file_len(en_corpus_fname))

src_params,trg_params=dict(params),dict(params) #we copy the default parameters
src_params["stop_words"]=["the","of","an","a","and","is","are", "to","been","it","as","for","have"]
trg_params["stop_words"]=["إلى"]

indexing_params={}
indexing_params["src"]=src_params
indexing_params["trg"]=trg_params
with open(indexing_params_fpath,"w") as indexing_fopen:
  json.dump(indexing_params,indexing_fopen)



ar_fopen=open(ar_corpus_fname)
en_fopen=open(en_corpus_fname)
src_sents,trg_sents=[],[]
for pair_i,pair in enumerate(zip(en_fopen,ar_fopen)):
  if pair_i>batch_size: break
  #print(pair_i)
  if pair_i>0 and pair_i%batch_size==0: #if the current line number of the bitext matches the btach size, we start processing batch
    #walign_sentences(src_sents,trg_sents,cur_align_fname)
    cur_align_fname=os.path.join(full_align_dir_path,"%s.txt"%pair_i) #the output goes into the filename named after the current line number
    cur_size=file_len(cur_align_fname) #we check how many lines in the current file
    if cur_size==len(src_sents): continue #if the files has an equal number of lines to the batch size, we proceed to next batch
    #uncheck the above line after you finish experiments
    print("indexing src sentences")
    
    src_index_obj=indexing(src_sents, tok,src_params)
    print("indexing trg sentences")
    trg_index_obj=indexing(trg_sents, cur_ar_tok_fn,trg_params)
    print("finished indexing, now aligning words")
    print("number of sentences:",len(src_sents))
    # start_i=1000
    # n_sents=50
    # sent_counter=0
    for sent_i in range(len(src_sents)): #range(start_i,start_i+n_sents):
      if sent_i%1000==0: print("now aligning", sent_i)
      #sent_counter+=1
      #if sent_counter>35: break
      
      if sent_i<cur_size: 
        #print("skipping now")
        continue #also uncheck this after finishing experiments
      print("sent_i",sent_i)
      #cur_obj={}
      #print(src_sents[sent_i])
      #print(trg_sents[sent_i])
      # cur_src0=src_sents[i_] 
      # cur_trg0=trg_sents[i_]
      cur_src_obj0=src_index_obj.sent_obj_list[sent_i]
      cur_trg_obj0=trg_index_obj.sent_obj_list[sent_i]
      output0=walign(cur_src_obj0,cur_trg_obj0,src_index_obj.inv_index,trg_index_obj.inv_index,walign_params0=walign_params)
      #walign(cur_src_obj0,cur_trg_obj0,src_index_obj.inv_index,trg_index_obj.inv_index)
      #cur_obj["src"]=src_sents[sent_i]
      json_str=json.dumps(output0)
      cur_align_fopen=open(cur_align_fname,"a")
      cur_align_fopen.write(json_str+"\n")
      cur_align_fopen.close()


    
    print(pair_i, "number of unique src words", len(src_index_obj.inv_index))
    print(pair_i, "number of unique trg words", len(trg_index_obj.inv_index))
    print("resetting sentence lists")
    print("----------")
    #print("processing sentences")
    src_sents,trg_sents=[],[]
    #break
  if pair_i%10000==0: print(pair_i)
  src_line,trg_line=pair
  src_line,trg_line=src_line.strip("\n\r\t "),trg_line.strip("\n\r\t ")
  src_sents.append(src_line)
  trg_sents.append(trg_line)

ar_fopen.close()
en_fopen.close()

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
indexing src sentences
indexing trg sentences
finished indexing, now aligning words
number of sentences: 100000
now aligning 0
now aligning 1000
now aligning 2000
now aligning 3000
now aligning 4000
now aligning 5000
now aligning 6000
now aligning 7000
now aligning 8000
now aligning 9000
now aligning 10000
now aligning 11000
now aligning 12000
now aligning 13000
now aligning 14000
now aligning 15000
now aligning 16000
now aligning 17000
now aligning 18000
now aligning 19000
now aligning 20000
now aligning 21000
now aligning 22000
now aligning 23000
now aligning 24000
now aligning 25000
now aligning 26000
now aligning 27000
now aligning 28000
now aligning 29000
now aligning 30000
now aligning 31000
sent_i 31520


KeyboardInterrupt: ignored

##Testing Alignment Output

In [ ]:
start_i=3900
n_sents=20
sent_counter=0
for sent_i in range(start_i,start_i+n_sents):
  #if sent_i%1000==0: print("now aligning", sent_i)
  sent_counter+=1
  #if sent_counter>35: break
  print("sent_i",sent_i)
  #if sent_i<cur_size: continue #also uncheck this after finishing experiments
  cur_obj={}
  print(src_sents[sent_i])
  print(trg_sents[sent_i])
  # cur_src0=src_sents[i_] 
  # cur_trg0=trg_sents[i_]
  cur_src_obj0=src_index_obj.sent_obj_list[sent_i]
  #if len(cur_src_obj0.tokens)>20: continue
  cur_trg_obj0=trg_index_obj.sent_obj_list[sent_i]
  output0=walign(cur_src_obj0,cur_trg_obj0,src_index_obj.inv_index,trg_index_obj.inv_index,walign_params0=walign_params)
  cur_aligned_spans=output0.get("align")
  if cur_aligned_spans!=None:
    cur_src_tokens=output0["src"]
    cur_trg_tokens=output0["trg"]
    for src_span0,trg_span0 in cur_aligned_spans:
      print("src_span0,trg_span0",src_span0,trg_span0)
      print(cur_src_tokens[src_span0[0]:src_span0[1]+1])
      print(cur_trg_tokens[trg_span0[0]:trg_span0[1]+1])
    # for a,b in output0.items():
    #   print(a,b)
    print("-------")
  #print(output0)
  # cur_obj["src"]=src_sents[sent_i]
  # json_str=json.dumps(cur_obj)
  # cur_align_fopen=open(cur_align_fname,"a")
  # cur_align_fopen.write(json_str+"\n")
  # cur_align_fopen.close()

sent_i 3900
For the period ending 30 April 1993, voluntary contributions received by UNIKOM for which no budgetary provision was made in the cost estimates, and as valued by the donor Government, are shown below. 93-40057 (E) 160793 /... Estimated value (United States
وفيما يتعلق بالفترة المنتهية في 30 نيسان/أبريل 1993، يرد أدناه بيان التبرعات التي تلقتها بعثة المراقبة والتي لم يرصد بشأنها اعتماد في الميزانية في تقدير التكاليف، ووفقا للقيمة التي حددتها الحكومة المتبرعة.
0 (((0, 23), (0, 32)), 7.862982656080205) cur_score 0 top 0
1 (((0, 26), (0, 32)), 8.597533636525494) cur_score 7.658684337986297 top 0
2 (((0, 35), (0, 34)), 9.134889846843263) cur_score 9.134889846843263 top 7.658684337986297
3 (((0, 35), (0, 34)), 9.501195559987828) cur_score 9.501195559987828 top 9.134889846843263
4 (((0, 35), (0, 34)), 9.521654623591965) cur_score 9.521654623591965 top 9.501195559987828
src_span0,trg_span0 (0, 0) (0, 0)
['<s>']
['<s>']
src_span0,trg_span0 (3, 3) (5, 5)
['period']
['ال_فترة']
src_sp

##Processing aligned materials

In [ ]:
import os,json

def extract_phrases(src_tokens0,trg_tokens0,align_list0,max_size=10): #extracting phrases from aligned pairs of src/trg tokens, filling the gaps of unaligned tokens 
  src_size,trg_size=len(src_tokens0),len(trg_tokens0)
  #expanded_spans=[]
  expanded_spans=list(align_list0)
  used_src_locs,used_trg_locs=[],[]
  for i0, align_span_pair0 in enumerate(align_list0):
    src_span0,trg_span0=align_span_pair0
    used_src_locs.extend(range(src_span0[0],src_span0[1]+1))
    used_trg_locs.extend(range(trg_span0[0],trg_span0[1]+1))
  # print("used_src_locs",used_src_locs)
  # print("used_trg_locs",used_trg_locs)

  for i0, align_span_pair0 in enumerate(align_list0):
    src_span0,trg_span0=align_span_pair0
    for align_span_pair1 in align_list0[i0+1:]:
      src_span1,trg_span1=align_span_pair1
      #print(align_span_pair0, align_span_pair1)
      combined_src_span=(src_span0[0],src_span1[1])
      combined_trg_span=min(trg_span0[0],trg_span1[1]),max(trg_span0[0],trg_span1[1])
      valid=True
      for align_span_pair2 in align_list0:
        if align_span_pair2==align_span_pair0 or align_span_pair2==align_span_pair1: continue
        src_span2,trg_span2=align_span_pair2
        if (src_span2[0]>=combined_src_span[0] and src_span2[0]<=combined_src_span[1]) or (src_span2[1]>=combined_src_span[0] and src_span2[1]<=combined_src_span[1]):
          if trg_span2[1]>combined_trg_span[1] or trg_span2[0]<combined_trg_span[0]:
            valid=False
            #print("outside:",align_span_pair2)
            break
      if not valid: continue
      expanded_spans.append((combined_src_span,combined_trg_span))

      # print("combined_src_span",combined_src_span)
      # print("combined_trg_span",combined_trg_span)
  final_spans=[]
  for combined_src_span, combined_trg_span in expanded_spans:
    x0,x1=combined_src_span
    y0,y1=combined_trg_span
    if x1-x0>max_size: continue
    src_starting_locs=[x0]
    src_ending_locs=[x1]
    trg_starting_locs=[y0]
    trg_ending_locs=[y1]
    go_x_pre,go_x_post,go_y_pre,go_y_post=True,True,True, True
    for offset0 in range(1,10):
      pre_x0=x0-offset0
      if pre_x0>=0 and not pre_x0 in used_src_locs: src_starting_locs.append(pre_x0)
      else: break
    for offset0 in range(1,10):
      post_x1=x1+offset0
      if post_x1<src_size and not post_x1 in used_src_locs: src_ending_locs.append(post_x1) 
      else: break
    for offset0 in range(1,10):
      pre_y0=y0-offset0
      if pre_y0>=0 and not pre_y0 in used_trg_locs: trg_starting_locs.append(pre_y0) 
      else: break
    for offset0 in range(1,10):
      post_y1=y1+offset0
      if post_y1<trg_size and not post_y1 in used_trg_locs: trg_starting_locs.append(post_y1)
      else: break
    # print("src_starting_locs",src_starting_locs)
    # print("src_ending_locs",src_ending_locs)
    # print("trg_starting_locs",trg_starting_locs)
    # print("trg_ending_locs",trg_ending_locs)



    # print("valid",valid)
    # print("---")
    for new_x0 in src_starting_locs:
      for new_x1 in src_ending_locs:
        if new_x1-new_x0>max_size: continue
        new_src_span=(new_x0,new_x1)
        for new_y0 in trg_starting_locs:
          for new_y1 in trg_ending_locs:
            if new_y1<new_y0: continue
            new_trg_span=(new_y0,new_y1)
            final_spans.append((new_src_span,new_trg_span))
  all_phrases=[]
  for src_span,trg_span in final_spans:
    #print(src_span, trg_span)
    cur_src_phrase=" ".join(cur_src[src_span[0]:src_span[1]+1])
    cur_trg_phrase=" ".join(cur_trg[trg_span[0]:trg_span[1]+1])
    all_phrases.append((cur_src_phrase,cur_trg_phrase))

    # print("expanded cur_src",cur_src[src_span[0]:src_span[1]+1])
    # print("expanded cur_trg",cur_trg[trg_span[0]:trg_span[1]+1])

  return all_phrases
      



exp_dir="exp01-test"
cur_walign_dir="un-walign"
full_align_dir_path=os.path.join(cur_walign_dir,exp_dir)
files=os.listdir(full_align_dir_path)
files=[os.path.join(full_align_dir_path,v) for v in files if v.endswith(".txt")]

matching_dict={}
src_counter_dict,trg_counter_dict={},{}
#print(files)
cur_i=0
cur_n=200000

min_src_count=20
min_trg_count=5

cur_fpath=files[0]
for cur_fpath in files:
  print(cur_fpath)
  fopen=open(cur_fpath)
  for line_i,line0 in enumerate(fopen):
    if line_i%2000==0: print("processing:",line_i)
    if line_i<cur_i: continue
    if line_i>cur_i+cur_n: break
    cur_obj=json.loads(line0.strip())
    cur_src,cur_trg,cur_align=cur_obj["src"],cur_obj["trg"],cur_obj.get("align",[])
    if cur_align==[]: continue
    all_phrases0=extract_phrases(cur_src,cur_trg,cur_align)
    for phrase_pair0 in all_phrases0:
      #print(phrase_pair0)
      src0,trg0=phrase_pair0
      src0=src0.lower()
      trg0=trg0.lower()
      src_counter_dict[src0]=src_counter_dict.get(src0,0)+1
      trg_counter_dict[trg0]=trg_counter_dict.get(trg0,0)+1
      tmp_dict=matching_dict.get(src0,{})
      tmp_dict[trg0]=tmp_dict.get(trg0,0)+1
      matching_dict[src0]=tmp_dict
    #print("=======")
  fopen.close()

new_ratio_dict={}
final_phrase_pair_list=[]
for src_phrase,b_dict in matching_dict.items():
  src_instances_n=sum(b_dict.values())
  if max(b_dict.values())<min_trg_count: continue
  if src_instances_n<min_src_count: continue
  #print(src_phrase)
  tmp_trg_ratio_dict={}
  for trg_phrase,b0 in b_dict.items():
    if b0<min_trg_count: continue
    overall_trg_phrase_count=trg_counter_dict[trg_phrase]
    ratio=b0/(src_instances_n+overall_trg_phrase_count-b0)
    final_phrase_pair_list.append((src_phrase,trg_phrase,b0,ratio))
    tmp_trg_ratio_dict[trg_phrase]=(ratio,b0)
  new_ratio_dict[src_phrase]=tmp_trg_ratio_dict
  #   print(src_phrase,trg_phrase, round(ratio,4))
  # print("-----")

  #print(a,src_instances_n)
final_phrase_pair_list.sort(key=lambda x:-x[-1])
# for a in final_phrase_pair_list[:50]:
#   print(a)
print("final_phrase_pair_list", len(final_phrase_pair_list))

un-walign/exp01-test/100000.txt
processing: 0
processing: 2000
processing: 4000
processing: 6000
processing: 8000
processing: 10000
processing: 12000
processing: 14000
processing: 16000
processing: 18000
processing: 20000
processing: 22000
processing: 24000
processing: 26000
processing: 28000
processing: 30000
processing: 32000
processing: 34000
processing: 36000
processing: 38000
processing: 40000
processing: 42000
processing: 44000
processing: 46000
processing: 48000
processing: 50000
processing: 52000
processing: 54000
processing: 56000
processing: 58000
processing: 60000
processing: 62000
processing: 64000
processing: 66000
processing: 68000
processing: 70000
processing: 72000
processing: 74000
processing: 76000
processing: 78000
processing: 80000
processing: 82000
processing: 84000
processing: 86000
processing: 88000
processing: 90000
processing: 92000
processing: 94000
processing: 96000
processing: 98000
un-walign/exp01-test/200000.txt
processing: 0
processing: 2000
processing: 4

In [ ]:
src_phrase="other countries"
src_phrase=src_phrase.strip()
overall_src_count=src_counter_dict[src_phrase]
tmp_dict=matching_dict[src_phrase]

final_list=[]
for cur_trg_phrase,trg_phrase_val in tmp_dict.items():
  if trg_phrase_val<2: continue
  overall_trg_count=trg_counter_dict[cur_trg_phrase]
  cur_ratio=trg_phrase_val/(overall_src_count+overall_trg_count-trg_phrase_val)
  final_list.append((src_phrase,cur_trg_phrase,trg_phrase_val, round(cur_ratio,4)))
  #print(cur_trg_phrase,trg_phrase_val, round(cur_ratio,4))
final_list.sort(key=lambda x:-x[-1])
for a in final_list[:10]:
  print(a)

('other countries', 'بلدان أخرى', 67, 0.1569)
('other countries', 'ال_بلدان ال_أخرى', 49, 0.1222)
('other countries', 'وـ ال_بلدان ال_أخرى', 13, 0.0458)
('other countries', 'إلى بلدان أخرى', 8, 0.0304)
('other countries', 'وـ بلدان أخرى', 4, 0.016)
('other countries', 'لـ بلدان أخرى', 4, 0.0154)
('other countries', 'بـ ال_بلدان ال_أخرى', 3, 0.0126)
('other countries', 'لـ ال_بلدان ال_أخرى', 3, 0.0122)
('other countries', 'وـ غير ـها من ال_بلدان', 3, 0.0119)
('other countries', 'غير ـها من ال_بلدان', 3, 0.0117)


In [ ]:
span1=(5,15)
span2=(18,19)
y_dist=get_span_dist(span1,span2)
print(y_dist)

3


In [ ]:
ar_corpus_fname="UNv1.0.ar-en-clean2.ar"
en_corpus_fname="UNv1.0.ar-en.en"

n_sents_max=200000
ar_fopen=open(ar_corpus_fname)
en_fopen=open(en_corpus_fname)

src_sents,trg_sents=[],[]
for i,pair in enumerate(zip(en_fopen,ar_fopen)):
  if i>=n_sents_max: break
  if i%20000==0: print(i)
  en_line,ar_line=pair
  en_line,ar_line=en_line.strip("\n\r\t "),ar_line.strip("\n\r\t ")
  src_sents.append(en_line)
  trg_sents.append(ar_line)

ar_fopen.close()
en_fopen.close()
print(len(src_sents),len(src_sents))
print("Now adding another bitext in a Google sheet")
import pandas as pd
#Job Bitexts
#xls_path='https://docs.google.com/spreadsheets/d/e/2PACX-1vSqipSUAaMWJrizvZtg1WIpds2pIiXtcZC6wjOLeVYie9p5O0lqiPnVdd8ambGKH8B0qJ4kmlVeEs6f/pub?output=xlsx'
#Ref bitexts
xls_path='https://docs.google.com/spreadsheets/d/e/2PACX-1vSVy7hY4fV0LHqTCypOjZAM9OrBevDDnpIcEKEAPKDsYRoxAz9TUBcDIgy8PvHkw5IDAZ6eRmKdZuRw/pub?output=xlsx'
cur_xls = pd.ExcelFile(xls_path)
for cur_sheet_name in cur_xls.sheet_names:
  print(cur_sheet_name)
  #cur_sheet_name="2"
  cur_sheet_obj=pd.read_excel(cur_xls, cur_sheet_name,keep_default_na=False)
  cur_sheet_obj['en']=cur_sheet_obj['en'].apply(str)
  cur_sheet_obj['ar']=cur_sheet_obj['ar'].apply(str)
  # print(cur_sheet_obj)
  # continue

  sent_pair_list=[]
  for i,row in enumerate(cur_sheet_obj.iterrows()):
    row_dict=row[1].to_dict()
    en0,ar0=row_dict["en"],row_dict["ar"]
    ar0=clean_ar(ar0)
    sent_pair_list.append((en0,ar0))
    src_sents.append(en0)
    trg_sents.append(ar0)

#Now indexing
print("Now indexing all sentences")
src_indexing_obj=indexing(src_sents,tok,stop_words=["the","of","and","its","his","her","it","is","are", "their"],ignore_punc=True)
print("finihsed indexing source sentences")
trg_indexing_obj=indexing(trg_sents,cur_ar_tok_fn,ignore_ar_pre_suf=True,ignore_punc=True)
print("finihsed indexing target sentences")

0
20000
40000
60000
80000
100000
120000
140000
160000
180000
200000 200000
Now adding another bitext in a Google sheet
A75760
A76562
A76564
A76572
A76529
A76547
A74594
A76549
A76552
A74597
A75785
Now indexing all sentences
0
5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
55000
60000
65000
70000
75000
80000
85000
90000
95000
100000
105000
110000
115000
120000
125000
130000
135000
140000
145000
150000
155000
160000
165000
170000
175000
180000
185000
190000
195000
200000
205000
210000
215000
220000
225000
230000
finihsed indexing source sentences
0
5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
55000
60000
65000
70000
75000
80000
85000
90000
95000
100000
105000
110000
115000
120000
125000
130000
135000
140000
145000
150000
155000
160000
165000
170000
175000
180000
185000
190000
195000
200000
205000
210000
215000
220000
225000
230000
finihsed indexing target sentences


#Starting Alignmenet

In [ ]:
import math

def overlaps(span1,span2):
  span1_i0,span1_i1=span1
  span2_i0,span2_i1=span2
  if span1_i0>=span2_i0 and span1_i0<=span2_i1: return True
  if span1_i1>=span2_i0 and span1_i1<=span2_i1: return True
  if span2_i0>=span1_i0 and span2_i0<=span1_i1: return True
  if span2_i1>=span1_i0 and span2_i1<=span1_i1: return True
  return False

def get_inner_dist(span1,span2):
  combined_xs=[]
  combined_xs.extend(span1)
  combined_xs.extend(span2)  
  combined_xs.sort()
  return combined_xs[2]-combined_xs[1]

def get_rec_el_children(el0,el_child_dict0,el_list0=[]):
  cur_children0=el_child_dict0.get(el0,[])
  for ch0 in cur_children0:
    el_list0.append(ch0)
    el_list0=get_rec_el_children(ch0,el_child_dict0,el_list0)
  return el_list0

def get_min_max(range1,range2):
  range1_x0,range1_x1=range1
  range2_x0,range2_x1=range2
  min_x=min(range1_x0,range1_x1,range2_x0,range2_x1)
  max_x=max(range1_x0,range1_x1,range2_x0,range2_x1)
  return (min_x,max_x)

def get_area_factor(x_range1,y_range1):
  x_dist=x_range1[1]-x_range1[0]+1
  y_dist=y_range1[1]-y_range1[0]+1
  cur_factor=(x_dist+y_dist)/2
  return cur_factor #math.log(cur_factor)


def get_tok_loc(tok_list):
  tok_locs_list=[]
  for i_,cur_tok in enumerate(tok_list):
    tok_locs_list.append(((cur_tok,),(i_,i_))) #to get unigrams and their locations
    if i_<len(tok_list)-1: tok_locs_list.append(((cur_tok,tok_list[i_+1]), (i_,i_+1))) #to get bigrams and their locations
  tok_locs_list.sort(key=lambda x:x[0])
  grouped=[(key,[v[1] for v in list(group)]) for key,group in groupby(tok_locs_list,lambda x:x[0])]
  return grouped


class walign:
  def __init__(self,src0,trg0,src_inv_dict0,trg_inv_dict0,max_n_elms=10,min_ratio=0.05):
    span_el_dict={} #which element corresponds to this span
    span_wt_dict={} #wight of each span
    span_child_dict={} #which subspans this span consists of
    
    span_top_elms_dict={} #show the top elements for each span
    el_wt_dict={} #show the weight of each element
    start_span_dict={} #show all the spans corresponding to a src location
    el_child_dict={} #show which elements are children of which
    all_phrase_pairs=[]

    src0=['<s>']+src0+['</s>']
    trg0=['<s>']+trg0+['</s>']
    self.src0=src0
    self.trg0=trg0
    src_grouped=get_tok_loc(src0) #group all the locations for a list of consecutive tokens (src then trg)
    trg_grouped=get_tok_loc(trg0)
    src_side,trg_side=[],[]

    for s_toks,s_locs_grp in src_grouped: #retrive the indexes for each token sequence, map them to the locations
      src_indexes0= rtrv_idx(s_toks,src_inv_dict0)
      src_side.append((s_toks,s_locs_grp,src_indexes0))
    for t_toks,t_locs_grp in trg_grouped:
      trg_indexes0= rtrv_idx(t_toks,trg_inv_dict0)
      trg_side.append((t_toks,t_locs_grp,trg_indexes0))

    #now starting the match the indexes of both src and trg token locations and get the corresponding rations/weights
    matching_list=[]
    all_elements=[] #to create elements (pairs of src-trg spans)
    tmp_ratio_dict={}
    for s_toks,s_locs_grp,src_indexes0 in src_side:
      s_toks_str=" ".join(s_toks)
      for t_toks,t_locs_grp,trg_indexes0 in trg_side:
        cur_ratio=0
        t_toks_str=" ".join(t_toks)
        if s_toks==t_toks: cur_ratio=1
        else: 
          if len(src_indexes0)==0 or len(trg_indexes0)==0: continue #cur_ratio= 0
          cur_ratio=idx_match(src_indexes0,trg_indexes0,5)
        if cur_ratio==0: continue
        for sl0 in s_locs_grp:
          for tl0 in t_locs_grp:
            cur_el=(sl0,tl0) #an element is a tuple containing the span of src indexes and span of trg indexes
            tmp_ratio_dict[cur_el]=cur_ratio
            #all_elements.append((cur_el,s_toks_str,t_toks_str,cur_ratio))
            all_elements.append((cur_el,cur_ratio))
            found_wt=span_wt_dict.get(sl0,0)
            if found_wt<cur_ratio:
              span_wt_dict[sl0]=cur_ratio
              span_el_dict[sl0]=cur_el
              span_child_dict[sl0]=[]
    #####
    #and now let's group all the elements by the starting location of the src tokenized sentence
    all_elements.sort(key=lambda x:(x[0][0],-x[-1]))
    all_elements_grouped=[(key,list(group)) for key,group in groupby(all_elements,lambda x:x[0][0])]

    #initially populating the start/span/element/weight dictionaries
    for k0,grp0 in all_elements_grouped:
      for el0,el_wt0 in grp0[:max_n_elms]:
        if el_wt0<min_ratio: continue
        el_wt_dict[el0]=el_wt0

    all_elements.sort(key=lambda x:-x[-1])
    used_src,used_trg=[],[]
    new_el_wt_list=[]
    for ae in all_elements: #get the elements below the top by iteratively discarding the top used elements
      el0,wt0=ae
      src_span,trg_span=el0    
      if any([v in used_src for v in src_span]): continue
      if any([v in used_trg for v in trg_span]): continue
      used_src.extend(src_span)
      used_trg.extend(trg_span)
      new_el_wt_list.append(ae)
      el_wt_dict[el0]=wt0#+0.2
    
    #all_elements.sort(key=lambda x:(x[0][0][0],-x[-1]))

    el_wt_list=list(el_wt_dict.items())
    el_wt_list.sort(key=lambda x:(x[0][0],-x[-1]))
    for el0,wt0 in el_wt_list:
      #print(el0,wt0)
      src_span0,trg_span0=el0
      start_i=src_span0[0]
      found_spans=start_span_dict.get(start_i,[])
      if not src_span0 in found_spans: start_span_dict[start_i]=found_spans+[src_span0]
      found_elements=span_top_elms_dict.get(src_span0,[])
      if not el0 in found_elements: span_top_elms_dict[src_span0]=found_elements+[el0]


    used_el_dict={}
    full_src_span=(0,len(src0)-1)
    full_trg_span=(0,len(trg0)-1)
    src_trg_span=(full_src_span,full_trg_span)
    #cur_full_span_wt=span_wt_dict.get(full_src_span,0)
    cur_el_full_span_wt=el_wt_dict.get(src_trg_span,0) #check if there is an element that covers the full src/trg span
    

    for inc0 in range(15): #increments to cover slices of src location indexes
      
      test_el_wt=sorted(list(el_wt_dict.items()),key=lambda x:-x[-1])
      tmp_el_full_span_wt=el_wt_dict.get(src_trg_span,0)
      print(inc0,test_el_wt[0], tmp_el_full_span_wt) #Checking the situation with the convergence of the full src/trg span element
      if tmp_el_full_span_wt>0:
        if tmp_el_full_span_wt>cur_el_full_span_wt: cur_el_full_span_wt=tmp_el_full_span_wt
        else: 
          print("alignment solved")
          break


      for i0 in range(len(src0)):
        first_spans=start_span_dict.get(i0,[])
        first_elements=[]
        for sp0 in first_spans:
          first_elements.extend(span_top_elms_dict.get(sp0,[]))
        valid_next_items_counter=0
        for el0 in first_elements:
          el0_wt=el_wt_dict.get(el0,0)
          el0_src_span,el0_trg_span=el0
          el0_src_span_i0,el0_src_span_i1=el0_src_span
          el0_trg_span_i0,el0_trg_span_i1=el0_trg_span

          second_spans=[]
          second_elements=[]
          for i1 in range(el0_src_span_i1,el0_src_span_i1+5):
            second_spans.extend(start_span_dict.get(i1,[]))
          for sp1 in second_spans: second_elements.extend(span_top_elms_dict.get(sp1,[]))
          for el1 in second_elements:
            if el1==el0: continue #we should clean up all the skipping conditions to make sure everything is organized and accounted for, and we're not skipping something that should be kept
            el1_wt=el_wt_dict.get(el1,0)
            el1_src_span,el1_trg_span=el1
            el1_src_span_i0,el1_src_span_i1=el1_src_span
            el1_trg_span_i0,el1_trg_span_i1=el1_trg_span
            trg_spans_overlap=overlaps(el0_trg_span,el1_trg_span)
            src_spans_overlap=overlaps(el0_src_span,el1_src_span)
            
            if el0_src_span==el1_src_span and el0_src_span_i0!=el0_src_span_i1: continue #if [(1,2),(4,5)] [(1,2),(7,6)]
            if trg_spans_overlap: continue #skip if target spans overlap
            if src_spans_overlap and el0_src_span!=el1_src_span: continue

            trg_span_inner_dist=get_inner_dist(el0_trg_span,el1_trg_span)
            if trg_span_inner_dist>10: continue #skip if there is much distance between target spans

            src_span_inner_dist=get_inner_dist(el0_src_span,el1_src_span)
            
            delta_x=el0_src_span_i1-el0_src_span_i0
            excluded=False
            trg_outer_dist=max(el1_trg_span) - min(el0_trg_span)
            if el0_src_span==el1_src_span:
              if el1_trg_span_i1<el0_trg_span_i0: excluded=True
              elif trg_outer_dist>6: excluded=True
              elif trg_span_inner_dist>1: excluded=True
            
            combined_src_span=get_min_max(el0_src_span,el1_src_span)
            combined_trg_span=get_min_max(el0_trg_span,el1_trg_span) 
            if combined_src_span[1]-combined_src_span[0]<2:
              if combined_trg_span[1]-combined_trg_span[0]>8: excluded=True
            if combined_src_span[1]-combined_src_span[0]<3:
              if combined_trg_span[1]-combined_trg_span[0]>10: excluded=True
            

            el_pair=(el0,el1)
            if used_el_dict.get(el_pair,False)==True: excluded=True
            used_el_dict[el_pair]=True

            if excluded: continue

            
            combined_wt=el0_wt+el1_wt
            

            area_factor=get_area_factor(combined_src_span,combined_trg_span)
            #combined_wt=combined_wt-0.02*area_factor
            combined_wt=combined_wt-trg_span_inner_dist*0.05

            combined_el=(combined_src_span,combined_trg_span)
            found_wt=el_wt_dict.get(combined_el,0)
            if found_wt>0 and (combined_src_span[0]==combined_src_span[1]): continue #very sketechy solution to avoid situations where a bigram is replaced by a less likely combination of two consecutive unigrams
            if found_wt>0 and (combined_trg_span[0]==combined_trg_span[1]): continue
            if combined_wt>found_wt:
              el_wt_dict[combined_el]=combined_wt
              el_child_dict[combined_el]=(el0,el1)
              tmp_elements_in_span=span_top_elms_dict.get(combined_src_span,[])
              if not combined_el in tmp_elements_in_span:
                tmp_elements_in_span.append(combined_el)
                tmp_elements_in_span.sort(key=lambda x:-el_wt_dict.get(x,0))
                span_top_elms_dict[combined_src_span]=tmp_elements_in_span[:max_n_elms]
              tmp_spans_starting_with_i0=start_span_dict.get(i0,[])
              if not combined_src_span in tmp_spans_starting_with_i0:
                start_span_dict[i0]=tmp_spans_starting_with_i0+[combined_src_span]
            #if found_wt>0:
            # if el0_src_span==el1_src_span and False:
            #   print(inc0,"excluded",excluded,trg_outer_dist)
            #   print(el0,el0_wt)
            #   print(el1,el1_wt)
            #   print("combined:",combined_el,combined_wt)
            #   print("found:",found_wt)
            #   print("-----")

            valid_next_items_counter+=1

    all_elements2=get_rec_el_children(src_trg_span,el_child_dict,el_list0=[])
    #all_elements=get_rec_children(list_spans_wts[0][0],span_child_dict,span_el_dict,el_list0=[])
    self.final_el_list=[]
    for ch_el in all_elements2:
      #print(ch_el, el_wt_dict[ch_el])
      self.final_el_list.append((ch_el, el_wt_dict[ch_el]))
      src_span0,trg_span0=ch_el
      cur_src_str=" ".join([src0[i_] for i_ in range(src_span0[0],src_span0[1]+1)])
      cur_trg_str=" ".join([trg0[i_] for i_ in range(trg_span0[0],trg_span0[1]+1)])  
      cur_pair=(cur_src_str,cur_trg_str)
      # print(cur_src_str)   
      # print(cur_trg_str) 
      # print("--") 
      # all_phrase_pairs.append((cur_pair))
      #all_phrase_pairs.append((cur_pair,sent_counter))
    # for a in all_phrase_pairs:
    #   print(a)

src_list=src_indexing_obj.all_tok_sentences
trg_list=trg_indexing_obj.all_tok_sentences
src_list_orig=src_indexing_obj.all_tok_original_sentences
trg_list_orig=trg_indexing_obj.all_tok_original_sentences
src_mappings_list=src_indexing_obj.all_mappings
trg_mappings_list=trg_indexing_obj.all_mappings


src_inv_dict=src_indexing_obj.inv_index
trg_inv_dict=trg_indexing_obj.inv_index

phrase_fpath="phrase_list_test.txt"
fopen=open(phrase_fpath,"w")

test_i=7104
n_sents=1
sent_counter=0
all_phrase_pairs=[]
for test_i in range(0,1000):
  print(test_i)
  src_tokens,trg_tokens=src_list[test_i],trg_list[test_i]
  src_mappings,trg_mappings=src_mappings_list[test_i],trg_mappings_list[test_i]
  src_original_tokens,trg_original_tokens=src_list_orig[test_i],trg_list_orig[test_i]
  # print("src_mappings",src_mappings)
  # print("trg_mappings",trg_mappings)
  src_original_tokens=["<s>"]+src_original_tokens+["</s>"]
  trg_original_tokens=["<s>"]+trg_original_tokens+["</s>"]

  new_src_mappings=[0]+[v+1 for v in src_mappings]+[len(src_original_tokens)-1]
  new_trg_mappings=[0]+[v+1 for v in trg_mappings]+[len(trg_original_tokens)-1]
  # print("new_src_mappings",new_src_mappings)
  # print("new_trg_mappings",new_trg_mappings)
  align_obj=walign(src_tokens,trg_tokens,src_inv_dict,trg_inv_dict,max_n_elms=10)
  # print(src_tokens)
  # print(trg_tokens)

  # print("Original SRC: >>>",src_original_tokens)
  # print("Original TRG: >>>",trg_original_tokens)
  # print("orig src:",len(src_original_tokens), "orig trg:",len(trg_original_tokens))
  # print("filtered src:",len(src_tokens), "filtered trg:",len(trg_tokens))
  # print("aligned src:",len(align_obj.src0), "aligned trg:",len(align_obj.trg0))
  for el0,wt0 in align_obj.final_el_list:
    # print(el0,wt0)
    src_span0,trg_span0=el0
    src_i0,src_i1=src_span0
    trg_i0,trg_i1=trg_span0
    new_src_i0,new_src_i1=new_src_mappings[src_i0],new_src_mappings[src_i1]
    new_trg_i0,new_trg_i1=new_trg_mappings[trg_i0],new_trg_mappings[trg_i1]
    src_phrase_tokens=src_original_tokens[new_src_i0:new_src_i1+1]
    trg_phrase_tokens=trg_original_tokens[new_trg_i0:new_trg_i1+1]
    src_phrase_str=" ".join(src_phrase_tokens)
    trg_phrase_str=" ".join(trg_phrase_tokens)
    # print(src_phrase_str)
    # print(trg_phrase_str)
    line='%s\t%s\t%s\n'%(src_phrase_str,trg_phrase_str,round(wt0,4))
    fopen.write(line)
    # #src_span0,trg_span0=ch_el
    # cur_src_str=" ".join([src_tokens[i_] for i_ in range(src_span0[0],src_span0[1]+1)])
    # cur_trg_str=" ".join([trg_tokens[i_] for i_ in range(trg_span0[0],trg_span0[1]+1)])  
    # print("Filtered - SRC: >>>",cur_src_str)
    # print("Filtered - TRG: >>>",cur_trg_str)

  # print("----")

fopen.close()
# for src_tokens,trg_tokens in zip(src_list[test_i:test_i+n_sents],trg_list[test_i:test_i+n_sents]):
#   print(sent_counter)
#   print(src_tokens)
#   print(trg_tokens)
  

Streaming output truncated to the last 5000 lines.
1 (((12, 14), (10, 14)), 2.021784913973186) 0
2 (((0, 13), (0, 11)), 3.0805376007702883) 0
3 (((0, 14), (0, 14)), 5.522061867548715) 5.522061867548715
4 (((0, 14), (0, 14)), 5.522061867548715) 5.522061867548715
alignment solved
337
0 (((0, 0), (0, 0)), 1) 0
1 (((0, 4), (0, 6)), 1.613978494623656) 0
2 (((0, 7), (0, 6)), 1.6272819883449043) 0
3 (((0, 21), (0, 18)), 2.581283214889811) 0
4 (((0, 22), (0, 18)), 2.816168752231888) 0
5 (((0, 22), (0, 18)), 2.816168752231888) 0
6 (((0, 22), (0, 18)), 2.816168752231888) 0
7 (((0, 22), (0, 18)), 2.816168752231888) 0
8 (((0, 22), (0, 18)), 2.816168752231888) 0
9 (((0, 22), (0, 18)), 2.816168752231888) 0
10 (((0, 22), (0, 18)), 2.816168752231888) 0
11 (((0, 22), (0, 18)), 2.816168752231888) 0
12 (((0, 22), (0, 18)), 2.816168752231888) 0
13 (((0, 22), (0, 18)), 2.816168752231888) 0
14 (((0, 22), (0, 18)), 2.816168752231888) 0
338
0 (((0, 0), (0, 0)), 1) 0
1 (((44, 46), (40, 42)), 2.2500868325172014

#Processing Phrase Table

In [ ]:
phrase_fpath="phrase_list_test.txt"
phrase_fopen=open(phrase_fpath)
phrase_corr_dict={}
for i0,line0 in enumerate(phrase_fopen):
  #continue
  #if i0>5000: break
  line_split=line0.strip().split("\t")
  if len(line_split)<3: continue
  src0,trg0,wt0=line_split[:3]
  wt0=float(wt0)
  if src0==trg0: continue
  src0_lower=src0.lower()
  tmp_dict=phrase_corr_dict.get(src0_lower,{})
  found_total_count,found_total_wt=tmp_dict.get(trg0,(0,0))
  found_total_count,found_total_wt=found_total_count+1,found_total_wt+wt0
  tmp_dict[trg0]=(found_total_count,found_total_wt)
  phrase_corr_dict[src0_lower]=tmp_dict




  #print([src0,trg0,wt0])

phrase_fopen.close()
for a,b in phrase_corr_dict.items():
  for a1,b1 in b.items():
    if b1[0]<5: continue
    print(a)
    print(a1)
    print(b1)
    print("-----")


international law commission
لجنة ال_قانون ال_دولي
(5, 6.6690000000000005)
-----
international law
ال_قانون ال_دولي
(10, 11.372)
-----
international
ال_دولي
(31, 14.904800000000007)
-----
international
دولي
(21, 10.096800000000004)
-----
international
دولية
(34, 13.739399999999993)
-----
international
ال_دولية
(35, 14.143499999999992)
-----
law
ال_قانون
(43, 30.375199999999978)
-----
law
قانون
(5, 3.532)
-----
commission
لجنة
(9, 2.2184999999999997)
-----
commission
ال_لجنة
(20, 4.9300000000000015)
-----
forty
ال_أربعين
(13, 7.695999999999998)
-----
session
ال_دورة
(10, 5.198999999999999)
-----
session
دورتها
(9, 3.1032)
-----
may
أيار / مايو
(10, 3.5210000000000004)
-----
may
قد
(11, 0.8162000000000001)
-----
july
تموز / يوليه
(10, 9.006)
-----
comments
تعليقات
(11, 8.3897)
-----
comments
ال_تعليقات
(7, 5.3389)
-----
governments
ال_حكومات
(22, 17.48119999999999)
-----
on
على
(41, 9.753899999999993)
-----
on
بشأن
(13, 2.1476)
-----
on
في
(18, 4.433400000000001)
-----
on
من
(11, 2.02840

In [ ]:
import math
src_list=src_indexing_obj.all_tok_sentences
trg_list=trg_indexing_obj.all_tok_sentences
src_inv_dict=src_indexing_obj.inv_index
trg_inv_dict=trg_indexing_obj.inv_index

def overlaps(span1,span2):
  span1_i0,span1_i1=span1
  span2_i0,span2_i1=span2
  if span1_i0>=span2_i0 and span1_i0<=span2_i1: return True
  if span1_i1>=span2_i0 and span1_i1<=span2_i1: return True
  if span2_i0>=span1_i0 and span2_i0<=span1_i1: return True
  if span2_i1>=span1_i0 and span2_i1<=span1_i1: return True
  return False

def get_inner_dist(span1,span2):
  combined_xs=[]
  combined_xs.extend(span1)
  combined_xs.extend(span2)  
  combined_xs.sort()
  return combined_xs[2]-combined_xs[1]

# def get_rec_children(span0,span_child_dict0,span_el_dict0,el_list0=[]):
#   cur_children0=span_child_dict0.get(span0,[])
#   for ch0 in cur_children0:
#     el_list0.append(span_el_dict0[ch0])
#     el_list0=get_rec_children(ch0,span_child_dict0,span_el_dict0,el_list0)
#   return el_list0

def get_rec_el_children(el0,el_child_dict0,el_list0=[]):
  cur_children0=el_child_dict0.get(el0,[])
  for ch0 in cur_children0:
    el_list0.append(ch0)
    el_list0=get_rec_el_children(ch0,el_child_dict0,el_list0)
  return el_list0

def get_min_max(range1,range2):
  range1_x0,range1_x1=range1
  range2_x0,range2_x1=range2
  min_x=min(range1_x0,range1_x1,range2_x0,range2_x1)
  max_x=max(range1_x0,range1_x1,range2_x0,range2_x1)
  return (min_x,max_x)

def get_dist_factor(el1,el2): #[(5,6),(9,9)] - [(7,7),(12,13)] #get inner distance between two elements, 1 if adjacent
  el1_xrange,el1_yrange=el1
  el2_xrange,el2_yrange=el2
  combined_xs=[]
  combined_xs.extend(el1_xrange)
  combined_xs.extend(el2_xrange)
  combined_ys=[]
  combined_ys.extend(el1_yrange)
  combined_ys.extend(el2_yrange)
  combined_xs.sort()
  combined_ys.sort()
  inner_xdist=combined_xs[2]-combined_xs[1]
  inner_ydist=combined_ys[2]-combined_ys[1]
  return (inner_xdist+inner_ydist)/2

def get_area_factor(x_range1,y_range1):
  x_dist=x_range1[1]-x_range1[0]+1
  y_dist=y_range1[1]-y_range1[0]+1
  cur_factor=(x_dist+y_dist)/2
  return cur_factor #math.log(cur_factor)


def get_tok_loc(tok_list):
  tok_locs_list=[]
  for i_,cur_tok in enumerate(tok_list):
    tok_locs_list.append(((cur_tok,),(i_,i_)))
    if i_<len(tok_list)-1: tok_locs_list.append(((cur_tok,tok_list[i_+1]), (i_,i_+1)))
  tok_locs_list.sort(key=lambda x:x[0])
  grouped=[(key,[v[1] for v in list(group)]) for key,group in groupby(tok_locs_list,lambda x:x[0])]
  return grouped


test_i=-len(sent_pair_list)
n_sents=len(sent_pair_list)-2

max_n_elms=10 #max number of elements per span

test_i=4001
test_i=100
#test_i=12005
n_sents=1
sent_counter=0
all_phrase_pairs=[]
for src0,trg0 in zip(src_list[test_i:test_i+n_sents],trg_list[test_i:test_i+n_sents]):
  print(sent_counter)
  print(src0)
  print(trg0)
  span_el_dict={} #which element corresponds to this span
  span_wt_dict={} #wight of each span
  span_child_dict={} #which subspans this span consists of
  
  span_top_elms_dict={} #show the top elements for each span
  el_wt_dict={} #show the weight of each element
  start_span_dict={} #show all the spans corresponding to a src location
  el_child_dict={} #show which elements are children of which

  src0=['<s>']+src0+['</s>']
  trg0=['<s>']+trg0+['</s>']
  src_grouped=get_tok_loc(src0) #group all the locations for a list of consecutive tokens (src then trg)
  trg_grouped=get_tok_loc(trg0)
  src_side,trg_side=[],[]

  for s_toks,s_locs_grp in src_grouped: #retrive the indexes for each token sequence, map them to the locations
    src_indexes0= rtrv_idx(s_toks,src_inv_dict)
    src_side.append((s_toks,s_locs_grp,src_indexes0))
  for t_toks,t_locs_grp in trg_grouped:
    trg_indexes0= rtrv_idx(t_toks,trg_inv_dict)
    trg_side.append((t_toks,t_locs_grp,trg_indexes0))

  #now starting the match the indexes of both src and trg token locations
  matching_list=[]
  all_elements=[] #to create elements (pairs of src-trg spans)
  tmp_ratio_dict={}
  for s_toks,s_locs_grp,src_indexes0 in src_side:
    s_toks_str=" ".join(s_toks)
    #continue
    #print(s_toks,s_locs_grp,len(src_indexes0))
    for t_toks,t_locs_grp,trg_indexes0 in trg_side:
      cur_ratio=0
      t_toks_str=" ".join(t_toks)
      if s_toks==t_toks: cur_ratio=1
      else: 
        #cur_ratio=idx_match(src_indexes0,trg_indexes0)
        if len(src_indexes0)==0 or len(trg_indexes0)==0: continue #cur_ratio= 0
        cur_ratio=idx_match(src_indexes0,trg_indexes0,5)
      if cur_ratio==0: continue
      for sl0 in s_locs_grp:
        for tl0 in t_locs_grp:
          cur_el=(sl0,tl0) #an element is a tuple containing the span of src indexes and span of trg indexes
          tmp_ratio_dict[cur_el]=cur_ratio
          #all_elements.append((cur_el,s_toks_str,t_toks_str,cur_ratio))
          all_elements.append((cur_el,cur_ratio))
          found_wt=span_wt_dict.get(sl0,0)
          if found_wt<cur_ratio:
            span_wt_dict[sl0]=cur_ratio
            span_el_dict[sl0]=cur_el
            span_child_dict[sl0]=[]
  


      #matching_list.append((s_toks,t_toks,cur_ratio))
  #and now let's group all the elements by the starting location of the src tokenized sentence
  all_elements.sort(key=lambda x:(x[0][0],-x[-1]))
  all_elements_grouped=[(key,list(group)) for key,group in groupby(all_elements,lambda x:x[0][0])]

  #initially populating the start/span/element/weight dictionaries
  for k0,grp0 in all_elements_grouped:
    start_i=k0[0]
    #start_span_dict[start_i]=start_span_dict.get(start_i,[])+[k0]
    #span_top_elms_dict[k0]=[v[0] for v in grp0[:max_n_elms]]
    for el0,el_wt0 in grp0[:max_n_elms]:
      if el_wt0<0.01: continue
      el_wt_dict[el0]=el_wt0
      #print(g0)
  #   print("---")
  # for a,b in start_span_dict.items():
  #   print(a,b)
  all_elements.sort(key=lambda x:-x[-1])
  used_src,used_trg=[],[]
  new_el_wt_list=[]
  for ae in all_elements: #get the elements below the top by iteratively discarding the top used elements
    el0,wt0=ae
    src_span,trg_span=el0    
    if any([v in used_src for v in src_span]): continue
    if any([v in used_trg for v in trg_span]): continue
    used_src.extend(src_span)
    used_trg.extend(trg_span)
    new_el_wt_list.append(ae)
    el_wt_dict[el0]=wt0#+0.2
    #print(ae)

  
  all_elements.sort(key=lambda x:(x[0][0][0],-x[-1]))
  # all_elements=new_el_wt_list+all_elements
  # for ae0 in all_elements[:30]:
  #   print(">>", ae0)
  el_wt_list=list(el_wt_dict.items())
  el_wt_list.sort(key=lambda x:(x[0][0],-x[-1]))
  for el0,wt0 in el_wt_list:
    #print(el0,wt0)
    src_span0,trg_span0=el0
    start_i=src_span0[0]
    found_spans=start_span_dict.get(start_i,[])
    if not src_span0 in found_spans: start_span_dict[start_i]=found_spans+[src_span0]
    found_elements=span_top_elms_dict.get(src_span0,[])
    if not el0 in found_elements: span_top_elms_dict[src_span0]=found_elements+[el0]

    
  # for a,b in start_span_dict.items():
  #   print("start span >>>",a,b)
  # for a,b in span_top_elms_dict.items():
  #   print("top elements +++",a,b)

  # break
  # all_elements_grouped=[(key,list(group)) for key,group in groupby(all_elements,lambda x:x[0][0][0])]
  # el_start_dict={}
  # for k0,el_grp in all_elements_grouped:
  #   #el_start_dict[k0]=el_grp[:10] 
  #   el_start_dict[k0]=dict(iter(el_grp[:10])) #for each starting src index , we identify the top 10 elements that start with it
  used_el_dict={}
  full_src_span=(0,len(src0)-1)
  full_trg_span=(0,len(trg0)-1)
  src_trg_span=(full_src_span,full_trg_span)
  cur_full_span_wt=span_wt_dict.get(full_src_span,0)
  cur_el_full_span_wt=el_wt_dict.get(src_trg_span,0)
  
  #continue  
  for inc0 in range(15): #increments to cover slices of src location indexes
    
    # test_wt=sorted(list(span_wt_dict.items()),key=lambda x:-x[-1])
    # tmp_full_span_wt=span_wt_dict.get(full_src_span,0)
    # test_wt1=[v for v in test_wt if v[0][0]==0]
    #print(inc0,test_wt[0],test_wt1[0],full_src_span, tmp_full_span_wt)
    test_el_wt=sorted(list(el_wt_dict.items()),key=lambda x:-x[-1])
    #test_el_wt1=[v for v in test_el_wt if v[0][0]==0]
    tmp_el_full_span_wt=el_wt_dict.get(src_trg_span,0)
    print(inc0,test_el_wt[0], tmp_el_full_span_wt)
    if tmp_el_full_span_wt>0:
      if tmp_el_full_span_wt>cur_el_full_span_wt: cur_el_full_span_wt=tmp_el_full_span_wt
      else: 
        print("alignment solved")
        break

    


    for i0 in range(len(src0)):
      i1=i0+inc0
      if i1>len(src0)-1: continue
      #if inc0>15: continue
      #first_items=el_start_dict.get(i0,[])
      #first_items=el_start_dict.get(i0,{})

      #New approach
      first_spans=start_span_dict.get(i0,[])
      first_elements=[]
      for sp0 in first_spans:
        first_elements.extend(span_top_elms_dict.get(sp0,[]))
      # print(i0)
      # print("first_spans",len(first_spans),first_spans[:5])
      # print("first_elements",len(first_elements),first_elements[:5])
      valid_next_items_counter=0
      for el0 in first_elements:
        el0_wt=el_wt_dict.get(el0,0)
        el0_src_span,el0_trg_span=el0
        el0_src_span_i0,el0_src_span_i1=el0_src_span
        el0_trg_span_i0,el0_trg_span_i1=el0_trg_span

        second_spans=[]
        second_elements=[]
        for i1 in range(el0_src_span_i1,el0_src_span_i1+5):
          second_spans.extend(start_span_dict.get(i1,[]))
        for sp1 in second_spans: second_elements.extend(span_top_elms_dict.get(sp1,[]))
        for el1 in second_elements:
          if el1==el0: continue
          el1_wt=el_wt_dict.get(el1,0)
          el1_src_span,el1_trg_span=el1
          el1_src_span_i0,el1_src_span_i1=el1_src_span
          el1_trg_span_i0,el1_trg_span_i1=el1_trg_span
          trg_spans_overlap=overlaps(el0_trg_span,el1_trg_span)
          src_spans_overlap=overlaps(el0_src_span,el1_src_span)
          if el0_src_span==el1_src_span and el0_src_span_i0!=el0_src_span_i1: continue #if [(1,2),(4,5)] [(1,2),(7,6)]
          #
          if trg_spans_overlap: continue #skip if target spans overlap
          if src_spans_overlap and el0_src_span!=el1_src_span: continue

          trg_span_inner_dist=get_inner_dist(el0_trg_span,el1_trg_span)
          if trg_span_inner_dist>10: continue #skip if there is much distance between target spans

          src_span_inner_dist=get_inner_dist(el0_src_span,el1_src_span)
          
          #if el0_src_span_i0==el0_src_span_i1==el1_src_span_i0 and el0_src_span_i1!=el1_src_span_i0: continue #vertical element with non-vertical element
          #if el0_src_span_i0==el0_src_span_i1==el1_src_span_i1 and el0_src_span_i1!=el1_src_span_i0: continue #vertical element with non-vertical element
          #print(el0,el1)
          delta_x=el0_src_span_i1-el0_src_span_i0
          excluded=False
          trg_outer_dist=max(el1_trg_span) - min(el0_trg_span)
          if el0_src_span==el1_src_span:
            if el1_trg_span_i1<el0_trg_span_i0: excluded=True
            elif trg_outer_dist>6: excluded=True
            elif trg_span_inner_dist>1: excluded=True
          #if delta_x<2 and trg_outer_dist>8: excluded=True
          
          combined_src_span=get_min_max(el0_src_span,el1_src_span)
          combined_trg_span=get_min_max(el0_trg_span,el1_trg_span) 
          if combined_src_span[1]-combined_src_span[0]<2:
            if combined_trg_span[1]-combined_trg_span[0]>8: excluded=True
          if combined_src_span[1]-combined_src_span[0]<3:
            if combined_trg_span[1]-combined_trg_span[0]>10: excluded=True
          

          el_pair=(el0,el1)
          if used_el_dict.get(el_pair,False)==True: excluded=True
          used_el_dict[el_pair]=True

          if excluded: continue

          
          combined_wt=el0_wt+el1_wt
          

          area_factor=get_area_factor(combined_src_span,combined_trg_span)
          #combined_wt=combined_wt-0.02*area_factor
          combined_wt=combined_wt-trg_span_inner_dist*0.05

          combined_el=(combined_src_span,combined_trg_span)
          found_wt=el_wt_dict.get(combined_el,0)
          if found_wt>0 and (combined_src_span[0]==combined_src_span[1]): continue
          if found_wt>0 and (combined_trg_span[0]==combined_trg_span[1]): continue
          if combined_wt>found_wt:
            el_wt_dict[combined_el]=combined_wt
            el_child_dict[combined_el]=(el0,el1)
            tmp_elements_in_span=span_top_elms_dict.get(combined_src_span,[])
            if not combined_el in tmp_elements_in_span:
              tmp_elements_in_span.append(combined_el)
              tmp_elements_in_span.sort(key=lambda x:-el_wt_dict.get(x,0))
              span_top_elms_dict[combined_src_span]=tmp_elements_in_span[:max_n_elms]
            tmp_spans_starting_with_i0=start_span_dict.get(i0,[])
            if not combined_src_span in tmp_spans_starting_with_i0:
              start_span_dict[i0]=tmp_spans_starting_with_i0+[combined_src_span]
          #if found_wt>0:
          if el0_src_span==el1_src_span and False:
            print(inc0,"excluded",excluded,trg_outer_dist)
            print(el0,el0_wt)
            print(el1,el1_wt)
            print("combined:",combined_el,combined_wt)
            print("found:",found_wt)
            print("-----")

          valid_next_items_counter+=1
          #if valid_next_items_counter>10: continue

        #print("second_elements",len(second_elements),second_elements[:5])
          

      
      continue

  all_elements2=get_rec_el_children(src_trg_span,el_child_dict,el_list0=[])
  #all_elements=get_rec_children(list_spans_wts[0][0],span_child_dict,span_el_dict,el_list0=[])
  for ch_el in all_elements2:
    print(ch_el)
    src_span0,trg_span0=ch_el
    cur_src_str=" ".join([src0[i_] for i_ in range(src_span0[0],src_span0[1]+1)])
    cur_trg_str=" ".join([trg0[i_] for i_ in range(trg_span0[0],trg_span0[1]+1)])  
    cur_pair=(cur_src_str,cur_trg_str)
    print(cur_src_str)   
    print(cur_trg_str) 
    print("--") 
    all_phrase_pairs.append((cur_pair,sent_counter))
  sent_counter+=1
  # print("\t".join(src0))
  # for t0 in trg0: print(t0)

# for a in all_phrase_pairs:
#   print(a)




0
['this', 'a', 'further', 'aspect', 'broader', 'task', 'settling', 'relationship', 'between', 'statute', 'existing', 'treaties']
['هذا', 'جانب', 'آخر', 'من', 'مهمة', 'أعم', 'متمثلة', 'في', 'تسوية', 'علاقة', 'بين', 'نظام', 'أساسي', 'معاهدات', 'قائمة']
0 (((0, 0), (0, 0)), 1) 0
1 (((11, 13), (14, 16)), 1.8009590917987863) 0
2 (((6, 13), (5, 16)), 3.6476920594577056) 2.66096910074624
3 (((0, 13), (0, 16)), 5.257242927554253) 5.257242927554253
4 (((0, 13), (0, 16)), 5.257242927554253) 5.257242927554253
alignment solved
((0, 9), (0, 11))
<s> this a further aspect broader task settling relationship between
<s> هذا جانب آخر من مهمة أعم متمثلة في تسوية علاقة بين
--
((0, 6), (0, 6))
<s> this a further aspect broader task
<s> هذا جانب آخر من مهمة أعم
--
((0, 1), (0, 1))
<s> this
<s> هذا
--
((0, 0), (0, 0))
<s>
<s>
--
((1, 1), (1, 1))
this
هذا
--
((2, 6), (2, 6))
a further aspect broader task
جانب آخر من مهمة أعم
--
((2, 4), (2, 4))
a further aspect
جانب آخر من
--
((2, 3), (3, 4))
a further
آخر 

In [ ]:
cur_span=(47, 47)
cur_span=(21, 22)
cur_span=(21, 21)
#cur_span=(22, 22)
#cur_span=(4, 5)
#cur_span=(34, 35)
#cur_span=(31, 31)
#cur_span=(28, 30)
#cur_span=(29, 29)
#cur_span=(29, 31)
#cur_span=(35, 35)
#cur_span=(21, 22)
cur_span=(23, 23)
test=span_top_elms_dict.get(cur_span,[]) 
for a in test:
  print(a, el_wt_dict.get(a,0))
  src_str=" ".join(src0[a[0][0]:a[0][1]+1])
  trg_str=" ".join(trg0[a[1][0]:a[1][1]+1])
  print(src_str)
  print(trg_str)
  print("----")  
# ((21, 22), (21, 22)) 0.9872094270568594
# secretary general
# أمين عام
# ((21, 21), (21, 21)) 0.911346964257843
# secretary
# أمين
# ----
# ((21, 21), (21, 22)) 1.4313109930348213
# secretary
# أمين عام

((23, 23), (11, 11)) 0.447362499598187
to
في
----
((23, 23), (14, 14)) 0.447362499598187
to
في
----
((23, 23), (32, 32)) 0.447362499598187
to
في
----
((23, 23), (20, 20)) 0.3907348358576708
to
من
----
((23, 23), (4, 4)) 0.3719612262313735
to
إلى
----


In [ ]:
el_child_dict[((21, 23), (20, 21))]

(((21, 21), (21, 21)), ((23, 23), (20, 20)))

In [ ]:
for a,b in el_wt_dict.items():
  if a[0][1]>a[0][0]+3: continue
  print(a,b)
  src_str=" ".join(src0[a[0][0]:a[0][1]+1])
  trg_str=" ".join(trg0[a[1][0]:a[1][1]+1])
  print(src_str, trg_str)
  print("----")
  #print(trg0[a[1][0]:a[1][1]+1])

Streaming output truncated to the last 5000 lines.
not be used نظام لا يجوز استخدام ـها كـ ذريعة لقمع حريات أساسية وـ
----
((6, 9), (4, 14)) 0.5698722927490186
not be used as نظام لا يجوز استخدام ـها كـ ذريعة لقمع حريات أساسية وـ
----
((6, 7), (9, 14)) 0.19729898966059234
not be كـ ذريعة لقمع حريات أساسية وـ
----
((6, 7), (14, 15)) 0.18138297549981997
not be وـ حقوق
----
((6, 7), (7, 14)) 0.17561852475700077
not be استخدام ـها كـ ذريعة لقمع حريات أساسية وـ
----
((6, 7), (13, 14)) 0.17270720657494257
not be أساسية وـ
----
((6, 8), (7, 14)) 0.12807995997190613
not be used استخدام ـها كـ ذريعة لقمع حريات أساسية وـ
----
((6, 8), (10, 14)) 0.0807053147495788
not be used ذريعة لقمع حريات أساسية وـ
----
((6, 8), (9, 14)) 0.07230965281988545
not be used كـ ذريعة لقمع حريات أساسية وـ
----
((6, 8), (13, 14)) 0.06553552029024011
not be used أساسية وـ
----
((6, 9), (10, 14)) 0.16873566921923983
not be used as ذريعة لقمع حريات أساسية وـ
----
((6, 9), (7, 14)) 0.17509057404612044
not be used as استخ

In [ ]:
test_ch=((17, 18), (3, 16))
children0=el_child_dict.get(test_ch,[])
#children0=get_rec_el_children(test_ch,el_child_dict,[])
for ch0 in children0:
  print(ch0)
# print(span_top_elms_dict[(57,57)])
# print(start_span_dict[56])

((17, 17), (3, 3))
((17, 18), (8, 16))


In [ ]:
cur_items=list(span_child_dict.items())
for a in cur_items[:100]:
  print(a)

((1, 1), [])
((47, 47), [])
((47, 48), [(47, 48), (47, 47)])
((45, 45), [])
((45, 46), [(45, 45), (46, 46)])
((51, 51), [])
((51, 52), [(51, 52), (51, 51)])
((57, 57), [])
((57, 58), [(57, 57), (58, 58)])
((43, 43), [(43, 43), (43, 43)])
((49, 49), [(49, 49), (49, 49)])
((43, 44), [(43, 44), (43, 43)])
((49, 50), [(49, 50), (49, 49)])
((42, 42), [])
((42, 43), [(42, 43), (42, 42)])
((54, 54), [])
((54, 55), [(54, 54), (55, 55)])
((55, 55), [])
((55, 56), [(55, 55), (56, 56)])
((41, 41), [])
((41, 42), [(41, 41), (42, 42)])
((46, 46), [])
((46, 47), [(46, 46), (47, 47)])
((53, 53), [])
((53, 54), [(53, 53), (54, 54)])
((58, 58), [])
((0, 0), [])
((0, 1), [(0, 0), (1, 1)])
((48, 48), [])
((48, 49), [])
((21, 21), [(21, 21), (21, 21)])
((21, 22), [(21, 22), (21, 21)])
((31, 31), [])
((31, 32), [])
((27, 27), [])
((27, 28), [])
((2, 2), [])
((15, 15), [(15, 15), (15, 15)])
((19, 19), [(19, 19), (19, 19)])
((52, 52), [])
((52, 53), [])
((15, 16), [(15, 15), (16, 16)])
((19, 20), [(19, 19), 

In [ ]:
fopen=open("test-out-align.txt","w")
for a in all_phrase_pairs:
  line=" ".join([str(v) for v in a])+"\n"
  fopen.write(line)
fopen.close()
  #print(a)

In [ ]:
from collections import Counter
just_pairs=[]
for a in all_phrase_pairs:
  pair,sent_i=a
  src0,trg0=pair
  just_pairs.append(pair)
just_pairs.sort()
grouped=[(key,[v[1] for v in list(group)]) for key,group in groupby(just_pairs,lambda x:x[0])]
grouped.sort(key=lambda x:len(x[1]))
for k0,grp0 in grouped:
  print(k0)
  #print(grp0)
  print(Counter(grp0))
  print("----")

Streaming output truncated to the last 5000 lines.
Counter({'لـ قياس قدرة': 1})
----
mechanism
Counter({'آلية': 1})
----
mechanisms were
Counter({'كانت آليات': 1})
----
meeting
Counter({'اجتماع': 1})
----
meetings
Counter({'اجتماعات': 1})
----
member delegation concerned
Counter({'أعضاء وفد معني': 1})
----
member states at
Counter({'دول أعضاء في': 1})
----
member states at current meeting
Counter({'دول أعضاء في اجتماع حالي': 1})
----
member states but
Counter({'دول أعضاء ولكن': 1})
----
member states but was nevertheless
Counter({'دول أعضاء ولكن ـها مع ذلك': 1})
----
member states but was nevertheless essential to ensuring smooth operation organization </s>
Counter({'دول أعضاء ولكن ـها مع ذلك ضرورية لـ ضمان سلاسة عمل منظمة </s>': 1})
----
member states capacity
Counter({'قدرة دول أعضاء': 1})
----
member states capacity to
Counter({'إلى قدرة دول أعضاء': 1})
----
member states capacity to pay and on
Counter({'إلى قدرة دول أعضاء على دفع وـ': 1})
----
member states capacity to pay and on m

In [ ]:
def get_dist_factor(el1,el2): #[(5,6),(9,9)] - [(7,7),(12,13)] #get inner distance between two elements, 1 if adjacent
  el1_xrange,el1_yrange=el1
  el2_xrange,el2_yrange=el2
  combined_xs=[]
  combined_xs.extend(el1_xrange)
  combined_xs.extend(el2_xrange)
  combined_ys=[]
  combined_ys.extend(el1_yrange)
  combined_ys.extend(el2_yrange)
  combined_xs.sort()
  combined_ys.sort()
  inner_xdist=combined_xs[2]-combined_xs[1]
  inner_ydist=combined_ys[2]-combined_ys[1]
  return (inner_xdist+inner_ydist)/2
test_el1=[(5,6),(9,9)]
test_el2=[(6,6),(10,11)]
print(get_dist_factor(test_el1,test_el2))

0.5


In [ ]:
def overlaps(span1,span2):
  span1_i0,span1_i1=span1
  span2_i0,span2_i1=span2
  if span1_i0>=span2_i0 and span1_i0<=span2_i1: return True
  if span1_i1>=span2_i0 and span1_i1<=span2_i1: return True
  if span2_i0>=span1_i0 and span2_i0<=span1_i1: return True
  if span2_i1>=span1_i0 and span2_i1<=span1_i1: return True
  return False
sp1=(2,3)
sp2=(4,4)
print(overlaps(sp1,sp2))

False


In [ ]:
def get_inner_dist(span1,span2):
  combined_xs=[]
  combined_xs.extend(span1)
  combined_xs.extend(span2)  
  combined_xs.sort()
  return combined_xs[2]-combined_xs[1]
sp1=(22,30)
sp2=(14,16)
print(get_inner_dist(sp1,sp2))

6


In [ ]:
for ix in range(1,10):
  print(ix,math.log(ix))

1 0.0
2 0.6931471805599453
3 1.0986122886681098
4 1.3862943611198906
5 1.6094379124341003
6 1.791759469228055
7 1.9459101490553132
8 2.0794415416798357
9 2.1972245773362196


In [ ]:
import pandas as pd
xls_path='https://docs.google.com/spreadsheets/d/e/2PACX-1vSqipSUAaMWJrizvZtg1WIpds2pIiXtcZC6wjOLeVYie9p5O0lqiPnVdd8ambGKH8B0qJ4kmlVeEs6f/pub?output=xlsx'
cur_xls = pd.ExcelFile(xls_path)
cur_sheet_name="2"
cur_sheet_obj=pd.read_excel(cur_xls, cur_sheet_name,keep_default_na=False)
sent_pair_list=[]
for i,row in enumerate(cur_sheet_obj.iterrows()):
  row_dict=row[1].to_dict()
  sent_pair_list.append((row_dict["en"],row_dict["ar"]))

for a in sent_pair_list:
  print(a)
# print(cur_sheet_obj.keys())

('A/C.5/76/SR.2', 'A/C.5/76/SR.2')
('A/C.5/76/SR.2', 'A/C.5/76/SR.2')
('United Nations', 'الأمم المتحدة')
('A/C.5/76/SR.2', 'A/C.5/76/SR.2')
('General Assembly', 'الجمعية العامة')
('Seventy-sixth session', 'الدورة السادسة والسبعون')
('Official Records', 'وثيقة رسمية')
('Distr.: General', 'Distr.: General')
('24 January 2022', '24 January 2022')
('Original: English', 'Arabic Original: English')
('21-14241', '21-14241')
('This record is subject to correction.', 'هذا المحضر قابل للتصويب.')
('Corrections should be sent as soon as possible, under the signature of a member of the delegation concerned, to the Chief of the Documents Management Section (dms@un.org), and incorporated in a copy of the record.', 'وينبغي إدراج التصويبات في نسخة من المحضر مذيلة بتوقيع أحد أعضاء الوفد المعني وإرسالها في أقرب وقت ممكن إلى: Chief of the Documents Management Section (dms@un.org).')
('Corrected records will be reissued electronically on the Official Document System of the United Nations (http://documents

In [ ]:
def idx_match(idx1,idx2,min_common=5): #This function is to give a ratio of matching between the inverted indexes for two words
    if len(idx1)==0 or len(idx2)==0: return 0
    intersection=len(list(set(idx1).intersection(set(idx2))))
    union=len(idx1)+len(idx2)-intersection
    if union==0: return 0
    if intersection<min_common: return 0    
    return float(intersection)/union

test_idx1=[1,2,4,5,6,9]
test_idx2=[5,6,9,12,25]
idx_match(test_idx1,test_idx2,min_common=2)

0.375

In [ ]:
def filter_tokens(token_list0, lang="en",stop_words=[], ignore_punc=False,ignore_ar_pre_suf=False,remove_al=True,index_words=True,lower=True,stemming=False):
  tokens_original=list(token_list0)
  tokens_copy_enum=[(i,v) for i,v in enumerate(tokens_original)]   
  #if lang=="ar": tokens=tok_ar(tokens,count_dict) #tok_ar(tokens)
  if ignore_punc: tokens_copy_enum=[(i,v) for i,v in tokens_copy_enum if not is_punct(v)]
  if ignore_ar_pre_suf: tokens_copy_enum=[(i,v) for i,v in tokens_copy_enum if not v.startswith("ـ") and not v.endswith("ـ")]
  if stop_words: tokens_copy_enum=[(i,v) for i,v in tokens_copy_enum if not v.lower() in stop_words]
  if remove_al: tokens_copy_enum=[(i,v.replace("ال_","")) for i,v in tokens_copy_enum if not v.lower() in stop_words]
  tokens_out=[v[1] for v in tokens_copy_enum]
  if lower: tokens_out=[v.lower() for v in tokens_out]
  cur_mapping=[v[0] for v in tokens_copy_enum]
  return tokens_out,cur_mapping

sent='As only one country had benefited from that ceiling historically, it should be reviewed in accordance with General Assembly resolution 55/5 C.'
tokens0=tok(sent)
a,b=filter_tokens(tokens0, ignore_punc=True)
print(a)
print(b)

['as', 'only', 'one', 'country', 'had', 'benefited', 'from', 'that', 'ceiling', 'historically', 'it', 'should', 'be', 'reviewed', 'in', 'accordance', 'with', 'general', 'assembly', 'resolution', '55', '5', 'c']
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 23, 24]


In [ ]:
#DUMP


  #     #print(first_items)
  #     #print(first_items[:3])
  #     #next_items=[]
  #     # next_items=list(first_items)
  #     # for tmp_i in range(i0+1,i1+1):
  #     #   next_items+=el_start_dict.get(tmp_i,[]) 
  #     #tmp_final_list=list(first_items)
  #     tmp_final_list=[] #the list of combined items with the first and next items
  #     #first_items.sort(key=lambda x:-x[-1])
  #     #next_items.sort(key=lambda x:-x[-1])
  #     #print(i0,i1)
  #     # print("first_items",len(first_items),"next_items",len(next_items))
  #     # print("first_items",first_items[:5],"next_items",next_items[:5])
      
  #     # for ni in next_items[-50:]:
  #     #   ni_element,ni_cur_ratio=ni
  #     #   print("NI",ni)
  #     # print("========")
  #     #for fi in first_items:
  #     cur_first_items=list(first_items.items())
  #     for fi in cur_first_items:
  #       fi_element,fi_cur_ratio=fi
  #       fi_src_range,fi_trg_range=fi_element
  #       fi_src_i0,fi_src_i1=fi_src_range
  #       fi_trg_i0,fi_trg_i1=fi_trg_range
  #       valid_next_items_counter=0
  #       next_items=list(cur_first_items)
  #       for i_plus in range(fi_src_i1+1,fi_src_i1+5):
  #         #next_items+=el_start_dict.get(i_plus,[]) 
  #         test_next=el_start_dict.get(i_plus,{})
  #         for nx in test_next.items(): next_items.append(nx)

  #         #print("test_next",test_next)
        

  #       # print(fi, len(next_items))
  #       # print(next_items[:3])
  #       # print("-------")
  #       # continue

  #       for ni in next_items:
  #         ni_element,ni_cur_ratio=ni
  #         if fi_element==ni_element: continue
  #         ni_src_range,ni_trg_range=ni_element
  #         ni_src_i0,ni_src_i1=ni_src_range
  #         ni_trg_i0,ni_trg_i1=ni_trg_range   
  #         if ni_src_i0>fi_src_i1+10: continue

  #         trg_spans_overlap=overlaps(fi_trg_range,ni_trg_range)
  #         if trg_spans_overlap: continue #skip if target spans overlap

  #         trg_span_inner_dist=get_inner_dist(fi_trg_range,ni_trg_range)
  #         if trg_span_inner_dist>10: continue #skip if there is much distance between target spans

  #         src_span_inner_dist=get_inner_dist(fi_src_range,ni_src_range)
          
          
          
  #         if fi_src_i0==fi_src_i1==ni_src_i0 and ni_src_i0!=ni_src_i1: continue #vertical element with non-vertical element
  #         #if ni_src_i0<=fi_src_i1: continue #skip if src spans overlap
          
          
  #         #if src_span_inner_dist>10: continue #skip if there is much distance between source spans
  #         if fi_src_range==ni_src_range: 
  #           if fi_src_i0!=fi_src_i1: continue
  #           if ni_trg_i0<fi_trg_i0: continue
  #           #print(">>>>> Vertical element",fi_element,ni_element)

  #         el_pair=(fi_element,ni_element)
  #         if used_el_dict.get(el_pair,False)==True: continue
  #         used_el_dict[el_pair]=True

  #         valid_next_items_counter+=1

  #         new_src_range=get_min_max(fi_src_range,ni_src_range)
  #         new_trg_range=get_min_max(fi_trg_range,ni_trg_range)
  #         combined_ratio=fi_cur_ratio+ni_cur_ratio
  #         cur_area_factor=get_area_factor(new_src_range,new_trg_range)

  #         if cur_area_factor<1: continue
  #         #adj_ratio=combined_ratio/cur_area_factor
  #         #adj_ratio=combined_ratio-trg_span_inner_dist*0.05-src_span_inner_dist*0.05-cur_area_factor*0.005
  #         adj_ratio=combined_ratio
  #         adj_wt=combined_ratio
  #         cur_el=(new_src_range,new_trg_range)
  #         found_span_wt=span_wt_dict.get(new_src_range,0)
  #         if adj_wt>found_span_wt:


  #           el_start_dict[i0][cur_el]=adj_wt
  #           span_wt_dict[new_src_range]=adj_wt
  #           span_el_dict[new_src_range]=cur_el
  #           # print("fi_element,fi_cur_ratio",fi)
  #           # print("ni_element,ni_cur_ratio",ni)
  #           # print("cur_el,adj_wt",cur_el,adj_wt)
  #           # print("el_start_dict[i0][cur_el]",el_start_dict[i0].get(cur_el,0))
  #           # print("UPDATED el_start_dict[i0][cur_el]",el_start_dict[i0].get(cur_el,0))
  #           # print("------------------")

  #           if new_src_range!=fi_src_range and new_src_range!=ni_src_range:  
  #             span_child_dict[new_src_range]=[fi_src_range,ni_src_range]
  #         found_ratio=tmp_ratio_dict.get(cur_el,0)
  #         if adj_ratio>found_ratio:
  #           tmp_final_list.append((cur_el,adj_ratio))
      
  #     # print("first_items",len(first_items),"next_items",len(next_items),"valid",valid_next_items_counter)
  #     # print("used_el_dict", len(used_el_dict.keys()))
  #     # if valid_next_items_counter==0:
  #     #   for fi in first_items:
  #     #     print(fi)
      
  #     tmp_final_list.sort(key=lambda x:-x[-1])
  #     used_els=[]
  #     final_items=[]
  #     for tp in tmp_final_list:
  #       el0,ratio0=tp
  #       if el0 in used_els: continue
  #       used_els.append(el0)
  #       final_items.append(tp)
  #       src_range0,trg_range0=tp[0]        
  #       cur_src_str=" ".join([src0[i_] for i_ in range(src_range0[0],src_range0[1]+1)])
  #       cur_trg_str=" ".join([trg0[i_] for i_ in range(trg_range0[0],trg_range0[1]+1)])

  #     final_items.sort(key=lambda x:-x[-1])
  #     #el_start_dict[i0]=first_items+final_items[:5] ##### CHECK

  # # list_spans_wts=[]
  # # for a,b in span_wt_dict.items():
  # #   list_spans_wts.append((a,b))
  
  # # list_spans_wts.sort(key=lambda x:-x[-1])

  # # #get_rec_children(span0,span_child_dict0,span_el_dict0,el_list0=[])
  # # print(list_spans_wts[0])
  # tmp_full_span_wt=span_wt_dict.get(full_src_span,0)
  #all_elements=get_rec_children(full_src_span,span_child_dict,span_el_dict,el_list0=[])

#Translation/QA from reference bitexts

In [ ]:
#Preparing references
import pandas as pd
import re
def get_bitext(xls_path0,sheet_name0="",src_col="en",trg_col="ar",filter_non_alpha=False, filter_numbers=False):
  #xls_path='https://docs.google.com/spreadsheets/d/e/2PACX-1vSVy7hY4fV0LHqTCypOjZAM9OrBevDDnpIcEKEAPKDsYRoxAz9TUBcDIgy8PvHkw5IDAZ6eRmKdZuRw/pub?output=xlsx'
  cur_xls = pd.ExcelFile(xls_path0)
  bitext_list=[]
  if sheet_name0!="": cur_sheet_list=[sheet_name0]
  else: cur_sheet_list=cur_xls.sheet_names
  for cur_sheet_name in cur_sheet_list:
    print(cur_sheet_name)
    cur_sheet_obj=pd.read_excel(cur_xls, cur_sheet_name,keep_default_na=False)
    cur_sheet_obj[src_col]=cur_sheet_obj[src_col].apply(str)
    cur_sheet_obj[trg_col]=cur_sheet_obj[trg_col].apply(str)
    for i,row in enumerate(cur_sheet_obj.iterrows()):
      row_dict=row[1].to_dict()
      en0,ar0=row_dict["en"],row_dict["ar"]
      test_alpha=re.sub("\W+","",en0)
      if filter_non_alpha and test_alpha=="": continue
      if filter_numbers and test_alpha.isdigit(): continue
      if trg_col=="ar": ar0=clean_ar(ar0)
      bitext_list.append((en0,ar0))
  return bitext_list


def get_key(txt):
  txt=re.sub("\W+","_",txt)
  txt=txt.strip("_")
  return txt.lower()
#Ref bitexts
xls_path='https://docs.google.com/spreadsheets/d/e/2PACX-1vSVy7hY4fV0LHqTCypOjZAM9OrBevDDnpIcEKEAPKDsYRoxAz9TUBcDIgy8PvHkw5IDAZ6eRmKdZuRw/pub?output=xlsx'
cur_xls = pd.ExcelFile(xls_path)
translation_dict={}
for cur_sheet_name in cur_xls.sheet_names:
  print(cur_sheet_name)
  cur_sheet_obj=pd.read_excel(cur_xls, cur_sheet_name,keep_default_na=False)
  cur_sheet_obj['en']=cur_sheet_obj['en'].apply(str)
  cur_sheet_obj['ar']=cur_sheet_obj['ar'].apply(str)

  sent_pair_list=[]
  key_str_pairs=[]
  for i,row in enumerate(cur_sheet_obj.iterrows()):
    row_dict=row[1].to_dict()
    en0,ar0=row_dict["en"],row_dict["ar"]
    en_key=get_key(en0)
    test_alpha=re.sub("\W+","",en0)
    if test_alpha=="": continue
    if test_alpha.isdigit(): continue

    key_str_pairs.append((en_key,en_key))
    #if en0.replace(".","").replace("-","").replace("/","").isdigit(): continue
    ar0=clean_ar(ar0)
    tmp_translation_dict=translation_dict.get(en_key,{})
    tmp_translation_dict[ar0]=tmp_translation_dict.get(ar0,0)+1
    translation_dict[en_key]=tmp_translation_dict
    sent_pair_list.append((en0,ar0))
    src_sents.append(en0)
    trg_sents.append(ar0)
  #for a in sent_pair_list[:20]: print(a)
test_items=list(translation_dict.items())
test_items.sort(key=lambda x:-sum(x[1].values()))
for a,b in test_items[:200]:
  b_items=list(b.items())
  b_items.sort(key=lambda x:-x[1])
  for a1,b1 in b_items:
    print(a,a1,b1)
  print("-----")

A75760


NameError: ignored

In [ ]:
from itertools import groupby
all_ref_keys=list(translation_dict.keys())
first_word_list=[(v.split("_")[0],v) for v in all_ref_keys]
first_word_list.sort()
first_word_list_grouped=[(key,[v[1] for v in list(group)]) for key,group in groupby(first_word_list,lambda x:x[0])]
first_word_dict=dict(iter(first_word_list_grouped))
first_word_dict_items=list(first_word_dict.items())
print(len(first_word_dict_items))
for a,b in first_word_dict_items[1000:1020]:
  print(a,b)
#print(all_ref_keys[:20])

1377
ptsd ['ptsd', 'ptsd_claims_are_assessed_and_processed_within_90_days_of_receipt_of_required_documentation_percentage', 'ptsd_claims_reviewed_and_processed_within_90_days_after_receipt_of_required_documentation_percentage']
public ['public_awareness_raising_activity_on_the_protection_monitoring_and_reporting_mechanism_on_grave_violations_against_children_in_situations_of_armed_conflicts_for_2_594_participants_including_552_women', 'public_events', 'public_events_were_conducted_pending_the_publication_of_the_report_on_human_rights_trends', 'public_health', 'public_information_and_publication_services', 'public_information_and_publication_services_including_subscriptions', 'public_information_and_publication_services_translation', 'public_information_campaigns_on_the_electoral_cycle_process_based_on_the_scope_of_the_mission_s_mandate_aimed_at_informing_malian_citizens_and_voters_about_the_electoral_procedures_to_prevent_conflicts_triggered_by_the_electoral_process_through_60_radio_pr

In [ ]:
from code_utils.general import * 
import code_utils.general as utils

job_xls_path='https://docs.google.com/spreadsheets/d/e/2PACX-1vSqipSUAaMWJrizvZtg1WIpds2pIiXtcZC6wjOLeVYie9p5O0lqiPnVdd8ambGKH8B0qJ4kmlVeEs6f/pub?output=xlsx'
cur_sheet_name="current"
cur_job_bitext=get_bitext(job_xls_path,sheet_name0=cur_sheet_name,src_col="en",trg_col="ar",filter_non_alpha=True, filter_numbers=True)
cur_terminology_bitext=get_bitext(job_xls_path,sheet_name0="terminology",src_col="en",trg_col="ar",filter_non_alpha=True, filter_numbers=True)
src_only=[v[0] for v in cur_job_bitext]
xl8_list=[]
for en0 in src_only:
  match_type="" #exact - terminology - template - linear - MT ... ??
  en_key0=get_key(en0) #first check the exact match
  eq=translation_dict.get(en_key0,{})
  eq_items=list(eq.items())
  eq_items.sort(key=lambda x:-x[1])  
  selected_eq=""
  if eq_items: 
    selected_eq=eq_items[0][0]
    match_type="exact"
  if match_type!="": continue
  print(en0)
  
  
  en_key0_split=en_key0.split("_")
  words=list(set(en_key0_split))
  valid_keys_tokens=[]
  for w in words:
    corr_keys=first_word_dict.get(w,[])
    for k0 in corr_keys:
      k0_split=k0.split("_")
      if list_in_list(k0_split,en_key0_split): valid_keys_tokens.append(k0)
  valid_keys_tokens.sort(key=lambda x:-len(x[0]))
  print(valid_keys_tokens)
  tmp_linear_str=" ".join(en_key0_split)
  for vk in valid_keys_tokens:
    key_str=vk.replace("_"," ")
    corr=translation_dict.get(vk,{})
    corr_items=sorted(list(corr.items()),key=lambda x:-x[1])
    top_corr_str=corr_items[0][0]
    tmp_linear_str=re.sub(r'\b%s\b'%key_str,top_corr_str,tmp_linear_str)
    #print(vk, corr_items[0])
  print(tmp_linear_str)

  
  xl8_list.append((en0,selected_eq,match_type))




test_out_fopen=open("xl8_out.txt","w")
for item in xl8_list:
  test_out_fopen.write("\t".join(item)+"\n")
test_out_fopen.close()  


current
terminology
A/76/xxx
['a']
(أ) 76 xxx
A/76/XXX
['a']
(أ) 76 xxx
MINUSCA will seek to strengthen its interregional cooperation and economies of scale with missions in the region, including the United Nations Regional Office for Central Africa (UNOCA), to enhance collaboration, information-sharing and analysis, and facilitation, as appropriate.
['united_nations', 'office', 'minusca']
بعثة الأمم المتحدة المتكاملة المتعددة الأبعاد لتحقيق الاستقرار في جمهورية أفريقيا الوسطى will seek to strengthen its interregional cooperation and economies of scale with missions in the region including the الأمم المتحدة regional المكتب for central africa unoca to enhance collaboration information sharing and analysis and facilitation as appropriate
MINUSCA will seek to leverage existing assets of other missions following their downsizing and withdrawal to meet its operational requirements, if any.
['other', 'minusca']
بعثة الأمم المتحدة المتكاملة المتعددة الأبعاد لتحقيق الاستقرار في جمهورية أفريقيا

#OLD

In [ ]:

#Job Bitexts
xls_path='https://docs.google.com/spreadsheets/d/e/2PACX-1vSqipSUAaMWJrizvZtg1WIpds2pIiXtcZC6wjOLeVYie9p5O0lqiPnVdd8ambGKH8B0qJ4kmlVeEs6f/pub?output=xlsx'
cur_xls = pd.ExcelFile(xls_path)
cur_sheet_name="current"
cur_job_bitext=get_bitext(xls_path,sheet_name0=cur_sheet_name,src_col="en",trg_col="ar",filter_non_alpha=True, filter_numbers=True)

cur_sheet_obj=pd.read_excel(cur_xls, cur_sheet_name,keep_default_na=False)
cur_sheet_obj['en']=cur_sheet_obj['en'].apply(str)
cur_sheet_obj['ar']=cur_sheet_obj['ar'].apply(str)

job_pair_list=[]
for i,row in enumerate(cur_sheet_obj.iterrows()):
  row_dict=row[1].to_dict()
  en0,ar0=row_dict["en"],row_dict["ar"]
  en_key=get_key(en0)

  
  if en_key.replace("_","")=="": continue
  if en_key.replace("_","").isdigit(): continue

  # print(en_key)
  # print(en0)
  # print("------")

  ar0=clean_ar(ar0)
  job_pair_list.append((en_key,en0,ar0))
translated_list=[]
used_en=[]
for en_key0,en0,ar0 in job_pair_list:
  #continue
  
  # if en_key0 in used_en: continue
  # used_en.append(en_key0)
  #print(en0)
  eq=translation_dict.get(en_key0,{})
  eq_items=list(eq.items())
  eq_items.sort(key=lambda x:-x[1])  
  selected_eq=""
  if eq_items: selected_eq=eq_items[0][0]
  translated_list.append((en0,selected_eq))
  # for eq_ar0 in eq_items:
  #   print(eq_ar0)
  # print("----")

test_out_fopen=open("xl8_out.txt","w")
for item in translated_list:
  test_out_fopen.write("\t".join(item)+"\n")
test_out_fopen.close()  


#Classification/embedding with gensim

In [ ]:
#Create a model
import time, os, re
#from code_utils.general import *
from gensim.models import Word2Vec

def tok(txt):
  return re.findall("\w+",txt)


en_corpus_fname="UNv1.0.ar-en.en"

n_sents_max=500000
en_fopen=open(en_corpus_fname)

src_sents=[]
for i,en_line in enumerate(en_fopen):
  if i>=n_sents_max: break
  if i%50000==0: print(i)
  en_line=en_line.strip("\n\r\t ")
  toks=tok(en_line.lower())
  if len(toks)<3: continue
  src_sents.append(toks)

#ar_fopen.close()
en_fopen.close()


# def get_sentences(fpath):
#   fopen=open(fpath)
#   tmp_sentences=[]
#   sent_counter=0
#   completed=False
#   for i,site_content in enumerate(fopen):
#     if completed: break
#     content_split=site_content.split("<br>")
#     #print(len(content_split), content_split[0])
#     if len(content_split)<4: continue
#     for line in content_split:
#       line=line.strip("\n\r\t ")
#       toks=tok(line.lower())
#       if len(toks)<3: continue
#       sent_counter+=1
#       if sent_counter%10000==0: print(sent_counter)
#       if sent_counter>2000000: 
#         completed=True
#         break
#       tmp_sentences.append(toks)
#       #yield toks
#   fopen.close()
#   return tmp_sentences

model_fname="un_en.wv"

# t0=time.time()
# data_fpath="crawl/oct21_au/cached_all.txt"
t0=time.time()
#sentences=get_sentences(data_fpath)
print("collected %s sentences"%len(src_sents))
print("Now creating the WV model")
un_model = Word2Vec(src_sents, size=100, min_count=5) #small model
t1=time.time()
print("collected sentences in %s seconds"%(t1-t0))
#fopen.close()

un_model.save(model_fname)
print("saved model as: ",model_fname)

0
50000
100000
150000
200000
250000
300000
350000
400000
450000
collected 474505 sentences
Now creating the WV model
collected sentences in 90.76132202148438 seconds
saved model as:  un_en.wv


In [ ]:
import gensim
from collections import Counter
from scipy import spatial

def get_words_vector(words,wv_model,excluded_words=[]):
  wd_counter=Counter(words)
  flag=False
  total_count0=0
  wd_vector_dict={}
  for wd0,wd0_n in wd_counter.items():
    if wd0 in excluded_words: continue
    try: cur_vec0=wv_model[wd0]
    except: continue
    wd_vector_dict[wd0]=cur_vec0
    weighted_vec0=cur_vec0*wd0_n
    if flag==False: total_vec0=weighted_vec0
    else: total_vec0+=weighted_vec0
    flag=True
    total_count0+=wd0_n
  if total_count0==0: return [],{}
  avg_vec=total_vec0/total_count0
  return avg_vec,wd_vector_dict

#cosine similarity between word vectors
def cos_sim(vector1,vector2):
  result = 1 - spatial.distance.cosine(vector1, vector2)
  return result

un_model = gensim.models.Word2Vec.load("un_en.wv")
#un_model["about"]
un_model.most_similar("agriculture")

cur_sent='To reduce transnational threats and enhance stabilization efforts, the Mission will continue to work closely with regional actors for the full operationalization of joint mixed commissions and fully functional cross-border mechanisms, in close cooperation with UNOCA.'
cur_tokens=tok(cur_sent)
sent_vec,_=get_words_vector(cur_tokens,un_model)
term1='United Nations Regional Office for Central Africa'
term2='United Nations Operations Centre in Afghanistan'

terms=['United Nations Regional Office for Central Africa','United Nations Operations Centre in Afghanistan',"Peace and Security","Human Rights","Environment","Spain","General Assembly","International Law"]
for tr in terms:
  term_tokens=tok(tr.lower())
  term_vec,_=get_words_vector(term_tokens,un_model)
  sim0=cos_sim(sent_vec,term_vec)
  print(tr,sim0)

# term1='Security'
# term2='Environment'
# term1_tokens=tok(term1.lower())
# term2_tokens=tok(term2.lower())
# term1_vec,_=get_words_vector(term1_tokens,un_model)
# term2_vec,_=get_words_vector(term2_tokens,un_model)
# print(cos_sim(sent_vec,term1_vec))
# print(cos_sim(sent_vec,term2_vec))



# term1_tokens=tok(term1.lower())
# term2_tokens=tok(term2.lower())
# term1_vec,_=get_words_vector(term1_tokens,un_model)
# term2_vec,_=get_words_vector(term2_tokens,un_model)
# print(cos_sim(sent_vec,term1_vec))
# print(cos_sim(sent_vec,term2_vec))


United Nations Regional Office for Central Africa 0.6001240015029907
United Nations Operations Centre in Afghanistan 0.538229763507843
Peace and Security 0.45091143250465393
Human Rights 0.27792084217071533
Environment 0.26981398463249207
Spain -0.17120014131069183
General Assembly -0.005136891268193722
International Law 0.3549458980560303


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  if sys.path[0] == '':


#Parsing and Segmentation Experiments

In [ ]:
from code_utils.parsing_lib import *
from code_utils.dep_lib import *


In [ ]:
sent1="""
At the same time, in line with the Secretary-General call for operating as ‘One UN’ and as part of DOS/OSCM, the Centre is responding to demands to achieve system-wide efficiencies and to deliver solutions to a broader client base that includes the wider United Nations system.
"""
conll0=get_conll(sent1.strip())
for c_line in conll0:
  print("\t".join([str(v) for v in c_line]))

1	At	at	IN	ADP	_	0	ROOT	_	_
2	the	the	DT	DET	_	4	det	_	_
3	same	same	JJ	ADJ	_	4	amod	_	_
4	time	time	NN	NOUN	_	1	pobj	_	_
5	,	,	,	PUNCT	_	1	punct	_	_
6	in	in	IN	ADP	_	5	intj	_	_
7	line	line	NN	NOUN	_	6	pobj	_	_
8	with	with	IN	ADP	_	7	prep	_	_
9	the	the	DT	DET	_	13	det	_	_
10	Secretary	Secretary	NNP	PROPN	_	12	compound	_	_
11	-	-	HYPH	PUNCT	_	12	punct	_	_
12	General	General	NNP	PROPN	_	13	compound	_	_
13	call	call	NN	NOUN	_	8	pobj	_	_
14	for	for	IN	ADP	_	13	prep	_	_
15	operating	operate	VBG	VERB	_	14	pcomp	_	_
16	as	as	IN	SCONJ	_	15	prep	_	_
17	‘	'	``	PUNCT	_	19	punct	_	_
18	One	one	CD	NUM	_	19	nummod	_	_
19	UN	UN	NNP	PROPN	_	16	pobj	_	_
20	’	'	''	PUNCT	_	19	punct	_	_
21	and	and	CC	CCONJ	_	32	cc	_	_
22	as	as	IN	SCONJ	_	32	prep	_	_
23	part	part	NN	NOUN	_	22	pobj	_	_
24	of	of	IN	ADP	_	23	prep	_	_
25	DOS	DOS	NNP	PROPN	_	27	nmod	_	_
26	/	/	SYM	SYM	_	27	punct	_	_
27	OSCM	OSCM	NNP	PROPN	_	24	pobj	_	_
28	,	,	,	PUNCT	_	32	punct	_	_
29	the	the	DT	DET	_	30	det	_	_
30	Centre	Centre	NNP	PROPN	_	32	

In [ ]:
# for c_line in conll0:
#   print("\t".join([str(v) for v in c_line])+"\n")
link_count_dict={}
side_dict={}
tok_dict={}
left_emanating_dict={}
right_emanating_dict={}

token_list=[v[1] for v in conll0]
for c_i,c_line in enumerate(conll0):
  cur_token=c_line[1]
  cur_tag=c_line[4]
  cur_i=int(c_line[0])
  cur_parent_i=int(c_line[6]) 
  tok_dict[cur_i]=cur_token
  
  if cur_parent_i==0: cur_side="0"
  elif cur_parent_i>cur_i: cur_side="R"
  elif cur_parent_i<cur_i: cur_side="L"
  side_dict[cur_i]=cur_side
  if cur_side=="R": left_emanating_dict[cur_parent_i]=left_emanating_dict.get(cur_parent_i,0)+1
  if cur_side=="L": right_emanating_dict[cur_parent_i]=right_emanating_dict.get(cur_parent_i,0)+1
  #let's process links - count how many links between two consecutive words
  if cur_tag=='PUNCT' and c_i==len(conll0)-1: continue #ignore last token from counting links if it is punctuation 
  if cur_parent_i==0: continue 
  min_i,max_i=min(cur_i,cur_parent_i),max(cur_i,cur_parent_i)
  for tmp_i in range(min_i,max_i):
    link_count_dict[tmp_i]=link_count_dict.get(tmp_i,0)+1

boundary_i_list=[0]
for i0 in range(1,len(conll0)-1):
  i1=i0+1
  cur_link_count=link_count_dict.get(i0,0)
  #print(tok_dict[i0],tok_dict[i1], ">> n links:",cur_link_count)
  side1,side2=side_dict[i0],side_dict[i1]
  #print("sides:",side1,side2)
  #if (side1,side2)==("L","R") or cur_link_count==1: boundary_i_list.append(i0)
  if (side1,side2)!=("L","R"): continue
  if left_emanating_dict.get(i1,0)>0: continue
  if right_emanating_dict.get(i0,0)>0: continue
  boundary_i_list.append(i0)
  #if (side1,side2)==("L","R"): 
    
boundary_i_list.append(len(conll0))
for i0,i1 in zip(boundary_i_list,boundary_i_list[1:]):
  cur_slice=" ".join(token_list[i0:i1])
  print(i0,i1, cur_slice)
  #print(i0,i1)
  #print()
  #print(side_dict[i0],side_dict[i1])
  
  print("----")
# for c_i,c_line in conll0:
#   #print(a, side_dict[cur_i])
#   cur_i0=int(c_line[0])


0 20 At the same time , in line with the Secretary - General call for operating as ‘ One UN ’
----
20 27 and as part of DOS / OSCM
----
27 41 , the Centre is responding to demands to achieve system - wide efficiencies and
----
41 57 to deliver solutions to a broader client base that includes the wider United Nations system .
----


In [ ]:
from code_utils.parsing_lib import *
from code_utils.dep_lib import *

def conll2seg(conll0_input):
  link_count_dict={}
  side_dict={}
  tok_dict={}
  left_emanating_dict={}
  right_emanating_dict={}

  token_list=[v[1] for v in conll0_input]
  for c_i,c_line in enumerate(conll0_input):
    cur_token=c_line[1]
    cur_tag=c_line[4]
    cur_i=int(c_line[0])
    cur_parent_i=int(c_line[6]) 
    tok_dict[cur_i]=cur_token
    
    if cur_parent_i==0: cur_side="0"
    elif cur_parent_i>cur_i: cur_side="R"
    elif cur_parent_i<cur_i: cur_side="L"
    side_dict[cur_i]=cur_side
    if cur_side=="R": left_emanating_dict[cur_parent_i]=left_emanating_dict.get(cur_parent_i,0)+1
    if cur_side=="L": right_emanating_dict[cur_parent_i]=right_emanating_dict.get(cur_parent_i,0)+1
    #let's process links - count how many links between two consecutive words
    if cur_tag=='PUNCT' and c_i==len(conll0_input)-1: continue #ignore last token from counting links if it is punctuation 
    if cur_parent_i==0: continue 
    min_i,max_i=min(cur_i,cur_parent_i),max(cur_i,cur_parent_i)
    for tmp_i in range(min_i,max_i):
      link_count_dict[tmp_i]=link_count_dict.get(tmp_i,0)+1

  boundary_i_list=[0]
  for i0 in range(1,len(conll0_input)-1):
    i1=i0+1
    cur_link_count=link_count_dict.get(i0,0)
    #print(tok_dict[i0],tok_dict[i1], ">> n links:",cur_link_count)
    side1,side2=side_dict[i0],side_dict[i1]
    #print("sides:",side1,side2)
    #if (side1,side2)==("L","R") or cur_link_count==1: boundary_i_list.append(i0)
    if (side1,side2)!=("L","R"): continue
    if left_emanating_dict.get(i1,0)>0: continue
    if right_emanating_dict.get(i0,0)>0: continue
    boundary_i_list.append(i0)
    #if (side1,side2)==("L","R"): 
      
  boundary_i_list.append(len(conll0_input))
  all_slices=[]
  for i0,i1 in zip(boundary_i_list,boundary_i_list[1:]):
    cur_slice=" ".join(token_list[i0:i1])
    all_slices.append(cur_slice)
    #print(i0,i1, cur_slice)  
  return all_slices
sent1="""
This proposal is based on the vision of supply chain management that guides OSCM operations and the strengthening of core digital technology capacities to increase the Organization’s resilience and effectiveness and to contribute to the Secretary-General’s Data and New Technologies Strategies.
"""

conll0=get_conll(sent1.strip())
for c_line in conll0:
  print("\t".join([str(v) for v in c_line]))

segs=conll2seg(conll0)
for s0 in segs:
  print(s0) 

1	This	this	DT	DET	_	2	det	_	_
2	proposal	proposal	NN	NOUN	_	4	nsubjpass	_	_
3	is	be	VBZ	AUX	_	4	auxpass	_	_
4	based	base	VBN	VERB	_	0	ROOT	_	_
5	on	on	IN	ADP	_	4	prep	_	_
6	the	the	DT	DET	_	7	det	_	_
7	vision	vision	NN	NOUN	_	5	pobj	_	_
8	of	of	IN	ADP	_	7	prep	_	_
9	supply	supply	NN	NOUN	_	10	compound	_	_
10	chain	chain	NN	NOUN	_	11	compound	_	_
11	management	management	NN	NOUN	_	8	pobj	_	_
12	that	that	WDT	DET	_	13	nsubj	_	_
13	guides	guide	VBZ	VERB	_	11	relcl	_	_
14	OSCM	OSCM	NNP	PROPN	_	15	compound	_	_
15	operations	operation	NNS	NOUN	_	13	dobj	_	_
16	and	and	CC	CCONJ	_	7	cc	_	_
17	the	the	DT	DET	_	18	det	_	_
18	strengthening	strengthening	NN	NOUN	_	7	conj	_	_
19	of	of	IN	ADP	_	18	prep	_	_
20	core	core	JJ	ADJ	_	23	amod	_	_
21	digital	digital	JJ	ADJ	_	22	amod	_	_
22	technology	technology	NN	NOUN	_	23	compound	_	_
23	capacities	capacitie	VBZ	VERB	_	19	pobj	_	_
24	to	to	TO	PART	_	25	aux	_	_
25	increase	increase	VB	VERB	_	18	acl	_	_
26	the	the	DT	DET	_	27	det	_	_
27	Organization	Organi

In [ ]:
xls_path0='https://docs.google.com/spreadsheets/d/e/2PACX-1vSqipSUAaMWJrizvZtg1WIpds2pIiXtcZC6wjOLeVYie9p5O0lqiPnVdd8ambGKH8B0qJ4kmlVeEs6f/pub?output=xlsx'
cur_bitext=get_bitext(xls_path0,sheet_name0="current")
src_list=[v[0] for v in cur_bitext]
for s_i, sent0 in enumerate(src_list):
  print(s_i, "::",sent0)
  cur_conll0=get_conll(sent0)
  segs0=conll2seg(cur_conll0)
  for s_ in segs0:
    print(">",s_)
  print("---")


current
0 :: A/75/769xxx
> A/75/769xxx
---
1 :: A/76/xxx
> A/76 / xxx
---
2 :: United Nations
> United Nations
---
3 :: A/76/730
> A/76/730
---
4 :: General Assembly
> General Assembly
---
5 :: Distr.: General
> Distr .
> : General
---
6 :: 2022-03-01 00:00:00
> 2022 - 03 - 01 00:00:00
---
7 :: Original: English
> Original : English
---
8 :: f
> f
---
9 :: Seventy-sixth session
> Seventy - sixth session
---
10 :: Agenda item 150
> Agenda item 150
---
11 :: Administrative and budgetary aspects of the financing of the United Nations peacekeeping operations
> Administrative and budgetary aspects of the financing of the United Nations peacekeeping operations
---
12 :: Budget for the United Nations Logistics Base at Brindisi, Italy, for the period from 1 July 2022 to 30 June 2023
> Budget for the United Nations Logistics Base at Brindisi , Italy , for the period from 1 July 2022 to 30 June 2023
---
13 :: Report of the Secretary-General
> Report of the Secretary - General
---
14 :: Contents


#Testing New approach for hashing matching

In [ ]:
ar_corpus_fname="UNv1.0.ar-en-clean2.ar"
en_corpus_fname="UNv1.0.ar-en.en"

n_sents_max=200000
ar_fopen=open(ar_corpus_fname)
en_fopen=open(en_corpus_fname)

src_counter,trg_counter={},{}
src_sents,trg_sents=[],[]
for i,pair in enumerate(zip(en_fopen,ar_fopen)):
  if i>=n_sents_max: break
  if i%20000==0: print(i)
  en_line,ar_line=pair
  en_line,ar_line=en_line.strip("\n\r\t "),ar_line.strip("\n\r\t ")
  # src_sents.append(en_line)
  # trg_sents.append(ar_line)
  src_tokens=tok(en_line)
  trg_tokens=cur_ar_tok_fn(ar_line)
  src_sents.append(src_tokens)
  trg_sents.append(trg_tokens)
  for a in src_tokens:src_counter[a]=src_counter.get(a,0)+1
  for a in trg_tokens:trg_counter[a]=trg_counter.get(a,0)+1
  # print(en_tokens)
  # print(ar_tokens)
  # print("---")
print("len src tokens=",len(src_counter.keys()))
print("len trg tokens=",len(trg_counter.keys()))

0
20000
40000
60000
80000
100000
120000
140000
160000
180000
len src tokens= 48972
len trg tokens= 64447


In [ ]:
import gensim
print("working on src")
en_model=gensim.models.Word2Vec(
        src_sents,
        size=100,
        window=10,
        min_count=2,
        workers=10,
        iter=10)
print("working on trg")
ar_model=gensim.models.Word2Vec(
        trg_sents,
        size=100,
        window=10,
        min_count=2,
        workers=10,
        iter=10)

en_model.save("un_en_model_200k.wv")
ar_model.save("un_ar_model_200k.wv")

working on src
working on trg


In [ ]:
en_model.similar_by_word("court")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `similar_by_word` (Method will be removed in 4.0.0, use self.wv.similar_by_word() instead).
  """Entry point for launching an IPython kernel.


[('tribunal', 0.8088857531547546),
 ('commission', 0.7441569566726685),
 ('jurisdiction', 0.7220062613487244),
 ('prosecutor', 0.7023400664329529),
 ('courts', 0.6946955919265747),
 ('judgement', 0.6882531642913818),
 ('Court', 0.6809461116790771),
 ('offender', 0.6621584892272949),
 ('accused', 0.658098578453064),
 ('offence', 0.6564067006111145)]

In [ ]:
ar_corpus_fname="UNv1.0.ar-en-clean2.ar"
en_corpus_fname="UNv1.0.ar-en.en"

n_sents_max=100000
ar_fopen=open(ar_corpus_fname)
en_fopen=open(en_corpus_fname)

src_counter,trg_counter={},{}
src_sents,trg_sents=[],[]
src_fwd_index,trg_fwd_index=[],[]
for i,pair in enumerate(zip(en_fopen,ar_fopen)):
  if i>=n_sents_max: break
  if i%20000==0: print(i)
  en_line,ar_line=pair
  en_line,ar_line=en_line.strip("\n\r\t "),ar_line.strip("\n\r\t ")
  #src_tokens=tok(en_line)
  #trg_tokens=cur_ar_tok_fn(ar_line)
  src_sents.append(en_line)
  trg_sents.append(ar_line)


src_index_obj=indexing(src_sents,tok)
trg_index_obj=indexing(trg_sents,cur_ar_tok_fn)

0
20000
40000
60000
80000


In [ ]:
en_inv_dict=src_index_obj.inv_index
en_tok_sents=src_index_obj.all_tok_sentences
ar_inv_dict=trg_index_obj.inv_index
ar_tok_sents=trg_index_obj.all_tok_sentences


def get_ngrams(sent_tokens0,max_ngram_size0=3):
  all_sent_ngrams=[]
  for s_i in range(len(sent_tokens0)):
    for ngram_size in range(1,max_ngram_size0+1):
      if s_i+ngram_size>len(sent_tokens0): continue
      all_sent_ngrams.append(tuple(sent_tokens0[s_i:s_i+ngram_size]))
  return all_sent_ngrams
      #print(sent_tokens0[s_i:s_i+ngram_size])

#test_indexes=rtrv_idx(["of","the"],en_inv_dict)
all_data=[]
used_dict={}
sent_i=100
for sent_i in range(200):
  print("sentence:",sent_i)
  en_sent=en_tok_sents[sent_i]
  ar_sent=ar_tok_sents[sent_i]

  print("en_sent",en_sent)
  print("ar_sent",ar_sent)
  mini_en_wv_model={}
  mini_ar_wv_model={}
  for en_word in list(set(en_sent)):
    try: mini_en_wv_model[en_word]= en_model[en_word]
    except: pass
  for ar_word in list(set(ar_sent)):
    try: mini_ar_wv_model[ar_word]= ar_model[ar_word]
    except: pass



  src_ngrams=list(set(get_ngrams(en_sent)))
  trg_ngrams=list(set(get_ngrams(ar_sent)))
  src_ngrams_indexes_list=[]
  trg_ngrams_indexes_list=[]
  for sn in src_ngrams:
    cur_src_indexes=rtrv_idx(sn,en_inv_dict)
    cur_src_vec,_=get_words_vector(sn,mini_en_wv_model)
    src_ngrams_indexes_list.append((sn,cur_src_vec,cur_src_indexes))
  for tn in trg_ngrams:
    cur_trg_indexes=rtrv_idx(tn,ar_inv_dict)
    cur_trg_vec,_=get_words_vector(tn,mini_ar_wv_model)
    trg_ngrams_indexes_list.append((tn,cur_trg_vec,cur_trg_indexes))

  matching_list=[]
  for sn,sn_vec,sn_indexes in src_ngrams_indexes_list:
    for tn,tn_vec,tn_indexes in trg_ngrams_indexes_list:
      match_ratio=idx_match(sn_indexes,tn_indexes)
      #print(sn,tn,match_ratio)
      #combined_vec=sn_vec+tn_vec
      
      combined_vec=np.concatenate((sn_vec, tn_vec), axis=0)
      matching_list.append((sn,tn,combined_vec,match_ratio))
    
  matching_list.sort(key=lambda x:-x[-1])
  for ml in matching_list[:100]:
    sn,tn,combined_vec,match_ratio=ml
    if match_ratio==0: continue
    sn_tn_key=(sn,tn)
    if used_dict.get(sn_tn_key,False): continue
    used_dict[sn_tn_key]=True
    #print(sn,tn,match_ratio,combined_vec.shape)
    all_data.append((sn,tn,combined_vec,match_ratio))
  if len(all_data)>10000: break
  print("all_data", len(all_data))


# max_ngram=3
# for s_i in range(len(en_sent)):
#   for ngram_size in range(1,max_ngram+1):
#     if s_i+ngram_size>len(en_sent): continue
#     print(en_sent[s_i:s_i+ngram_size])
#print(en_tok_sents[100])

sentence: 0
en_sent ['ORIGINAL', ':', 'ENGLISH', '/', 'SPANISH']
ar_sent ['ORIGINAL', ':', 'ENGLISH', '/', 'SPANISH']


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:30: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:33: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


all_data 100
sentence: 1
en_sent ['INTERNATIONAL', 'LAW', 'COMMISSION']
ar_sent ['لجنة', 'ال_قانون', 'ال_دولي']
all_data 111
sentence: 2
en_sent ['Forty', '-', 'fifth', 'session']
ar_sent ['ال_دورة', 'ال_خامسة', 'وـ', 'ال_أربعون']
all_data 185
sentence: 3
en_sent ['3', 'May', '-', '23', 'July', '1993']
ar_sent ['3', 'أيار', '/', 'مايو', '-', '23', 'تموز', '/', 'يوليه', '1993']
all_data 285
sentence: 4
en_sent ['COMMENTS', 'OF', 'GOVERNMENTS', 'ON', 'THE', 'REPORT', 'OF', 'THE', 'WORKING', 'GROUP']
ar_sent ['تعليقات', 'ال_حكومات', 'على', 'تقرير', 'ال_فريق', 'ال_عامل', 'ال_معني']
all_data 307
sentence: 5
en_sent ['ON', 'THE', 'QUESTION', 'OF', 'AN', 'INTERNATIONAL', 'CRIMINAL', 'JURISDICTION']
ar_sent ['بـ', 'مسألة', 'إنشاء', 'قضاء', 'جنائي', 'دولي']
all_data 323
sentence: 6
en_sent ['CONTENTS']
ar_sent ['ال_محتويات']
all_data 324
sentence: 7
en_sent ['Page']
ar_sent ['ال_صفحة']
all_data 325
sentence: 8
en_sent ['INTRODUCTION', '.', '2']
ar_sent ['مقدمة', '.', '2']
all_data 345
sentence:

In [ ]:
all_data[0]
len(all_data)

361

In [ ]:
from random import shuffle
n_layers=2
n_hidden=128 #256 #64#64
LR=0.0000001
n_epochs=100
n_output=1
cur_matching_in_out=False

torch.manual_seed(1)
random.seed(1)

cur_data_item=all_data[0]
sn,tn,combined_vec,match_ratio=cur_data_item
n_input=combined_vec.shape[0]
rnn=RNN(n_input,n_hidden,n_output,n_layers,cur_matching_in_out)
loss_func = nn.MSELoss()
rnn = RNN(n_input, n_hidden, n_output,n_layers,matching_in_out=cur_matching_in_out).to(device)
optimizer = torch.optim.Adam(rnn.parameters(), lr=LR) 

shuffle(all_data)
train_data,test_data=split_train_test(all_data)
# def split_train_test(all_data0,train_ratio=0.8):
#   train_size=int(len(all_data0)*train_ratio)
#   train_set0,tes_set0=all_data0[:train_size],all_data0[train_size:]
#   return train_set0,tes_set0

for epoch_i in range(n_epochs):
  train_loss_items=[]
  test_loss_items=[]
  for cur_data_item in train_data:
    sn,tn,combined_vec,match_ratio=cur_data_item
    actual_out=[match_ratio]
    actual_out_tensor=torch.tensor(actual_out,dtype=torch.float32)
    input_tensor=torch.tensor(combined_vec,dtype=torch.float32)
    input_tensor=input_tensor.reshape(1,n_input)
    rnn_out=rnn(input_tensor)
    rnn_out_flat=rnn_out.ravel()
    rnn_out_val=rnn_out_flat.item()
    
    loss=loss_func(actual_out_tensor,rnn_out_flat)
    loss_val=loss.item()
    loss.backward()
    optimizer.step()  
    train_loss_items.append(loss.item())
  for data_i,cur_data_item in enumerate(test_data):
    sn,tn,combined_vec,match_ratio=cur_data_item
    actual_out=[match_ratio]
    actual_out_tensor=torch.tensor(actual_out,dtype=torch.float32)
    input_tensor=torch.tensor(combined_vec,dtype=torch.float32)
    input_tensor=input_tensor.reshape(1,n_input)
    rnn_out=rnn(input_tensor)
    rnn_out_flat=rnn_out.ravel()
    rnn_out_val=rnn_out_flat.item()
    
    loss=loss_func(actual_out_tensor,rnn_out_flat)
    loss_val=loss.item()
    test_loss_items.append(loss.item())
    #print("epoch_i", epoch_i, "rnn output:", round(rnn_out_val,4),"actual:",round(match_ratio,4),"loss value:",round(loss_val,4))
    if epoch_i>20 and data_i<20:
      print("epoch_i", epoch_i, "rnn output:", round(rnn_out_val,4),"actual:",round(match_ratio,4),sn,tn)
  train_loss_avg=sum(train_loss_items)/len(train_loss_items)
  test_loss_avg=sum(test_loss_items)/len(test_loss_items)
  print("epoch_i", epoch_i,"train_loss_avg",round(train_loss_avg,4),"test_loss_avg",round(test_loss_avg,4))
  print("==========")


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:45: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


epoch_i 0 train_loss_avg 0.1154 test_loss_avg 0.0845
epoch_i 1 train_loss_avg 0.0752 test_loss_avg 0.0621
epoch_i 2 train_loss_avg 0.0634 test_loss_avg 0.0629
epoch_i 3 train_loss_avg 0.0718 test_loss_avg 0.0785
epoch_i 4 train_loss_avg 0.0889 test_loss_avg 0.0935
epoch_i 5 train_loss_avg 0.0876 test_loss_avg 0.0758
epoch_i 6 train_loss_avg 0.0661 test_loss_avg 0.0568
epoch_i 7 train_loss_avg 0.0554 test_loss_avg 0.0537
epoch_i 8 train_loss_avg 0.0575 test_loss_avg 0.0595
epoch_i 9 train_loss_avg 0.0662 test_loss_avg 0.0659
epoch_i 10 train_loss_avg 0.0686 test_loss_avg 0.0616
epoch_i 11 train_loss_avg 0.0593 test_loss_avg 0.0524
epoch_i 12 train_loss_avg 0.0518 test_loss_avg 0.0488
epoch_i 13 train_loss_avg 0.0498 test_loss_avg 0.0502
epoch_i 14 train_loss_avg 0.0513 test_loss_avg 0.0531
epoch_i 15 train_loss_avg 0.0538 test_loss_avg 0.0553
epoch_i 16 train_loss_avg 0.0539 test_loss_avg 0.0549
epoch_i 17 train_loss_avg 0.0517 test_loss_avg 0.0504
epoch_i 18 train_loss_avg 0.0477 test_

KeyboardInterrupt: ignored

In [ ]:

start_i=300
n_sents=20
for sent_i in range(start_i,start_i+n_sents):
  print("sentence:",sent_i)
  en_sent=en_tok_sents[sent_i]
  ar_sent=ar_tok_sents[sent_i]

  print("en_sent",en_sent)
  print("ar_sent",ar_sent)
  mini_en_wv_model={}
  mini_ar_wv_model={}
  for en_word in list(set(en_sent)):
    try: mini_en_wv_model[en_word]= en_model[en_word]
    except: pass
  for ar_word in list(set(ar_sent)):
    try: mini_ar_wv_model[ar_word]= ar_model[ar_word]
    except: pass



  src_ngrams=list(set(get_ngrams(en_sent)))
  trg_ngrams=list(set(get_ngrams(ar_sent)))
  src_ngrams_indexes_list=[]
  trg_ngrams_indexes_list=[]
  for sn in src_ngrams:
    cur_src_indexes=rtrv_idx(sn,en_inv_dict)
    cur_src_vec,_=get_words_vector(sn,mini_en_wv_model)
    src_ngrams_indexes_list.append((sn,cur_src_vec,cur_src_indexes))
  for tn in trg_ngrams:
    cur_trg_indexes=rtrv_idx(tn,ar_inv_dict)
    cur_trg_vec,_=get_words_vector(tn,mini_ar_wv_model)
    trg_ngrams_indexes_list.append((tn,cur_trg_vec,cur_trg_indexes))

  matching_list=[]
  for sn,sn_vec,sn_indexes in src_ngrams_indexes_list:
    for tn,tn_vec,tn_indexes in trg_ngrams_indexes_list:
      match_ratio=idx_match(sn_indexes,tn_indexes)
      #print(sn,tn,match_ratio)
      #combined_vec=sn_vec+tn_vec
      if sn_vec==[] or tn_vec==[]: continue
      
      combined_vec=np.concatenate((sn_vec, tn_vec), axis=0)
      
      combined_vec_tensor=torch.tensor(combined_vec,dtype=torch.float32)
      combined_vec_tensor=combined_vec_tensor.reshape([1,combined_vec.shape[0]])
      rnn_out=rnn(combined_vec_tensor)
      rnn_out_val=rnn_out.ravel().item()

      matching_list.append((sn,tn,combined_vec,rnn_out_val,match_ratio))
  matching_list.sort(key=lambda x:-x[-2])
  for ml in matching_list[:100]:
    sn,tn,combined_vec,rnn_out_val,match_ratio=ml
    print(sn,tn,"rnn:", rnn_out_val,"match:", match_ratio)
  print("==========")

sentence: 300
en_sent ['3', '.', 'The', 'convention', 'establishing', 'the', 'court', 'should', 'grant', 'all', 'States', '(', 'including', ',', 'as', 'the', 'case', 'may', 'be', ',', 'also', 'non', '-', 'contracting', 'States', ')', 'the', 'right', 'to', 'designate', 'persons', 'having', 'the', 'qualifications', 'of', 'competence', 'and', 'impartiality', 'that', 'are', 'required', 'for', 'the', 'participation', 'in', 'the', 'judgement', 'organs', 'appointed', 'for', 'the', 'various', 'cases', 'to', 'be', 'examined', '.']
ar_sent ['3', '-', 'وـ', 'يجب', 'في', 'ال_اتفاقية', 'التي', 'تقضي', 'بـ', 'إنشاء', 'ال_محكمة', 'أن', 'تمنح', 'جميع', 'ال_دول', ')', 'وـ', 'من', 'ـها', 'أيضا', 'ال_دول', 'ال_متعاقدة', '،', 'حسب', 'طبيعة', 'ال_حالة', '(', 'ال_حق', 'في', 'تسمية', 'أشخاص', 'يملكون', 'مؤهلات', 'ال_اختصاص', 'وـ', 'ال_نزاهة', 'ال_مطلوبة', 'لـ', 'ال_اشتراك', 'في', 'هيئات', 'إصدار', 'ال_حكم', 'ال_معينة', 'من', 'أجل', 'مختلف', 'ال_قضايا', 'ال_منظورة', '.']


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  del sys.path[0]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  app.launch_new_instance()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:40: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:45: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


('judgement', 'organs', 'appointed') ('ال_حالة', '(', 'ال_حق') rnn: 0.5478397011756897 match: 0
('3',) ('ال_حالة', '(', 'ال_حق') rnn: 0.5252849459648132 match: 0
('organs', 'appointed') ('ال_حالة', '(', 'ال_حق') rnn: 0.5184096693992615 match: 0
('impartiality',) ('(', 'ال_حق') rnn: 0.505550742149353 match: 0
('organs', 'appointed') ('(', 'ال_حق') rnn: 0.5028063654899597 match: 0
('judgement', 'organs', 'appointed') ('(', 'ال_حق') rnn: 0.5015252828598022 match: 0
('appointed',) ('(', 'ال_حق') rnn: 0.4960797131061554 match: 0
('designate',) ('(', 'ال_حق') rnn: 0.4955708682537079 match: 0
('3',) ('(', 'ال_حق', 'في') rnn: 0.4935663044452667 match: 0
('designate',) ('ال_حالة', '(') rnn: 0.4892522096633911 match: 0
('3',) ('ال_حق',) rnn: 0.48894166946411133 match: 0.004741702021462441
('qualifications',) ('ال_حالة', '(', 'ال_حق') rnn: 0.4888571500778198 match: 0
('judgement', 'organs', 'appointed') ('ال_نزاهة',) rnn: 0.48518431186676025 match: 0
('designate',) ('ال_حق',) rnn: 0.4821773767471

In [ ]:
tn_vec

[]

In [ ]:
n_input

200

In [ ]:
src_keys=list(src_counter.keys())
print(src_keys[100:110])
trg_keys=list(trg_counter.keys())
print(trg_keys[100:110])

['Code', 'Crimes', 'against', 'Peace', 'Security', 'Mankind', 'which', 'was', 'devoted', 'to']
['ال_جرائم', 'ال_مخلة', 'سلم', 'ال_إنسانية', 'أمن', 'ـها', 'الذي', 'كرس', 'لـ', 'إمكانية']


In [ ]:
from scipy import spatial
# for a in trg_keys[:10]:
#   vec=ar_model[a]
  # src_sents.append(src_tokens)
  # trg_sents.append(trg_tokens)
for src0,trg0 in zip(src_sents[:20],trg_sents[:20]):
  tmp_src_vec_dict,tmp_trg_vec_dict={},{}
  try: 
    for a in src0: tmp_src_vec_dict[a]=en_model[a]
  except: pass
  try: 
    for a in trg0: tmp_trg_vec_dict[a]=ar_model[a]
  except: pass
  for a0,b0 in tmp_src_vec_dict.items():
    tmp=[]
    print(a0)
    for a1,b1 in tmp_trg_vec_dict.items():
      
      cos_val = abs( 1 - spatial.distance.cosine(b0, b1))
      tmp.append((a1,cos_val))
      #print(a0,a1,cos_val)
    tmp.sort(key=lambda x:-x[1])
    for x in tmp[:5]:
      print(x)
    print("----")



ORIGINAL
('/', 0.1210264042019844)
(':', 0.09776163846254349)
('ENGLISH', 0.09312044829130173)
('SPANISH', 0.07970868051052094)
('ORIGINAL', 0.07445640116930008)
----
:
('/', 0.1281573325395584)
(':', 0.10443489998579025)
('ENGLISH', 0.09897531569004059)
('SPANISH', 0.08429171144962311)
('ORIGINAL', 0.07810582965612411)
----
ENGLISH
('/', 0.12239494174718857)
(':', 0.1018538549542427)
('ENGLISH', 0.09902690351009369)
('SPANISH', 0.08270221203565598)
('ORIGINAL', 0.07603658735752106)
----
/
('/', 0.1405307948589325)
(':', 0.1113162711262703)
('ENGLISH', 0.10568653047084808)
('SPANISH', 0.09063097089529037)
('ORIGINAL', 0.08553128689527512)
----
SPANISH
('/', 0.10009922087192535)
(':', 0.07723880559206009)
('SPANISH', 0.07304131239652634)
('ENGLISH', 0.07231893390417099)
('ORIGINAL', 0.05214103311300278)
----
INTERNATIONAL
('ال_قانون', 0.06248722970485687)
('ال_دولي', 0.05865810811519623)
('لجنة', 0.04715884476900101)
----
LAW
('ال_قانون', 0.059738751500844955)
('ال_دولي', 0.056236479431

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  if sys.path[0] == '':


In [ ]:
test_en=en_model["general"]
keys=list(test_en.keys())
print(len(keys),keys[:10])
#test_ar=ar_model["العامة"]

In [ ]:
en_model["the"]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


array([ 1.6295501 , -0.3641678 , -0.8005455 , -0.5281693 ,  1.0934305 ,
        0.8150351 ,  1.8593708 , -0.7521438 ,  0.4360393 , -0.5191244 ,
       -0.49931085,  1.0282642 ,  1.3791842 , -0.8435117 ,  0.40540653,
       -1.3977058 ,  0.72926307, -0.88094467, -0.7700975 , -0.29425058,
       -1.2420062 ,  1.053161  ,  0.25826624,  1.2520448 ,  1.8816774 ,
        0.00995926, -0.9422078 , -0.30660504, -1.1771382 ,  0.21316102,
       -1.1989622 ,  1.0406231 ,  1.3392975 , -0.5352807 ,  1.3703241 ,
       -0.0862059 , -0.8194424 , -2.6602635 ,  0.07741913,  1.3939295 ,
        1.1210356 , -0.7005276 , -2.4378896 ,  1.5436195 ,  1.4134588 ,
       -1.7137322 , -0.4230278 ,  1.8795353 , -1.2041483 ,  0.8313264 ,
        0.58736694,  0.07563703, -1.5684607 ,  2.3167934 ,  0.34603527,
        0.10522225,  0.5311361 , -0.02403292, -0.11283378,  2.7475295 ,
       -0.05356972, -0.05729892, -1.7050958 ,  1.0823265 ,  0.12822641,
       -1.9642165 , -0.4824889 , -1.7970736 ,  0.19963118, -0.90

In [ ]:
from itertools import groupby
def hash_tok(token0,vec_size=200):
  hash_val=abs(hash(token0))
  remainder=hash_val%vec_size
  return remainder
cur_vec_size=1000
lr=0.00001
max_val=20000
src_vec_dict={}
trg_vec_dict={}
src_counter,trg_counter={},{}

counter=0
for src0,trg0 in zip(src_sents,trg_sents):
  if counter>max_val: break
  if counter%10==0: print(counter)
  counter+=1  
  src_lower=[v.lower() for v in src0]
  trg_lower=[v.lower() for v in trg0]
  src_hashes=[hash_tok(v,cur_vec_size) for v in src_lower]
  trg_hashes=[hash_tok(v,cur_vec_size) for v in trg_lower]
  for s0 in list(set(src_lower)):
    tmp_vec=src_vec_dict.get(s0,[0]*cur_vec_size)
    new_vec=[]
    for i0,cell0 in enumerate(tmp_vec):
      cur_offset=lr#/len(trg_lower)
      if i0 in trg_hashes: cell0+=cur_offset
      else: cell0-=cur_offset
      new_vec.append(cell0)
    src_vec_dict[s0]=new_vec
  for t0 in list(set(trg_lower)):
    tmp_vec=trg_vec_dict.get(t0,[0]*cur_vec_size)
    new_vec=[]
    for i0,cell0 in enumerate(tmp_vec):
      cur_offset=lr#/len(src_lower)
      if i0 in src_hashes: cell0+=cur_offset
      else: cell0-=cur_offset
      new_vec.append(cell0)
    trg_vec_dict[t0]=new_vec

print("finished training")
counter=0
for src0,trg0 in zip(src_sents,trg_sents):
  if counter>200: break
  counter+=1

  src_lower=[v.lower() for v in src0]
  trg_lower=[v.lower() for v in trg0]
  src_hashes=[(v,hash_tok(v,cur_vec_size)) for v in src_lower]
  trg_hashes=[(v,hash_tok(v,cur_vec_size)) for v in trg_lower]
  for s0,s_hash0 in list(set(src_hashes)):
    tmp_vec=src_vec_dict.get(s0,[0]*cur_vec_size)
    print(s0)
    # print(tmp_vec)
    candidates=[]
    for t_tok0,t_hash0 in list(set(trg_hashes)):
      
      cur_val=tmp_vec[t_hash0]
      other_way_vec=trg_vec_dict[t_tok0]
      other_way_val=other_way_vec[s_hash0]
      combined=cur_val+other_way_val
      #candidates.append((t_tok0,cur_val,other_way_val,combined))
      candidates.append((t_tok0,round(combined,5)))
      #print(t_tok0,t_hash0, tmp_vec[t_hash0])
    candidates.sort(key=lambda x:x[-1],reverse=True)
    for can0 in candidates[:5]:
      print(can0)
    print("---")
  print("====")


  # src_hashes,trg_hashes=[],[]
  # for t0 in trg0:
  #   cur_hash=hash_tok(t0.lower(),cur_vec_size)
  #   trg_hashes.append(cur_hash)
  # for s0 in src0:
  #   cur_hash=hash_tok(s0.lower(),cur_vec_size)
  #   src_hashes.append(cur_hash)    
  # for s0 in src0:
  #   s_tok0=s0.lower()
  #   src_counter[s_tok0]=src_counter.get(s_tok0,0)+1
  #   tmp_vec=src_vec_dict.get(s_tok0,[0]*cur_vec_size)
  #   for th0 in trg_hashes: tmp_vec[th0]+=1/len(trg_hashes)
  #   src_vec_dict[s_tok0]=tmp_vec
  # for t0 in trg0:
  #   t_tok0=t0.lower()
  #   trg_counter[t_tok0]=trg_counter.get(t_tok0,0)+1
  #   tmp_vec=trg_vec_dict.get(t_tok0,[0]*cur_vec_size)
  #   for sh0 in src_hashes: tmp_vec[sh0]+=1/len(src_hashes) #0.0001
  #   trg_vec_dict[t_tok0]=tmp_vec

  #print("------")

# for a,b in src_vec_dict.items():
#   print(a,b)

# counter=0
# for src0,trg0 in zip(src_sents,trg_sents):
#   if counter>50: break
#   counter+=1
#   print(src0)
#   print(trg0)
#   src_hashes,trg_hashes=[],[]
#   tmp_trg_hash_dict={}
#   for t0 in trg0:
#     t_tok0=t0.lower()
#     cur_hash=hash_tok(t_tok0,cur_vec_size)
#     trg_hashes.append((t_tok0,cur_hash))
#     tmp_trg_hash_dict[t_tok0]=trg_vec_dict.get(t_tok0,[0]*cur_vec_size)

#   for s0 in src0:
#     s_tok0=s0.lower()
#     cur_hash=hash_tok(s_tok0,cur_vec_size)
#     src_hashes.append((s_tok0,cur_hash)) 
  
#   for s0 in src0:
#     s_tok0=s0.lower()
#     cur_hash=hash_tok(s_tok0,cur_vec_size)
#     cur_src_vec=src_vec_dict.get(s_tok0,[0]*cur_vec_size)
#     cur_src_vec_sum=sum(cur_src_vec)
#     cur_src_tok_count=src_counter.get(s_tok0,0)
#     print(s0)
#     candidates=[]
#     for t0_,h_hash0_ in trg_hashes:
#       t_val=cur_src_vec[h_hash0_] #round(cur_src_vec[h_hash0_],4)
#       cur_trg_tok_count=trg_counter.get(t0_,0)
#       t_val_norm=round(t_val/cur_src_vec_sum,4)
#       #t_val_norm=round(t_val/cur_trg_tok_count)

#       other_way_vec=trg_vec_dict[t0_]
#       other_way_vec_sum=sum(other_way_vec)
#       other_way_val=other_way_vec[cur_hash]#round(,4)
#       other_way_val_norm=round(other_way_val/other_way_vec_sum,4)
#       #other_way_val_norm=round(other_way_val/cur_src_tok_count)
      
#       t_val_norm=t_val
#       other_way_val_norm=other_way_val
#       #combined=round(t_val+other_way,4)
#       combined=round(t_val_norm+other_way_val_norm,6)
#       #print(t0_,combined)
#       candidates.append((t0_,combined))
#       # print(t_val,other_way)
#       # print("----")
#       #print(t0_,h_hash0_,t_val,other_way)
#     # for th0 in trg_hashes: tmp_vec[th0]+=0.0001
#     # src_vec_dict[s_tok0]=tmp_vec
#     candidates.sort(key=lambda x:-x[1])
#     for can0 in candidates[:5]:
#       print(can0)
#     print("========")
     


# collision_list=[]
# for src_toks in src_sents:
#   for a in src_toks:
#     cur_pair=(a.lower(), hash_tok(a.lower()))
#     if cur_pair in collision_list: continue
#     collision_list.append(cur_pair)
#   #print("--------")
# collision_list.sort(key=lambda x:x[1])
# for cl in collision_list:
#   print(cl)

Streaming output truncated to the last 5000 lines.
a
('.', -0.01105)
('في', -0.01401)
('بـ', -0.01879)
('فـ', -0.03486)
('حكما', -0.03676)
---
will
('سـ', 0.00172)
('ال_اختصاص', -0.00721)
('حكما', -0.00732)
('أحكاما', -0.00738)
('يتطلب', -0.00755)
---
international
('ال_دولية', 0.00138)
('حكما', -0.01019)
('ال_اختصاص', -0.01052)
('أحكاما', -0.01053)
('ال_جرائم', -0.01067)
---
which
('حكما', -0.01033)
('أحكاما', -0.01113)
('ال_اختصاص', -0.0113)
('يتطلب', -0.01192)
('ال_جرائم', -0.01197)
---
or
('أو', 0.01148)
('حكما', -0.00481)
('أحكاما', -0.00627)
('ال_اختصاص', -0.00654)
('إسناد', -0.00665)
---
jurisdiction
('ال_اختصاص', -0.00026)
('ال_جرائم', -0.00099)
('حكما', -0.00117)
('إسناد', -0.00127)
('أحكاما', -0.00141)
---
statute
('ال_أساسي', 0.00128)
('ال_نظام', -0.00069)
('حكما', -0.00088)
('ال_اختصاص', -0.00099)
('إسناد', -0.00108)
---
over
('أحكاما', -0.00154)
('حكما', -0.00156)
('إسناد', -0.00174)
('ال_اختصاص', -0.00181)
('يتطلب', -0.00203)
---
provisions
('أحكاما', -0.00101)
('حكما', -

In [ ]:
counter=0
for src0,trg0 in zip(src_sents,trg_sents):
  if counter>500: break
  counter+=1

  src_lower=[v.lower() for v in src0]
  trg_lower=[v.lower() for v in trg0]
  src_hashes=[(v,hash_tok(v,cur_vec_size)) for v in src_lower]
  trg_hashes=[(v,hash_tok(v,cur_vec_size)) for v in trg_lower]
  for s0,s_hash0 in list(set(src_hashes)):
    tmp_vec=src_vec_dict.get(s0,[0]*cur_vec_size)
    print(s0)
    # print(tmp_vec)
    candidates=[]
    for t_tok0,t_hash0 in list(set(trg_hashes)):
      cur_val=tmp_vec[t_hash0]
      other_way_vec=trg_vec_dict[t_tok0]
      other_way_val=other_way_vec[s_hash0]
      combined=cur_val+other_way_val
      if combined<0: continue
      #candidates.append((t_tok0,cur_val,other_way_val,combined))
      candidates.append((t_tok0,round(combined,5)))
      #print(t_tok0,t_hash0, tmp_vec[t_hash0])
    candidates.sort(key=lambda x:x[-1],reverse=True)
    for can0 in candidates[:5]:
      print(can0)
    print("---")
  print("====")

Streaming output truncated to the last 5000 lines.
go
---
11
---
court
('محكمة', 0.00023)
---
====
jurisidiction
('ال_ضيق', 0.0)
---
,
('.', 0.09434)
('لـ', 0.04235)
('من', 0.01974)
('أن', 0.00747)
---
all
('جميع', 0.00691)
('يوسع', 0.00078)
---
however
('يوسع', 0.00162)
('بيد', 0.00056)
---
to
('.', 0.04704)
('لـ', 0.02905)
('أن', 0.01619)
('من', 0.01318)
('يوسع', 0.00191)
---
"
('"', 0.00924)
('يوسع', 0.00032)
---
crimes
('ال_جرائم', 0.00059)
---
that
('أن', 0.02863)
('يوسع', 0.01893)
---
expands
('ال_ضيق', 0.0)
---
limited
('يوسع', 1e-05)
---
of
('.', 0.12139)
('لـ', 0.06716)
('من', 0.02661)
('أن', 0.00412)
---
an
---
.
('.', 0.18957)
('لـ', 0.06316)
('من', 0.01605)
---
report
---
character
---
the
('.', 0.15209)
('لـ', 0.0811)
('من', 0.03579)
('أن', 0.01704)
---
international
('ال_دولي', 0.00171)
---
====
would
---
such
---
before
---
how
---
trafficking
('ال_اتجار', 0.00042)
---
(
(')', 0.07117)
('(', 0.07093)
('تلك', 0.03194)
---
same
---
the
('.', 0.15209)
('وـ', 0.13902)
('لـ',

In [ ]:
span1,span2=(6, 6),(5, 39)
span1,span2=(6, 6),(7, 2)

def check_span_overlap(span1,span2):
  span1_x1,span1_x2=span1
  span2_x1,span2_x2=span2
  d1=abs(span1_x2-span1_x1)
  d2=abs(span2_x2-span2_x1)
  max_x=max(span1_x1,span1_x2,span2_x1,span2_x2)
  min_x=min(span1_x1,span1_x2,span2_x1,span2_x2)
  d_total=max_x-min_x
  overlapping=False
  if d_total<=d1+d2: overlapping=True
  return overlapping
cur_span1,cur_span2=(3, 5),(8, 12)
cur_span1,cur_span2=(3, 9),(8, 12)
cur_overlapping=check_span_overlap(cur_span1,cur_span2)
# print("d1",d1)
# print("d2",d2)
# print("d_total",d_total)
print("overlapping",cur_overlapping)

overlapping True


#Testing filtering

In [ ]:
params={}
params["lang"]="en"
params["ignore_punc"]=False
params["stop_words"]=[]
params["ignore_ar_pre_suf"]=True
params["remove_al"]=False
params["index_words"]=True
params["lower"]=False
params["stemming"]=False

def filter_tokens(token_list0, params0={}):
#def filter_tokens(token_list0, lang="en",stop_words=[], ignore_punc=False,ignore_ar_pre_suf=False,remove_al=True,index_words=True,lower=True,stemming=False):
  tokens_original=list(token_list0)
  tokens_copy_enum=[(i,v) for i,v in enumerate(token_list0)]   
  ignore_punc=params0.get("ignore_punc",False)
  ignore_ar_pre_suf=params0.get("ignore_ar_pre_suf",False)
  remove_al=params0.get("remove_al",True)
  lower=params0.get("lower",True)
  stemming=params0.get("stemming",False)
  lang=params0.get("lang","en")
  stop_words=params0.get("stop_words",[])
  #if lang=="ar": tokens=tok_ar(tokens,count_dict) #tok_ar(tokens)
  if ignore_punc: tokens_copy_enum=[(i,v) for i,v in tokens_copy_enum if not is_punct(v)]
  if ignore_ar_pre_suf: tokens_copy_enum=[(i,v) for i,v in tokens_copy_enum if not v.startswith("ـ") and not v.endswith("ـ")]
  if stop_words: tokens_copy_enum=[(i,v) for i,v in tokens_copy_enum if not v.lower() in stop_words]
  if remove_al: tokens_copy_enum=[(i,v.replace("ال_","")) for i,v in tokens_copy_enum if not v.lower() in stop_words]
  tokens_out=[v[1] for v in tokens_copy_enum]
  if lower: tokens_out=[v.lower() for v in tokens_out]
  cur_mapping=[v[0] for v in tokens_copy_enum]
  return tokens_out,cur_mapping, tokens_original

#cur_ar_tok_fn(text)  
ar_corpus_fname="UNv1.0.ar-en-clean2.ar"
en_corpus_fname="UNv1.0.ar-en.en"
ar_fopen=open(ar_corpus_fname)
en_fopen=open(en_corpus_fname)
src_sents,trg_sents=[],[]
for pair_i,pair in enumerate(zip(en_fopen,ar_fopen)):
  src_line,trg_line=pair
  src_line,trg_line=src_line.strip("\n\r\t "),trg_line.strip("\n\r\t ")
  src_sents.append(src_line)
  trg_sents.append(trg_line)
  if pair_i>200: break

ar_fopen.close()
en_fopen.close()
sent_i=50
cur_sent=trg_sents[sent_i]
cur_sent=src_sents[sent_i]
cur_tokens=cur_ar_tok_fn(cur_sent)
filtered0,mapping0,original0=filter_tokens(cur_tokens, params)
print(cur_sent)
print(cur_tokens)
print(filtered0)


8. In paragraphs 442 and 443 of the report, the Working Group sets out one possible arrangement for nominating judges to serve on a court and constituting it when required.
['8', '.', 'In', 'paragraphs', '442', 'and', '443', 'of', 'the', 'report', ',', 'the', 'Working', 'Group', 'sets', 'out', 'one', 'possible', 'arrangement', 'for', 'nominating', 'judges', 'to', 'serve', 'on', 'a', 'court', 'and', 'constituting', 'it', 'when', 'required', '.']
['8', '.', 'In', 'paragraphs', '442', 'and', '443', 'of', 'the', 'report', ',', 'the', 'Working', 'Group', 'sets', 'out', 'one', 'possible', 'arrangement', 'for', 'nominating', 'judges', 'to', 'serve', 'on', 'a', 'court', 'and', 'constituting', 'it', 'when', 'required', '.']


In [ ]:
!python align_utils.py

{'lang': 'en', 'ignore_punc': True, 'stop_words': [], 'ignore_ar_pre_suf': True, 'remove_al': True, 'index_words': True, 'lower': True, 'stemming': False}


#Run experiments with pypy

In [ ]:
!chmod 777 run_walign.py
!chmod 777 ./pypy3.9-v7.3.9-linux64/bin/pypy
#!./pypy3.9-v7.3.9-linux64/bin/pypy run_walign.py exp04 250000
!./pypy3.9-v7.3.9-linux64/bin/pypy run_walign.py exp01-test 100000
#!pypy run_walign.py

loaded necessary functions
argv ['run_walign.py', 'exp01-test', '100000']
exp_dir exp01-test batch_size 100000
batch_size 100000
0
10000
20000
30000
40000
50000
60000
70000
80000
90000
batch already processed, moving on to next batch
110000
120000
130000
140000
150000
160000
170000
180000
190000
indexing src sentences
indexing trg sentences
finished indexing, elapsed: 119.3
now aligning - number of sentences: 199999
now aligning 129250 elapsed: 60.9
now aligning 129300 elapsed: 83.8
now aligning 129350 elapsed: 82.9
now aligning 129400 elapsed: 49.7
now aligning 129450 elapsed: 128.8
now aligning 129500 elapsed: 152.7
now aligning 129550 elapsed: 148.0
now aligning 129600 elapsed: 30.8
couldn't find full el: ((0, 10), (0, 17)) used the top el instead ((0, 8), (0, 8))
now aligning 129650 elapsed: 47.6
now aligning 129700 elapsed: 70.6
now aligning 129750 elapsed: 56.1
couldn't find full el: ((0, 22), (0, 32)) used the top el instead ((1, 22), (12, 32))
now aligning 129800 elapsed: 76.2


In [ ]:
!apt-get install pypy

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  pypy-lib
Suggested packages:
  pypy-doc pypy-tk
The following NEW packages will be installed:
  pypy pypy-lib
0 upgraded, 2 newly installed, 0 to remove and 49 not upgraded.
Need to get 13.1 MB of archives.
After this operation, 84.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 pypy-lib amd64 5.10.0+dfsg-3build2 [2,303 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 pypy amd64 5.10.0+dfsg-3build2 [10.8 MB]
Fetched 13.1 MB in 3s (4,584 kB/s)
Selecting previously unselected package pypy-lib:amd64.
(Reading database ... 155639 files and directories currently installed.)
Preparing to unpack .../pypy-lib_5.10.0+dfsg-3build2_amd64.deb ..

In [ ]:
!wget https://downloads.python.org/pypy/pypy3.9-v7.3.9-linux64.tar.bz2
!tar xf pypy3.9-v7.3.9-linux64.tar.bz2

--2022-06-30 12:44:04--  https://downloads.python.org/pypy/pypy3.9-v7.3.9-linux64.tar.bz2
Resolving downloads.python.org (downloads.python.org)... 151.101.0.175, 151.101.64.175, 151.101.128.175, ...
Connecting to downloads.python.org (downloads.python.org)|151.101.0.175|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 32841328 (31M) [application/x-tar]
Saving to: ‘pypy3.9-v7.3.9-linux64.tar.bz2.3’

pypy3.9-v7.3.9-linu 100%[===================>]  31.32M  40.2MB/s    in 0.8s    

2022-06-30 12:44:04 (40.2 MB/s) - ‘pypy3.9-v7.3.9-linux64.tar.bz2.3’ saved [32841328/32841328]



In [ ]:
!./pypy3.9-v7.3.9-linux64/bin/pypy align_utils.py

{'lang': 'en', 'ignore_punc': True, 'stop_words': [], 'ignore_ar_pre_suf': True, 'remove_al': True, 'index_words': True, 'lower': True, 'stemming': False}


In [ ]:
!python run_walign.py

loaded necessary functions
0
10000
20000
30000
40000
50000
60000
70000
80000
90000
indexing src sentences
indexing trg sentences
finished indexing, elapsed: 76.7
now aligning - number of sentences: 100000
now aligning 0 elapsed: 0.0
now aligning 10 elapsed: 0.0
now aligning 20 elapsed: 0.0
now aligning 30 elapsed: 0.0
sent_i 36
sent_i 37
sent_i 38
sent_i 39
now aligning 40 elapsed: 2.0
sent_i 40
sent_i 41
sent_i 42
sent_i 43
sent_i 44
sent_i 45
sent_i 46
sent_i 47
sent_i 48
sent_i 49
now aligning 50 elapsed: 39.8
sent_i 50
sent_i 51
sent_i 52
sent_i 53
sent_i 54
sent_i 55
sent_i 56
sent_i 57
sent_i 58
sent_i 59
now aligning 60 elapsed: 5.4
sent_i 60
sent_i 61
sent_i 62
sent_i 63
sent_i 64
sent_i 65
sent_i 66
sent_i 67
sent_i 68
sent_i 69
now aligning 70 elapsed: 7.1
sent_i 70
sent_i 71
sent_i 72
sent_i 73
sent_i 74
sent_i 75
sent_i 76
sent_i 77
sent_i 78
sent_i 79
now aligning 80 elapsed: 11.7
sent_i 80
sent_i 81
sent_i 82
sent_i 83
sent_i 84
sent_i 85
sent_i 86
sent_i 87
sent_i 88
sen

In [ ]:
from collections import Counter

list1=["a","b","a","c"]
c1=Counter(list1)

#Index everything - Sep 2022

In [ ]:
ar_corpus_fname="UNv1.0.ar-en-clean2.ar"
en_corpus_fname="UNv1.0.ar-en.en"

n_sents_max=200000
ar_fopen=open(ar_corpus_fname)
en_fopen=open(en_corpus_fname)

src_counter,trg_counter={},{}
src_sents,trg_sents=[],[]
for i,pair in enumerate(zip(en_fopen,ar_fopen)):
  #if i>=n_sents_max: break
  if i%20000==0: print(i)
  en_line,ar_line=pair
  en_line,ar_line=en_line.strip("\n\r\t "),ar_line.strip("\n\r\t ")
  src_tokens=tok(en_line)
  trg_tokens=cur_ar_tok_fn(ar_line)


0
20000
40000
60000
80000
100000


KeyboardInterrupt: ignored

In [ ]:
!wc UNv1.0.ar-en-clean2.ar

  18539207  397660045 4458094300 UNv1.0.ar-en-clean2.ar


#Combining into bitexts - Sep 2022

In [ ]:

ar_corpus_fname="UNv1.0.ar-en-clean2.ar"
en_corpus_fname="UNv1.0.ar-en.en"
en_ar_corpus_combined_fname="UNv1.0.en-ar-combined3.txt"
en_ar_corpus_combined_fpath=os.path.join(cwd,en_ar_corpus_combined_fname)

ar_fopen=open(ar_corpus_fname)
en_fopen=open(en_corpus_fname)
combined_fopen=open(en_ar_corpus_combined_fname,"w")

n_sents_max=200000

for i,pair in enumerate(zip(en_fopen,ar_fopen)):
  if i>=n_sents_max: break
  if i%20000==0: print(i)
  en_line,ar_line=pair
  en_line,ar_line=en_line.strip("\n\r\t "),ar_line.strip("\n\r\t ")
  en_line,ar_line=en_line.replace("\t"," "),ar_line.replace("\t"," ")
  combined_line="%s\t%s\n"%(en_line,ar_line)
  combined_fopen.write(combined_line)

ar_fopen.close()
en_fopen.close()
combined_fopen.close()

0
20000
40000
60000
80000
100000
120000
140000
160000
180000


#Going full-scale - Sept 2022

In [ ]:
#preparing index shelves
import shelve
en_ar_corpus_combined_fname="UNv1.0.en-ar-combined.txt"


fopen=open(en_ar_corpus_combined_fname)
cur_loc=0
cur_line="-"
counter=0
src_fwd_index0,trg_fwd_index0=[],[]
while cur_line!="":
#for i0,f0 in enumerate(fopen):
#for i0 in range(10):  
  if counter>1000000: break
  if counter%10000==0: print(counter)
  cur_loc=fopen.tell()
  f0=fopen.readline()
  cur_line=f0.strip("\n\r\t")
  line_split=cur_line.split("\t")
  if len(line_split)!=2: continue
  src0,trg0=line_split
  src_tokens0,trg_tokens0=tok(src0),tok(trg0)
  cur_params={}
  cur_params["excluded_words"]=excluded_src_tokens+excluded_trg_tokens
  #print(fopen.tell())
  filtere_src0=filter_toks(src_tokens0,cur_params)
  filtere_trg0=filter_toks(trg_tokens0,cur_params)

  indexed_src0=[(v,cur_loc+0.001*vi) for vi,v in enumerate(filtere_src0) if vi<1000]
  indexed_trg0=[(v,cur_loc+0.001*vi) for vi,v in enumerate(filtere_trg0) if vi<1000]
  src_fwd_index0.extend(indexed_src0)
  trg_fwd_index0.extend(indexed_trg0)
  #indexed_trg0=filter_toks(trg_tokens0,cur_params)

  # print("cur_loc",cur_loc)
  # print(filtere_src0)
  # print(filtere_trg0)
  # print("indexed_src0",indexed_src0)
  # print("indexed_trg0",indexed_trg0)
  counter+=1
  #cur_loc+=len(f0)
fopen.close()
src_fwd_index0.sort()
trg_fwd_index0.sort()
grouped_src=[(key,[v[1] for v in list(group)]) for key,group in groupby(src_fwd_index0,lambda x:x[0])]
grouped_trg=[(key,[v[1] for v in list(group)]) for key,group in groupby(trg_fwd_index0,lambda x:x[0])]

src_shelve_fpath="test-src.shelve"
src_shelve_fopen=shelve.open(src_shelve_fpath)
for k0,grp0 in grouped_src: src_shelve_fopen[k0]=grp0
src_shelve_fopen.close()
trg_shelve_fpath="test-trg.shelve"
trg_shelve_fopen=shelve.open(trg_shelve_fpath)
for k0,grp0 in grouped_trg: trg_shelve_fopen[k0]=grp0
trg_shelve_fopen.close()


print("src_fwd_index0",len(src_fwd_index0))
print("trg_fwd_index0",len(trg_fwd_index0))
print("grouped_src",len(grouped_src))
print("grouped_trg",len(grouped_trg))
# for a in src_fwd_index0[-50:]:
#   print(a)

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000
410000
420000
430000
440000
450000
460000
470000
480000
490000
500000
510000
520000
530000
540000
550000
560000
570000
580000
590000
600000
610000
620000
630000
640000
650000
660000
670000
680000
690000
700000
710000
720000
730000
740000
750000
760000
770000
780000
790000
800000
810000
820000
830000
840000
850000
860000
870000
880000
890000
900000
910000
920000
930000
940000
950000
960000
970000
980000
990000
1000000
src_fwd_index0 16415181
trg_fwd_index0 19001975
grouped_src 93832
grouped_trg 255323


In [ ]:
fopen=open(en_ar_corpus_combined_fname)
fopen.seek(592)
print([fopen.readline()])
fopen.close()

['Australia . 3\tاستراليا . 4\n']


#Extract word and phrase pairs - Sept 2022

In [ ]:
import time, shelve
en_ar_corpus_combined_fname="UNv1.0.en-ar-combined.txt"
# excluded_src_tokens=["the","a","an","and","of","its","his","her","to","is","are","am","been", "have","has","had", "in", "on", "at", "with", "from","for","that","he","she","it","would"] #we will need to update this
# excluded_trg_tokens=["من","على","في","مع", "إلى","له","تم","إلى","في"]


t0=time.time()
cur_params={}
cur_params["excluded_words"]=excluded_src_tokens+excluded_trg_tokens
cutoff_ratio=0.01




fopen=open(en_ar_corpus_combined_fname)
src_shelve_fpath="test-src.shelve"
trg_shelve_fpath="test-trg.shelve"
src_shelve_fopen=shelve.open(src_shelve_fpath)
trg_shelve_fopen=shelve.open(trg_shelve_fpath)

test_word="main"
test_out=src_shelve_fopen.get(test_word,[])
all_lines=[]
src_line_ids=[]
for a in test_out[:1000]:
  cur_line_id=int(a)
  fopen.seek(cur_line_id)
  cur_line=fopen.readline()
  all_lines.append(cur_line)
  src_line_ids.append(cur_line_id)
src_line_ids=list(set(src_line_ids))

all_lines.sort(key=lambda x:len(x))
for cur_line in all_lines[:50]:
  line_split=cur_line.split("\t")
  if len(line_split)!=2: continue
  src0,trg0=line_split
  src_tokens0,trg_tokens0=tok(src0),tok(trg0)
  filtere_src0=filter_toks(src_tokens0,cur_params)
  filtere_trg0=filter_toks(trg_tokens0,cur_params)
  print(src0,trg0)
  candidates=[]
  for ft in filtere_trg0:
    cur_trg_line_ids=[int(v) for v in trg_shelve_fopen.get(ft,[])]
    cur_trg_line_ids=list(set(cur_trg_line_ids))
    intersection0=list(set(src_line_ids).intersection(set(cur_trg_line_ids)))
    ratio=len(intersection0)/(len(src_line_ids)+len(cur_trg_line_ids)-len(intersection0))
    if ratio<cutoff_ratio: continue
    candidates.append((ft,len(intersection0), round(ratio,4)))
    #print(ft, ratio) # len(cur_trg_line_ids),cur_trg_line_ids[:5]
  candidates.sort(key=lambda x:-x[-1])
  for c in candidates:
    print(c)
  print("----")

src_shelve_fopen.close()
trg_shelve_fopen.close()

t1=time.time()
elapsed=t1-t0
# for cur_line in all_lines[:50]:
#   print([cur_line])
#   line_split=cur_line.split("\t")
#   if len(line_split)!=2: continue
#   src0,trg0=line_split
#   src_tokens0,trg_tokens0=tok(src0),tok(trg0)
#   cur_params={}
#   cur_params["excluded_words"]=excluded_src_tokens+excluded_trg_tokens
#   #print(fopen.tell())
#   filtere_src0=filter_toks(src_tokens0,cur_params)
#   filtere_trg0=filter_toks(trg_tokens0,cur_params)
#   print("filtere_src0",filtere_src0)
#   print("filtere_trg0",filtere_trg0)
#   print("-----")

print("test_out",len(test_out),test_out[:10],"elapsed",elapsed)
fopen.close()

Main Committee اللجنة الرئيسية

('الرئيسية', 480, 0.0464)
----
Satellite main-hub معدات السواتل

----
Main publications: المنشورات الرئيسية

('الرئيسية', 480, 0.0464)
----
A. Main assumptions ألف - الافتراضات الرئيسية

('الرئيسية', 480, 0.0464)
----
Summary of main findings موجز النتائج الرئيسية

('الرئيسية', 480, 0.0464)
----
Their main functions are: ومهامها الأساسية كما يلي:

----
and by main determining factor والعامل المحدد الرئيسي

('الرئيسي', 207, 0.0566)
('والعامل', 14, 0.0137)
----
Main diplomatic experience الأنشطة الدبلوماسية الرئيسية

('الرئيسية', 480, 0.0464)
----
(b) Main trunking contract - )ب( عقد مقسم الهاتف الرئيسي

('الرئيسي', 207, 0.0566)
----
Main trunking contract - )ب( عقد كابل الاتصالات الرئيسي -

('الرئيسي', 207, 0.0566)
----
(b) Main trunking contract - العقد الرئيسي للخطوط الهاتفية

('الرئيسي', 207, 0.0566)
----
B. Main international treaties باء - المعاهدات الدولية الرئيسية

('الرئيسية', 480, 0.0464)
----
Representation on the Main Committee التمثيل في اللجن

In [ ]:
def get_phrase(phrase_tokens0,index0,max_sent_tokens=1000):
  cur_token_indexes0=index0.get(phrase_tokens0[0],[])
  cur_token_indexes0=[v*max_sent_tokens for v in cur_token_indexes0]
  #print(phrase_tokens0[0], len(cur_token_indexes0))
  for t_i in range(1,len(phrase_tokens0)):
    tmp_token=phrase_tokens0[t_i]
    tmp_indexes=index0.get(tmp_token,[])
    #print(tmp_token, len(tmp_indexes), tmp_indexes[:10])
    tmp_indexes_offset=[v*max_sent_tokens-t_i for vi,v in enumerate(tmp_indexes)]
    #print(tmp_indexes_offset[:10])
    cur_token_indexes0=list(set(tmp_indexes_offset).intersection(cur_token_indexes0))
  return list(set([int(v/max_sent_tokens) for  v in cur_token_indexes0]))

fopen=open(en_ar_corpus_combined_fname)
src_shelve_fpath="test-src.shelve"
trg_shelve_fpath="test-trg.shelve"
src_shelve_fopen=shelve.open(src_shelve_fpath)
trg_shelve_fopen=shelve.open(trg_shelve_fpath)

test_phrase="catastrophic"
test_phrase_tokens=[v.strip() for v in test_phrase.split() if v.strip()]
#test_out=src_shelve_fopen.get(test_word,[])
test_out=get_phrase(test_phrase_tokens,src_shelve_fopen)
all_lines=[]
src_line_ids=[]
for a in test_out[:1000]:
  cur_line_id=int(a)
  fopen.seek(cur_line_id)
  cur_line=fopen.readline()
  all_lines.append(cur_line)
  src_line_ids.append(cur_line_id)
src_line_ids=list(set(src_line_ids))

all_lines.sort(key=lambda x:len(x))
for cur_line in all_lines[:50]:
  line_split=cur_line.split("\t")
  if len(line_split)!=2: continue
  src0,trg0=line_split
  src_tokens0,trg_tokens0=tok(src0),tok(trg0)
  filtere_src0=filter_toks(src_tokens0,cur_params)
  filtere_trg0=filter_toks(trg_tokens0,cur_params)
  print(src0,trg0)
  candidates=[]
  for fi,ft in enumerate(filtere_trg0):
    cur_trg_line_ids=[int(v) for v in trg_shelve_fopen.get(ft,[])]
    cur_trg_line_ids=list(set(cur_trg_line_ids))
    intersection0=list(set(src_line_ids).intersection(set(cur_trg_line_ids)))
    ratio=len(intersection0)/(len(src_line_ids)+len(cur_trg_line_ids)-len(intersection0))
    if ratio>cutoff_ratio: candidates.append((ft,len(intersection0), round(ratio,4)))
    
    if fi==0: continue
    pair=(filtere_trg0[fi-1],ft)
    cur_pair_index_ids=get_phrase(pair,trg_shelve_fopen)
    pair_intersection0=list(set(src_line_ids).intersection(set(cur_pair_index_ids)))
    pair_ratio=len(pair_intersection0)/(len(src_line_ids)+len(cur_pair_index_ids)-len(pair_intersection0))
    if ratio>cutoff_ratio: candidates.append((pair,len(pair_intersection0), round(pair_ratio,4)))


    #print(ft, ratio) # len(cur_trg_line_ids),cur_trg_line_ids[:5]
  candidates.sort(key=lambda x:-x[-1])
  for c in candidates:
    print(c)
  print("----")

src_shelve_fopen.close()
trg_shelve_fopen.close()

t1=time.time()
elapsed=t1-t0
# for cur_line in all_lines[:50]:
#   print([cur_line])
#   line_split=cur_line.split("\t")
#   if len(line_split)!=2: continue
#   src0,trg0=line_split
#   src_tokens0,trg_tokens0=tok(src0),tok(trg0)
#   cur_params={}
#   cur_params["excluded_words"]=excluded_src_tokens+excluded_trg_tokens
#   #print(fopen.tell())
#   filtere_src0=filter_toks(src_tokens0,cur_params)
#   filtere_trg0=filter_toks(trg_tokens0,cur_params)
#   print("filtere_src0",filtere_src0)
#   print("filtere_trg0",filtere_trg0)
#   print("-----")

print("test_out",len(test_out),test_out[:10],"elapsed",elapsed)
fopen.close()

There is destruction of catastrophic proportions in the city. وتعاني المدينة من دمار رهيب.

('دمار', 3, 0.0175)
('رهيب', 2, 0.0165)
(('المدينة', 'دمار'), 1, 0.0088)
(('دمار', 'رهيب'), 1, 0.0088)
----
The humanitarian situation in the city remains catastrophic. ولا تزال الحالة الإنسانية في تلك المدينة مفجعة.

('مفجعة', 6, 0.0395)
(('المدينة', 'مفجعة'), 1, 0.0088)
----
In order to avoid catastrophic consequences, we demand: وتفاديا للكوارث التي ستحدث من جراء ذلك، فإننا نطلب ما يلي:

----
(b) Inhuman conditions prevail, with potentially catastrophic consequences. )ب( تسود ظروف غير انسانية يمكن أن تؤدي إلى عواقب وخيمة؛

('وخيمة', 12, 0.0494)
(('عواقب', 'وخيمة'), 5, 0.0281)
('عواقب', 9, 0.0155)
(('تؤدي', 'عواقب'), 1, 0.0086)
----
The fire business was particularly exposed to insolvency because of catastrophic events. فتعرض نشاط التأمين من الحريق بوجه خاص للإعسار بسبب الكوارث.

----
57. Field studies of catastrophic floods are made by UNESCO scientific missions. 57 - وتقوم البعثات العلمية لل